## 第8章 文埋め込み

### 文埋め込みモデルの実装

#### 教師なしSimCSEの実装

In [1]:
from transformers.trainer_utils import set_seed

# 乱数シードの設定
set_seed(42)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### データセットの読み込みと前処理

In [2]:
from datasets import load_dataset

unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

Using custom data configuration default
Reusing dataset jawiki-sentences (/root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc)


In [3]:
# 訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [4]:
# 訓練セットの中身を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [5]:
# 訓練セットから空白行を事例を削除
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-ecc223a34f1ffe88.arrow


In [6]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐために、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-e9e6caaca5ef995f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-9484a91dba79bc01.arrow


In [7]:
# 前処理後の訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [8]:
# 前処理後の訓練セットの内容を確認
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [9]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)
Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)


#### トークナイザとcollate関数の準備

In [10]:
from transformers import AutoTokenizer

base_model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [11]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [12]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデル準備

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)
        # パラメータをメモリ上に隣接した形で配置
        # これを実行しない場合、モデルの保存でエラーになることがある
        # for param in model.parameters():
        #     param.data = param.data.contiguous()
        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

#### Trainerの準備

In [14]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).correlation

    return {"spearman": spearman}

In [15]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="../model/outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=256,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=256,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to="none",  # 外部ツールへのログを無効化
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# !pip install transformers[torch]

In [17]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

####  訓練の実行

In [77]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,0.000400,2.331667,0.752163
500,0.000500,2.402906,0.755473
750,0.000400,2.385067,0.753259
1000,0.000300,2.366536,0.757948
1250,0.000400,2.350700,0.759100
1500,0.000800,2.320323,0.760896
1750,0.000400,2.332596,0.759427
2000,0.000200,2.334367,0.757139
2250,0.000300,2.300360,0.758956
2500,0.000100,2.297671,0.758454


TrainOutput(global_step=3907, training_loss=0.00033567894401759603, metrics={'train_runtime': 1629.5441, 'train_samples_per_second': 613.669, 'train_steps_per_second': 2.398, 'total_flos': 0.0, 'train_loss': 0.00033567894401759603, 'epoch': 1.0})

#### 性能評価

In [78]:
# 検証セットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.3203227519989014,
 'eval_spearman': 0.7608961210671525,
 'eval_runtime': 10.878,
 'eval_samples_per_second': 1144.601,
 'eval_steps_per_second': 4.504,
 'epoch': 1.0}

In [79]:
# テストセットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.1774518489837646,
 'eval_spearman': 0.7878933714566898,
 'eval_runtime': 1.3524,
 'eval_samples_per_second': 1077.307,
 'eval_steps_per_second': 4.436,
 'epoch': 1.0}

In [80]:
#### トークナイザの保存とモデルの保存
encoder_path = "../model/outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('../model/outputs_unsup_simcse/encoder/tokenizer_config.json',
 '../model/outputs_unsup_simcse/encoder/special_tokens_map.json',
 '../model/outputs_unsup_simcse/encoder/vocab.txt',
 '../model/outputs_unsup_simcse/encoder/added_tokens.json')

#### 教師ありSimCSEの実装

In [18]:
# 乱数シードの設定を行う
from transformers.trainer_utils import set_seed

set_seed(42)

#### データセットの読み込みと前処理

In [19]:
from datasets import load_dataset
# JSNLIの訓練セットを読み込む
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

Using custom data configuration default
Reusing dataset jsnli (/root/.cache/huggingface/datasets/llm-book___jsnli/default/1.0.0/b59ac9cb188ddb68dc451bae1b33ac9ebe501a5d2c41f17c5ec06ad0621186d7)


In [20]:
# JSNLIの訓練セットの形式と事例数を確認
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [21]:
from pprint import pprint

# JSNLIの訓練セットの内容を確認
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [22]:
import csv
import random
from typing import Iterator

# JSNLI訓練セットから、前提文とラベルごとに仮説文をまとめたdictを作成する
premise2hypotheses = {}

primises = jsnli_dataset["premise"] # 前提文
hypotheses = jsnli_dataset["hypothesis"] # 仮説文
labels = jsnli_dataset["label"] # ラベル

for premise, hypothesis, label in zip(primises, hypotheses, labels):
    if premise not in premise2hypotheses:
        premise2hypotheses[premise] = {
            "entailment": [],
            "neutral": [],
            "contradiction": []
        }
        
    premise2hypotheses[premise][label].append(hypothesis)

In [23]:
def generate_sup_train_example():
    """
    教師ありSimCSEの訓練セットを生成する関数
    Returns:
        dict: 'premise', 'entailment_hypothesis', 'contradiction_hypothesis'をキーとする辞書
    """
    # 結果を格納する辞書を初期化
    dataset_dict = {
        "premise": [],
        "entailment_hypothesis": [],
        "contradiction_hypothesis": []
    }
    
    # JSNLIのデータから三つ組を生成
    for premise, hypotheses in premise2hypotheses.items():
        # 「矛盾」ラベルの仮説文が存在しない場合はスキップ
        if len(hypotheses["contradiction"]) == 0:
            continue
        
        # 「含意」ラベルの仮説文それぞれに対して処理
        for entailment_hypothesis in hypotheses["entailment"]:
            # 「矛盾」ラベルの仮説文をランダムに選択
            contradiction_hypothesis = random.choice(hypotheses["contradiction"])
            
            # 各要素をリストに追加
            dataset_dict["premise"].append(premise)
            dataset_dict["entailment_hypothesis"].append(entailment_hypothesis)
            dataset_dict["contradiction_hypothesis"].append(contradiction_hypothesis)
    
    return dataset_dict

# データセットの作成
try:
    dataset_dict = generate_sup_train_example()
    sup_train_dataset = Dataset.from_dict(dataset_dict)
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

In [24]:
# 訓練セットの形式と事例数を確認
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [25]:
# 訓練セットの内容を確認
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


#### collate関数の準備

In [26]:
from torch import Tensor
from transformers import BatchEncoding

def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """訓練セットのミニバッチを作成"""
    premises = []
    hypotheses = []
    
    for example in examples:
        premises.append(example["premise"])
        
        entailment_hypotheis = example["entailment_hypothesis"]
        contradiction_hypothesis = example["contradiction_hypothesis"]
        
        hypotheses.extend(
            [entailment_hypotheis, contradiction_hypothesis]
        )
    # ミニバッチに含まれる前提文と仮説文にトークナイザを適用
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    
    # 前提文と仮説文の類似度行列における正例ペアの位置を示すTensorを作成
    # 行列にi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)
    
    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

#### モデルの準備

In [27]:
# 教師ありSimCSEモデルを初期化
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

#### Trainerの準備

In [28]:
# 教師ありSimCSEに訓練のハイパーパラメータを設定
sup_training_args = TrainingArguments(
    output_dir="../model/outputs_sup_simcse",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=5e-5,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=250,
    logging_steps=250,
    save_steps=250,
    save_total_limit=1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="spearman",
    remove_unused_columns=False,
    report_to="none",  # 外部ツールへのログを無効化
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
# 教師ありSimCSEのTrainerを初期化
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [30]:
# 教師ありSimCSEの訓練を行う
sup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,1.438000,2.782808,0.791591
500,1.100000,2.710751,0.780901
750,1.017300,2.759371,0.786594
1000,0.967000,2.818005,0.788974
1250,0.926100,2.781399,0.798985
1500,0.828600,2.828457,0.792996
1750,0.762300,2.907998,0.801895
2000,0.752000,2.885943,0.800020
2250,0.743200,2.904130,0.793232
2500,0.724700,2.885217,0.796730


TrainOutput(global_step=4065, training_loss=0.8097005076953846, metrics={'train_runtime': 1370.6597, 'train_samples_per_second': 379.608, 'train_steps_per_second': 2.966, 'total_flos': 0.0, 'train_loss': 0.8097005076953846, 'epoch': 3.0})

#### 性能評価

In [31]:
# 検証セットで教師ありSimCSEのモデル評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.9079980850219727,
 'eval_spearman': 0.8018950489894365,
 'eval_runtime': 10.6323,
 'eval_samples_per_second': 1171.049,
 'eval_steps_per_second': 9.217,
 'epoch': 3.0}

In [32]:
# テストセットで教師ありSimCSEのモデル評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.628929376602173,
 'eval_spearman': 0.8184395733197469,
 'eval_runtime': 1.3935,
 'eval_samples_per_second': 1045.534,
 'eval_steps_per_second': 8.611,
 'epoch': 3.0}

### 最近傍探索ライブラリFaissを使った検索

#### Faissを利用した最近傍探索の実装

In [33]:
# !pip install faiss-cpu==1.7.4

In [34]:
# !pip install numpy

#### データセットの読み込みと前処理

In [35]:
from datasets import load_dataset

paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)

Using custom data configuration default
Reusing dataset jawiki-paragraphs (/root/.cache/huggingface/datasets/llm-book___jawiki-paragraphs/default/1.0.0/0f2d7acd99ad7ae0615fd07442dbd1654d37c5d60a39fc720efe28acff3f86f8)


In [36]:
# 段落データ形式と事例数を確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [5]:
from pprint import pprint

# 段落データの内容を確認
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [6]:
# 段落データのうち、各記事の最初の段落のみを使う
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

100%|██████████| 9669/9669 [01:29<00:00, 107.46ba/s]


#### トークナイザとモデルの準備

In [8]:
# !pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.8 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.10/dist-packages/pip/_v

In [10]:
# !pip install transformers[ja,sentencepice]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 23.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556721 sha256=463b39e4a7d2e5a1ee1e993f15b0e8a75aab651ef3a4095e62cd9946f240fff7
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vj8u2b7/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=57ad393d8dfdfb5afff7d3c562756db80d26982fbb4067ba3f4033bb4030523e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vj8u2b7/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf

In [11]:
from transformers import AutoModel, AutoTokenizer

model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

In [13]:
# 読み込んだモデルをGPUのメモリに移動
device = "cuda:0"
encoder = encoder.to(device)

#### モデルによる埋め込みの計算

In [16]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)
    
    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]
            
    # ベクトルをNuPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [17]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

100%|██████████| 1340/1340 [21:01<00:00,  1.06ba/s]


In [18]:
# 埋め込みを付与した段落データの形式と事例数を確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [19]:
# 埋め込みを計算した段落データの内容を確認
pprint(paragraph_dataset[0])

{'embeddings': [0.042540159076452255,
                -0.04196695238351822,
                -0.032314036041498184,
                0.018254535272717476,
                -0.06702621281147003,
                -0.06091688573360443,
                -0.05339366942644119,
                0.00584664149209857,
                0.005609563086181879,
                0.0004761741147376597,
                0.0544019490480423,
                -0.03015177696943283,
                -0.01541396789252758,
                -0.09760468453168869,
                0.031523678451776505,
                0.0070890034548938274,
                0.004207753110677004,
                -0.01840118132531643,
                -0.07030576467514038,
                0.00973212718963623,
                0.006157499272376299,
                -0.03276313096284866,
                -0.00840569194406271,
                -0.023154262453317642,
                0.051235586404800415,
                0.0434039942920208,
              

In [21]:
# 埋め込みを付与した段落データをディスクに保存
paragraph_dataset.save_to_disk(
    "../model/outputs_unsup_simcse/embedded_paragraphs"
)

#### Faissによる最近傍探索を試す

In [32]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

ImportError: You must install Faiss to use FaissIndex. To do so you can run `conda install -c pytorch faiss-cpu` or `conda install -c pytorch faiss-gpu`. A community supported package is also available on pypi: `pip install faiss-cpu` or `pip install faiss-gpu`. Note that pip may not have the latest version of FAISS, and thus, some of the latest features and bug fixes may not be available.

In [2]:
import re
from html.parser import HTMLParser
from html import unescape

class HTMLToMarkdownConverter(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.markdown = []
        self.list_stack = []
        self.list_item_count = 0
        self.code_block = False
        self.emphasis = False
        self.strong = False
        
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)
        
        if tag == 'h1':
            self.markdown.append('# ')
        elif tag == 'h2':
            self.markdown.append('## ')
        elif tag == 'h3':
            self.markdown.append('### ')
        elif tag == 'h4':
            self.markdown.append('#### ')
        elif tag == 'h5':
            self.markdown.append('##### ')
        elif tag == 'h6':
            self.markdown.append('###### ')
        elif tag == 'p':
            if self.markdown and self.markdown[-1] != '\n':
                self.markdown.append('\n\n')
        elif tag == 'br':
            self.markdown.append('\n')
        elif tag == 'strong' or tag == 'b':
            self.strong = True
            self.markdown.append('**')
        elif tag == 'em' or tag == 'i':
            self.emphasis = True
            self.markdown.append('*')
        elif tag == 'code':
            if 'class' in attrs:
                self.code_block = True
                self.markdown.append(f"\n```{attrs['class']}\n")
            else:
                self.markdown.append('`')
        elif tag == 'pre':
            if not self.code_block:
                self.code_block = True
                self.markdown.append('\n```\n')
        elif tag == 'ul':
            self.list_stack.append('*')
        elif tag == 'ol':
            self.list_stack.append('1')
            self.list_item_count = 0
        elif tag == 'li':
            indent = '  ' * (len(self.list_stack) - 1)
            if self.list_stack[-1] == '*':
                self.markdown.append(f'\n{indent}* ')
            else:
                self.list_item_count += 1
                self.markdown.append(f'\n{indent}{self.list_item_count}. ')
        elif tag == 'blockquote':
            self.markdown.append('\n> ')
            
    def handle_endtag(self, tag):
        if tag == 'p':
            self.markdown.append('\n\n')
        elif tag == 'strong' or tag == 'b':
            self.strong = False
            self.markdown.append('**')
        elif tag == 'em' or tag == 'i':
            self.emphasis = False
            self.markdown.append('*')
        elif tag == 'code':
            if self.code_block:
                self.code_block = False
                self.markdown.append('\n```\n\n')
            else:
                self.markdown.append('`')
        elif tag == 'pre':
            if self.code_block:
                self.code_block = False
                self.markdown.append('\n```\n\n')
        elif tag in ['ul', 'ol']:
            self.list_stack.pop()
            if not self.list_stack:
                self.markdown.append('\n')
        elif tag == 'blockquote':
            self.markdown.append('\n\n')
            
    def handle_data(self, data):
        if data.strip():
            self.markdown.append(data.strip())
            
    def convert(self, html):
        self.feed(html)
        markdown = ''.join(self.markdown)
        # 複数の空行を1つにまとめる
        markdown = re.sub(r'\n\s*\n', '\n\n', markdown)
        # 前後の余分な改行を削除
        markdown = markdown.strip()
        return markdown

def convert_html_to_markdown(html):
    """
    HTMLをMarkdownに変換する関数
    
    Parameters:
        html (str): 変換するHTML文字列
    
    Returns:
        str: 変換されたMarkdown文字列
    """
    converter = HTMLToMarkdownConverter()
    return converter.convert(html)


In [3]:
html_example = """

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="UTF-8">
<title>お手続きの流れ（お借換え）｜住宅ローン｜りそな銀行・埼玉りそな銀行</title>
<meta name="description" content="お手続の流れ（お借換え）。りそなの住宅ローンにお借換えいただく際の、お手続きの流れをご紹介！月々の返済額や返済期間など、住宅ローンのプランにお悩みの場合は、いつでもお気軽にご相談ください！">
<meta name="keywords" content="住宅ローン,銀行,りそな,審査,融資,お借換え">
<meta name="viewport" content="width=device-width,minimum-scale=1.0,maximum-scale=1.0,user-scalable=0">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="format-detection" content="telephone=no">
<link rel="canonical" href="https://www.resonabank.co.jp/kojin/jutaku/procedure2.html">
<link href="/apple-touch-icon.png" rel="apple-touch-icon-precomposed" type="image/png">
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon">
<link href="/favicon.ico" rel="icon" type="image/x-icon">
<link rel="stylesheet" href="/common/type2018/css/reset.css">
<link rel="stylesheet" href="/common/type2018/css/base.css">
<link rel="stylesheet" href="/common/type2018/css/base_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/parts.css">
<link rel="stylesheet" href="/common/type2018/css/parts_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/font.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_renewal2019.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_rwd_renewal2019.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_renewal2020.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_rwd_renewal2020.css">
<script src="/common/type2018/js/jquery-3.5.1.min.js"></script>
<script src="/common/type2018/js/modernizr.custom.39326.js"></script>
<script src="/common/type2018/js/jquery.plugin_jq3.js"></script>
<script src="/common/type2018/js/common_jq3.js"></script>
<script src="/common/type2018/js/drawer.rb.js"></script>
<script src="/common/type2018/js/component_jq3.js"></script>
<script src="/common/type2018/js/common_page_united.js"></script>
<script src="/kojin/jutaku/js/jutaku_renewal2019.js"></script>
<script src="https://cdn-blocks.karte.io/4e588d08afda9814698a46572afbf27c/builder.js"></script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-5FQFWZZ');</script>
<!-- End Google Tag Manager -->

<!-- print CSS-->
<link rel="stylesheet" href="/common/type2018/css/print.css" media="print">
<!-- FAQ LOADLIST -->
<link rel="stylesheet" href="/kojin/faq/css/faq_load.css">
<script src="/kojin/faq/js/faq_load_jq3.js"></script>
<script type="text/javascript">
	window.pageSettings = {};
	window.pageSettings.CATEGORY_NAV_CURRENT = '/kojin/kariru.html';
</script>
<!-- Google Tag Manager RB -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-MCPCNC');</script>
<!-- End Google Tag Manager RB -->

<!-- Google Tag Manager SR -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-N44PMJ');</script>
<!-- End Google Tag Manager SR -->
</head>
<body class="architectureType2018">
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-5FQFWZZ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->

<!-- Google Tag Manager (noscript) RB -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-MCPCNC"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) RB -->

<!-- Google Tag Manager (noscript) SR -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-N44PMJ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) SR -->
<div id="drawerMenu">
	<div data-select-bank="りそな銀行" aria-hidden="false"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=rb_unite&loc=1sp"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=rb_unite&loc=1sp">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="/kojin/tsumitatebox/?loc=1sp" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="/kojin/toshin/?loc=1sp">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="/kojin/fundwrap/?loc=1sp" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=rb_unite&loc=1sp">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=rb_unite&loc=1sp">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="/kojin/kinsen/?loc=1sp">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/gaika/?bank=rb_unite&loc=1sp">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="/kojin/shikin_unyo/?loc=1sp">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="/kojin/taishoku_unyo/?loc=1sp"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="/kojin/kokusai/?loc=1sp">個人向け国債・国債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=rb_unite&loc=1sp">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=rb_unite&loc=1sp">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=1sp">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=1sp">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=rb_unite&loc=1sp">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=rb_unite&loc=1sp">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=1sp">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?loc=1sp">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=rb_unite&loc=1sp">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=rb_unite&loc=1sp"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="/kojin/apaman/?loc=1sp"><span>アパート・マンションローン</span></a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=1sp"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=rb_unite&loc=1sp">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="/kojin/souzokusyoukei.html?loc=1sp">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="/kojin/zaisankanri.html?loc=1sp">財産管理業務</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="/kojin/private/?loc=1sp">プライベートバンキング業務</a></li>
											<li class="kojin04Menu07 menuIcon01" role="treeitem"><a href="/nenkin/ideco/?loc=1sp">iDeCo（個人型確定拠出年金）</a></li>
											<li class="kojin04Menu08 menuIcon01" role="treeitem"><a href="/nenkin/401k/?loc=1sp">企業型確定拠出年金</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="/direct/?loc=1sp">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=rb_unite&loc=1sp">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">コメント：電話マークアイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=rb_unite&loc=1sp">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="/kojin/exchange/gaika/?loc=1sp">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/kaigai/soukin/?loc=1sp"><span>海外送金</span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=rb_unite&loc=1sp">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=rb_unite&loc=1sp">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=1sp">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="/kojin/koueikyougi/?loc=1sp"><span>即時口座決済</span></a></li>
											<li class="kojin05Menu11 menuIcon01" role="treeitem"><a href="/kojin/yoyaku/?loc=1sp"><span>来店予約サービス</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="/hojin/?loc=1sp">法人のお客さま</a></li>
			<li><a href="/nenkin/?loc=1sp">企業年金・iDeCoのお客さま</a></li>
			<li><a href="/about/?loc=1sp">りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<div data-select-bank="埼玉りそな銀行" aria-hidden="true"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=sr_unite">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=sr_unite">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=sr_unite">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
											<li class="kojin04Menu04 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
											<li class="kojin04Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=sr_unite">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">電話アイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span>海外送金</span><span class="iconWrap"><span class="iconCCobaltGreen icoWAuto">NEW</span></span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
			<li><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<header role="banner">
		<div id="selectBank">
	<div id="selectBankInner">
		<dl>
			<dt class="selectBankTtl">ご利用中(予定)の銀行を選択してください</dt>
			<dd>
				<ul class="selectBankBtnList" role="tablist">
					<li class="select selectBankBtnWrap" data-select-bank="りそな銀行" role="tab" aria-selected="true" aria-label="りそな銀行">
						<a id="selectBankBtnResona" href="javascript:;" class="btnT01 selectBankBtn" tabindex="1"><img src="/common/images/logo.svg" id="svgSelectResona" alt="りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="false"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="false"><span>現在表示している銀行</span></span>
					</li>
					<li class="selectBankBtnWrap" data-select-bank="埼玉りそな銀行" role="tab" aria-selected="false" aria-label="埼玉りそな銀行">
						<a id="selectBankBtnSaitama" href="javascript:;" class="btnT01 selectBankBtn" tabindex="2"><img src="/common/images/logo_header_saitama_mark.svg" id="svgSelectSaitama" alt="埼玉りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="true"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="true"><span>現在表示している銀行</span></span>
					</li>
				</ul>
			</dd>
		</dl>
		<div class="closeWrap">
			<a href="#" class="btnT01 glyphClose" id="closeSelectBank" tabindex="3"><span>閉じる</span></a>
		</div>
	</div>
</div>

		<div data-select-bank="りそな銀行" aria-hidden="false">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
					<p class="spContent"><a href="/sp/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="1">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menusp.shtml" target="_blank" class="glyphLogin"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/" target="_blank" class="glyphGps"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="/">個人のお客さま</a></li>
					<li class="nav02"><a href="/hojin/">法人のお客さま</a></li>
					<li class="nav03"><a href="/nenkin/">企業年金・iDeCoのお客さま</a></li>
					<li class="nav04"><a href="/about/">りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="1">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索" id="qHead">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=rb_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=rb_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=rb_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/gaika/?bank=rb_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="/kojin/kokusai/"><span>個人向け国債・国債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=rb_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=rb_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=rb_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10" role="treeitem"><a href="/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=rb_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=rb_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=rb_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=rb_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=rb_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink navDropImg navDropImg01 clear">
									<dl>
										<dt><a href="/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=rb_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu05"><a href="/kojin/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu07"><a href="/nenkin/ideco/"><span>iDeCo（個人型確定<br>拠出年金）</span></a></li>
												<li class="kojin04Menu08"><a href="/nenkin/401k/"><span>企業型確定拠出年金</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=rb_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>コメント：電話マークアイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=rb_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/kaigai/soukin/"><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=rb_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=rb_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
												<li class="kojin05Menu11"><a href="/kojin/yoyaku/"><span>来店予約サービス</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="https://www.saitamaresona.co.jp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
					<p class="spContent"><a href="https://www.saitamaresona.co.jp/sp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="https://www.saitamaresona.co.jp/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="5">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menusp.shtml" class="glyphLogin" target="_blank"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" class="glyphGps" target="_blank"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="https://www.saitamaresona.co.jp/">個人のお客さま</a></li>
					<li class="nav02"><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
					<li class="nav04"><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="5">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="https://www.saitamaresona.co.jp/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=sr_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=sr_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/"><span>個人向け国債・国債・地方債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=sr_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=sr_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10"><a href="https://www.saitamaresona.co.jp/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=sr_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=sr_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=sr_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=sr_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=sr_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=sr_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu04"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/"><span>きょういく信託</span></a></li>
												<li class="kojin04Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu06"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html"><span>iDeCo<br>(個人型確定拠出年金)</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="https://www.saitamaresona.co.jp/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>電話アイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=sr_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span class="dropLinkNewIcon01">New</span><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=sr_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=sr_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
	</header>
	<!-- //#header -->
	<div class="topicPath" itemscope itemtype="http://schema.org/BreadcrumbList" data-select-bank="りそな銀行" aria-hidden="false">
		<div class="topicPathInner">
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/"><span itemprop="name">銀行ホーム</span></a><meta itemprop="position" content="1" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/kariru.html?bank=rb_unite"><span itemprop="name">お金を借りたい</span></a><meta itemprop="position" content="2" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/jutaku/?bank=rb_unite"><span itemprop="name">住宅ローン（新規）</span></a><meta itemprop="position" content="3" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/jutaku/procedure2.html?bank=rb_unite"><span itemprop="name">お手続きの流れ（お借換え）</span></a><meta itemprop="position" content="4" /></span>
		</div>
	</div>
	<div class="topicPath" itemscope itemtype="http://schema.org/BreadcrumbList" data-select-bank="埼玉りそな銀行" aria-hidden="true">
		<div class="topicPathInner">
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="https://www.saitamaresona.co.jp/"><span itemprop="name">銀行ホーム</span></a><meta itemprop="position" content="1" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/kariru.html?bank=sr_unite"><span itemprop="name">お金を借りたい</span></a><meta itemprop="position" content="2" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/jutaku/?bank=sr_unite"><span itemprop="name">住宅ローン（新規）</span></a><meta itemprop="position" content="3" /></span>
			<span itemprop="itemListElement" itemscope itemtype="http://schema.org/ListItem"><a itemprop="item" href="/kojin/jutaku/procedure2.html?bank=sr_unite"><span itemprop="name">お手続きの流れ（お借換え）</span></a><meta itemprop="position" content="4" /></span>
		</div>
	</div>
	<!-- #topicPath -->

	<div id="wrap">
		<div id="mainArea">
			<main>
				<section>
					<div class="coverTitle">
						<div class="coverTitleInner">
							<h1 class="h1Title01"><span class="titleInner">お手続きの流れ（お借換え）</span></h1>
							<p class="coverTitleLead" data-select-bank="りそな銀行" aria-hidden="false">りそな銀行の住宅ローンにお借換えいただく際の、お手続きの流れをご紹介します。月々の返済額や返済期間など、住宅ローンのプランにお悩みの方もいつでも気軽にご相談いただけます！</p>
							<p class="coverTitleLead" data-select-bank="埼玉りそな銀行" aria-hidden="true">埼玉りそな銀行の住宅ローンにお借換えいただく際の、お手続きの流れをご紹介します。月々の返済額や返済期間など、住宅ローンのプランにお悩みの方もいつでも気軽にご相談いただけます！</p>
						</div><!-- /.coverTitleInner -->
					</div><!-- //.coverTitle -->

					<div class="mainInner">
						<div class="mainContents">
							<div class="procedure_layout01 v2-bgGreen01">
								<div class="layoutColumnC2_1">
									<div class="column03">
										<div class="v3-itemBalloonA01" data-select-bank="りそな銀行" aria-hidden="false">
											<p class="tBold fs18_20 spFs16 tCenter">月々の返済額や返済期間など、<br class="spContent">プランを見直すチャンスです！<br>りそな銀行の口座がない方でも<br class="spContent">是非ご相談ください</p>

											<div class="tCenter mt10">
												<a href="/kojin/tenpo_atm/landing01/?jutaku" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 mb00"><span>99％のお客さまから<br class="spContent">「相談してよかった！」の声</span></a>
											</div>
										</div>
										<div class="v3-itemBalloonA01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<p class="mt35 mb30 spMt00 spMb00 tBold fs18_20 spFs16 tCenter">月々の返済額や返済期間など、<br class="spContent">プランを見直すチャンスです！<br>埼玉りそな銀行の口座がない方でも<br class="spContent">是非ご相談ください</p>
<!--
											<div class="tCenter mt10">
												<a href="/kojin/tenpo_atm/landing01/?jutaku" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 mb00"><span>99％のお客さまから<br class="spContent">「相談してよかった！」の声</span></a>
											</div>
-->
										</div>
									</div>
									<div class="column04">
										<div class="layoutColumnC2_2 w330">
											<div class="w110 spW25p">
												<figure class="pt40 spPr20 tLeft"><img src="/kojin/jutaku/images/risonya_02.png" alt=""></figure>
											</div>
											<div class="w220 spW70p">
												<p class="jutaku-itemBalloonA01 bottomLeft fs18_16">借換えるとどれくらいおトクになるのでしょうか？</p>
											</div>
										</div>
									</div>
								</div>
							</div>

							<ul class="layoutColumnA2_2 mt40 mlrAuto w900 procedure_layout02">
								<li class="blockInnerB01">
									<p>
										<a href="/kojin/jutaku/procedure1.html?bank=rb_unite" class="btnT01 btnSMiddle btnWFull glyphArrowTriangleRight overT01" data-select-bank="りそな銀行" aria-hidden="false"><span>新規の場合</span></a>
										<a href="/kojin/jutaku/procedure1.html?bank=sr_unite" class="btnT01 btnSMiddle btnWFull glyphArrowTriangleRight overT01" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>新規の場合</span></a>
									</p>
								</li>
								<li class="blockInnerB01 tCenter">
									<span class="btnT02 btnSMiddle btnWFull"><span>お借換えの場合</span></span>
								</li>
							</ul>

							<section>
								<h2 class="h2Title01"><span class="titleInner">お借換えまでの流れ</span></h2>

								<div class="layoutColumnC4_1 procedure_step">
									<div class="tCenter">
										<a href="#step1">
											<p class="text01">STEP1</p>
											<figure class="pcContent"><img src="/kojin/jutaku/images/risonya_04.png" alt=""></figure>
											<p class="text02 glyphArrowCircleBottom"><span>事前審査</span></p>
										</a>
									</div>
									<div class="tCenter">
										<a href="#step2">
											<p class="text01">STEP2</p>
											<figure class="pcContent"><img src="/kojin/jutaku/images/risonya_05.png" alt=""></figure>
											<p class="text02 glyphArrowCircleBottom"><span>正式審査</span></p>
										</a>
									</div>
									<div class="tCenter procedureComment">
										<p class="v3-itemBalloonA01 v3-balloonBottomArrow w100p fs18_16 spMt15 spW240"><span class="tOrange01">電子契約</span>で手続き</p>
										<a href="#step3">
											<p class="text01">STEP3</p>
											<figure class="pcContent"><img src="/kojin/jutaku/images/risonya_06.png" alt=""></figure>
											<p class="text02 glyphArrowCircleBottom"><span>ご契約手続</span></p>
										</a>
									</div>
									<div class="tCenter">
										<a href="#step4">
											<p class="text01">STEP4</p>
											<figure class="pcContent"><img src="/kojin/jutaku/images/risonya_07.png" alt=""></figure>
											<p class="text02 glyphArrowCircleBottom"><span>お借入れ</span></p>
										</a>
									</div>
								</div>

								<div class="colorCardA03 mt30 spMt15">
									<div class="cardInner tCenter">
										<a href="/kojin/jutaku/documents.html?bank=rb_unite" class="btnT01 btnSMiddle overT01 glyphMemo02 mb00 w320" data-select-bank="りそな銀行" aria-hidden="false"><span>必要書類の確認</span></a>
										<a href="/kojin/jutaku/documents.html?bank=sr_unite" class="btnT01 btnSMiddle overT01 glyphMemo02 mb00 w320" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>必要書類の確認</span></a>
									</div>
								</div>

								<section id="step1">
									<h3 class="h3Title01 v2-hasTextLabel procedure_title"><span class="titleInner"><span class="textLabel">STEP1</span><span class="textToLabel">事前審査のお申込み</span></span></h3>
									<p>お借換えにご関心がございましたら、まずは事前審査をお申込みください。</p>
									<p>インターネットなら、24時間いつでも簡易な審査をお申込みいただけます。</p>
									<ul class="listMemo01 mt10">
										<li><span class="listMark">※</span>ご入力・アップロードされました内容に基づいて、当社並びに<a href="/kojin/jutaku/modal_help01.html" data-modal="ha05" class="iconQuestion01 v3-jsModal01">保証会社<span></span></a>において簡易な審査を行います。</li>
									</ul>
									<div class="mt20 tCenter" data-select-bank="りそな銀行" aria-hidden="false">
										<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle w500 spW100p iconNewWindow"><span>Webで事前審査</span></a>
									</div>
									<div class="mt20 tCenter" data-select-bank="埼玉りそな銀行" aria-hidden="true">
										<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle w500 spW100p iconNewWindow"><span>Webで事前審査</span></a>
									</div>
								</section>

								<section id="step2">
									<h3 class="h3Title01 v2-hasTextLabel procedure_title procedure_title01">
										<span class="titleInner"><span class="textLabel">STEP2</span><span class="textToLabel">正式審査のお申込み</span></span>
										<span class="figure pcContent"><img src="/kojin/jutaku/images/risonya_10.png" alt=""></span>
									</h3>
									<p>りそな住宅ローンマイページにて、正式審査のお申込みが可能です。<br>ご提出いただいた書類に基づいて、当社ならびに<a href="/kojin/jutaku/modal_help01.html" data-modal="ha05" class="iconQuestion01 v3-jsModal01">保証会社<span></span></a>において審査いたします。</p>
									<ul class="listMemo01 mt10">
										<li class="mb00"><span class="listMark">※</span>審査にあたり、書類の内容等についてお問合せすることがございます。</li>
										<li class="mb00"><span class="listMark">※</span>追加資料のご提出をお願いする場合もあります。</li>
									</ul>
									<div class="v2-colorCardA06 mt30 spMt20">
										<div class="cardInner">
											<p class="glyphLamp01 spTop"><span>お借入中の金融機関に対し、<a href="/kojin/jutaku/modal_help01.html" data-modal="sa08" class="iconQuestion01 v3-jsModal01">全額繰上返済<span></span></a>のお手続きを行なっていただきます。</span></p>
										</div>
									</div>
									<div class="v2-colorCardA06 mt30 spMt20">
										<div class="cardInner">
											<p class="glyphLamp01"><span>ご契約の前に当社の住宅ローンについてご説明した
												<a href="/kojin/jutaku/guide/" data-select-bank="りそな銀行" aria-hidden="false"><span>お手続きガイド（動画）</span></a>
												<a href="https://www.saitamaresona.co.jp/kojin/jutaku/guide/index.html" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>お手続きガイド（動画）</span></a>を必ずご視聴ください。ご契約時の所要時間を短縮できます。<br>ご契約のお手続き時に、ご不明点についてご説明させていただきます。</span>
											</p>
										</div>
									</div>
								</section>

								<section id="step3">
									<h3 class="h3Title01 v2-hasTextLabel procedure_title"><span class="titleInner"><span class="textLabel">STEP3</span><span class="textToLabel">ご契約手続き</span></span></h3>
									<div class="blockInnerB03">
										<div>
											<p>Web上で契約書に電子署名を行い契約を締結します。</p>
											<p>別途司法書士等同席のもと、ご融資対象物件（土地と建物）への<a href="/kojin/jutaku/modal_help01.html" data-modal="ta05" class="v3-jsModal01 iconQuestion01"><span>抵当権</span></a>設定のご契約が必要です。</p>
										</div>
										<div class="tCenter"><img src="/kojin/jutaku/images/common_img03.png" alt="電子契約に対しての声"></div>
										<div class="newAdjustment">
											<ul class="listMemo01 fsSmall tDefault01 spTLeft">
												<li><span class="listMark">※</span>お借入金額が5,000万円超1億円以下の場合は6万円、1,000万円超5,000万円以下の場合は2万円です。また別途、電子契約サービス手数料（Web完結型電子契約：5,500円、店頭型電子契約：11,000円［いずれも消費税等込］）を申し受けます。</li>
												<li><span class="listMark">※</span>Web環境がご利用できない場合は、書面でもご契約ができます。</li>
											</ul>
										</div>
										<div class="tCenter marginLvH3" data-select-bank="りそな銀行" aria-hidden="false"><a href="/kojin/jutaku/econtract.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01"><span>詳しくみる</span></a></div>
										<div class="tCenter marginLvH3" data-select-bank="埼玉りそな銀行" aria-hidden="true"><a href="/kojin/jutaku/econtract.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01"><span>詳しくみる</span></a></div>
									</div>
								</section>

								<section id="step4">
									<h3 class="h3Title01 v2-hasTextLabel procedure_title"><span class="titleInner"><span class="textLabel">STEP4</span><span class="textToLabel">お借入れ</span></span></h3>
									<p>金利はお借入日時点のものが適用されます。</p>
								</section>
							</section>
						</div>
					</div><!-- //.mainInner -->

<div class="v2-bgYellow01 v3_fullColumnA01 conversionAreaA01">
	<div class="columnInner v2-cardBlockA03">
		<div class="layoutColumnA2_2 mlrAuto w900">
			<div class="tCenter">
				<p class="v4-decorationTextB02 convText01 fs18_20 spFs16 tBold mb15 spMb10 js-sptile01"><span>30秒で診断！</span></p>
				<a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" onclick="wa_rb_jutaku_sim_kae();" oncontextmenu="wa_rb_jutaku_sim_kae();" target="_blank" class="btnT02 btnSMiddle w430 js-sptile02 karte-hoverBtn01" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローン<br class="spContent">シミュレーション</span></a>
				<a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" onclick="wa_sr_jutaku_sim_kae();" oncontextmenu="wa_sr_jutaku_sim_kae();" target="_blank" class="btnT02 btnSMiddle w430 js-sptile02" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローン<br class="spContent">シミュレーション</span></a>
			</div>
			<div class="tCenter">
				<p class="v4-decorationTextB02 fs18_20 spFs16 tBold mb15 spMb10 js-sptile01"><span>審査結果も<br class="spContent">Webで確認！</span></p>
				<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle w430 iconNewWindow js-sptile02" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
				<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle w430 iconNewWindow js-sptile02" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
			</div>
		</div>

		<div class="detail mt30 spMt00">
			<div class="tCenter">
				<p class="tBold fs18_20 spFs16">ご検討中の方は<br class="spContent">まずはお気軽にご相談ください</p>
				<p class="v4-decorationTextB02 tOrange01 fs18_26 spFs18 tBold mb15">
					<span data-select-bank="りそな銀行" aria-hidden="false">土・日・祝日や17時以降も営業</span>
					<span data-select-bank="埼玉りそな銀行" aria-hidden="true">土・日・祝日も営業</span>
				</p>
				<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&freeWord=%E3%82%BB%E3%83%96%E3%83%B3%E3%83%87%E3%82%A4%E3%82%BA#A03" onclick="this.href=_wa.link(this.href);wa_rb_sevendays_soudan();" oncontextmenu="wa_rb_sevendays_soudan();" target="_blank" class="btnT03 btnSMiddle w430 iconNewWindow karte-hoverBtn02" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで相談予約</span></a>
				<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT03 btnSMiddle w430 iconNewWindow" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで相談予約</span></a>
				<div class="w430 mlrAuto tLeft">
					<a href="/kojin/tenpo_atm/" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="glyphArrowTriangleRight js-tenpoAtm iconNewWindow"><span>店舗を検索</span></a>
					<a href="/kojin/tenpo_atm/" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="wa_sr_kouza_shop_button();" oncontextmenu="wa_sr_kouza_shop_button();" target="_blank" class="glyphArrowTriangleRight js-tenpoAtm iconNewWindow"><span>店舗を検索</span></a>
				</div>
			</div>
		</div>
	</div>
</div>


					<div class="mainInner">
						<div class="mainContents">
							<div class="column1">
								<div class="columnInner">
									<div class="col" data-select-bank="りそな銀行" aria-hidden="false">
										<section>
	<div class="v3-columnNaviTitle01">
		<h2 class="h2Title01 noBorder mb00 pb00"><span class="titleInner"><a href="/kojin/column/loan/?page=jutakuloan">住宅ローンのコラム</a></span></h2>
		<p class="v3-columnNaviTitleLink"><a href="/kojin/column/loan/?page=jutakuloan" class="glyphArrowTriangleRight"><span>一覧へ</span></a></p>
	</div>
	<ul class="v3-columnNaviBlockA01">
		<li class="v3-columnNaviItem01">
			<a href="/kojin/column/jutaku/column_0012.html">
				<div class="img"><img src="/kojin/column/images/jutaku_img_01.png" alt=""></div>
				<div class="text">
					<p>最低限揃えておきたい防災グッズとは？災害に備えて補償は必要？</p>
				</div>
			</a>
		</li>
		<li class="v3-columnNaviItem01">
			<a href="/kojin/column/jutaku/column_0013.html">
				<div class="img"><img src="/kojin/column/images/jutaku_img_02.png" alt=""></div>
				<div class="text">
					<p>火災保険・地震保険だけで大丈夫？いざというときに備えておきたい災害保険</p>
				</div>
			</a>
		</li>
		<li class="v3-columnNaviItem01">
			<a href="/kojin/column/jutaku/column_0014.html">
				<div class="img"><img src="/kojin/column/images/jutaku_img_03.png" alt=""></div>
				<div class="text">
					<p>【2024年版】住宅ローン控除（減税）とは？仕組みと改正ポイント・計算方法</p>
				</div>
			</a>
		</li>
		<li class="v3-columnNaviItem01">
			<a href="/kojin/column/jutaku/column_0015.html">
				<div class="img"><img src="/kojin/column/images/jutaku_img_04.png" alt=""></div>
				<div class="text">
					<p>リバースモーゲージとは？仕組みやメリット・デメリットを詳しく解説</p>
				</div>
			</a>
		</li>
	</ul>
</section>

									</div><!-- /.col -->
								</div><!-- /.columnInner -->
							</div><!-- /.column1 -->

<div class="column1">
	<div class="columnInner">
		<div class="col">
			<aside class="relevanceBlock01">
				<div class="blockInner">
					<h2 class="blockTitle">関連情報</h2>
					<div class="blockDetail">
						<ul class="layoutColumnA3_1 fs18_17 spFs16">
							<li>
								<dl class="detailList01">
									<dt class="title">住宅ローンの商品</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/?bank=rb_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a></li>
											<li><a href="/kojin/jutaku/karikae/?bank=rb_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a></li>
											<li><a href="/kojin/jutaku.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="/kojin/jutaku/?bank=sr_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a></li>
											<li><a href="/kojin/jutaku/karikae/?bank=sr_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a></li>
											<li><a href="/kojin/jutaku.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">特典・プラン</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/lin/" class="glyphArrowTriangleRight"><span>りそな女性向け住宅ローン『凛next』</span></a></li>
											<li><a href="/kojin/jutaku/zeh/?bank=rb_unite" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a></li>
											<li><a href="/kojin/jutaku/sumikae/?bank=rb_unite" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="https://www.saitamaresona.co.jp/kojin/jutaku/lin/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな女性向け住宅ローン『凛next』</span></a></li>
											<li><a href="/kojin/jutaku/zeh/?bank=sr_unite" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a></li>
											<li><a href="/kojin/jutaku/sumikae/?bank=sr_unite" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">団信・火災</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/danshin/?bank=rb_unite" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a></li>
											<li><a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a></li>
											<li><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a></li>
											<li><a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>がん保障特約</span></a></li>
											<li><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>ペア団信</span></a></li>
											<li><a href="/kojin/jutaku/danshin/danshin.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a></li>
											<li><a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a></li>
											<li><a href="/kojin/jutaku/shizen/?bank=rb_unite" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="/kojin/jutaku/danshin/?bank=sr_unite" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a></li>
											<li><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a></li>
											<li><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a></li>
											<li><a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>がん保障特約</span></a></li>
											<li><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>ペア団信</span></a></li>
											<li><a href="/kojin/jutaku/danshin/danshin.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a></li>
											<li><a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a></li>
											<li><a href="/kojin/jutaku/shizen/?bank=sr_unite" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">金利・シミュレーション</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/loan_viewer.html" class="glyphArrowTriangleRight"><span>金利一覧</span></a></li>
											<li><a href="/kojin/jutaku/simulation/shinki/?bank=rb_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a></li>
											<li><a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="https://www.saitamaresona.co.jp/kojin/loan_viewer.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>金利一覧</span></a></li>
											<li><a href="/kojin/jutaku/simulation/shinki/?bank=sr_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a></li>
											<li><a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">お手続き・諸費用など</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/procedure1.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a></li>
											<li><a href="/kojin/jutaku/procedure2.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a></li>
											<li><a href="/kojin/jutaku/fee/?bank=rb_unite" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a></li>
											<li><a href="/kojin/jutaku/documents.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a></li>
											<li><a href="/kojin/jutaku/pair_loan.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>ペアローン</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="/kojin/jutaku/procedure1.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a></li>
											<li><a href="/kojin/jutaku/procedure2.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a></li>
											<li><a href="/kojin/jutaku/fee/?bank=sr_unite" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a></li>
											<li><a href="/kojin/jutaku/documents.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a></li>
											<li><a href="/kojin/jutaku/pair_loan.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>ペアローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">お申込み・審査</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a></li>
											<li><a href="https://www.jl.resona-gr.co.jp/fmweb/login" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで正式審査申込（りそな住宅ローンマイページ）</span></a></li>
											<li><a href="/kojin/jutaku/econtract.html?bank=rb_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a></li>
											<li><a href="https://navi.resona-gr.co.jp/jutaku/login?bank=rb_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a></li>
											<li><a href="https://www.jl.resona-gr.co.jp/fmweb/login" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで正式審査申込（りそな住宅ローンマイページ）</span></a></li>
											<li><a href="/kojin/jutaku/econtract.html?bank=sr_unite" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a></li>
											<li><a href="https://navi.resona-gr.co.jp/jutaku/login?bank=sr_unite" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">はじめての方</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/column/jutaku/myhome_first.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>はじめてのマイホーム&amp;住宅ローン</span></a></li>
											<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>マイホームToDoリスト</span></a></li>
											<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新生活必要なものリスト</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="/kojin/column/jutaku/myhome_first.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>はじめてのマイホーム&amp;住宅ローン</span></a></li>
											<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>マイホームToDoリスト</span></a></li>
											<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新生活必要なものリスト</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="detailList01">
									<dt class="title">用語集・Q&amp;A</dt>
									<dd class="detail">
										<ul class="linkList04" data-select-bank="りそな銀行" aria-hidden="false">
											<li><a href="/kojin/jutaku/yougo/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>住宅ローンの用語集</span></a></li>
											<li><a href="/kojin/jutaku/faq/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a></li>
										</ul>
										<ul class="linkList04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
											<li><a href="/kojin/jutaku/yougo/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>住宅ローンの用語集</span></a></li>
											<li><a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</div>
				</div>
			</aside>
		</div><!-- //.col -->
	</div><!-- //.columnInner -->
</div>


						</div>
					</div><!-- //.mainInner -->
					<div class="v3_fullColumnA01 v2-bgPink01">
						<div class="columnInner">
							<div class="blockInnerB03">
	<div class="blockInnerB05">
		<div class="blockInnerB02 tCenter">
			<div class="dTable vMiddle w58p spW100p mlrAuto">
				<div class="dTcell w16p"><img src="/kojin/jutaku/images/cv_img_01.png" alt=""></div>
				<div class="dTcell w84p">
					<p class="v3-itemBalloonA01 v3-balloonLeftArrow tBold spTLeft ml10">人生最大の買い物なのに、”保障の数”だけで選んでいませんか？</p>
				</div>
			</div>
			<p class="tBold"><span class="v2-textMarker01"><span class="fsXXL">医療のプロが選ぶ<span class="fs40_20 spFs14">※1</span><br class="spContent">こだわりの団信はこちら</span></span></p>
			<ul class="dankakuColumn02 w80p spW100p mlrAuto">
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_01.png" alt="" class="w50p spW48"></span></div>
					<p class="text fsExtraS spFs10">がん</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_02.png" alt="" class="w50p spW48"></span></div>
					<p class="text fsExtraS spFs10">心筋梗塞</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_03.png" alt="" class="w35p spW32"></span></div>
					<p class="text fsExtraS spFs10">脳卒中</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_04.png" alt="" class="w50p spW48"></span></div>
					<p class="text fsExtraS spFs10">死亡</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_05.png" alt="" class="w50p spW48"></span></div>
					<p class="text fsExtraS spFs10">高度障害</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_06.png" alt="" class="w50p spW48"></span></div>
					<p class="text fsExtraS spFs10">病気・ケガ<br>による16の状態</p>
				</li>
				<li>
					<div class="alignMiddle tile24"><span><img src="/kojin/jutaku/images/cv_ico_07.png" alt="" class="w35p spW32"></span></div>
					<p class="text fsExtraS spFs10">所定の<br>要介護状態</p>
				</li>
			</ul>
		</div>
		<div class="blockInnerB01">
			<div class="blockInnerB01 tCenter">
				<p class="v4-decorationTextB02 fsMaximum tBold"><span><span class="tOrange01">２人に１人が選ぶ団信</span><span class="fs24_16 spFs14 tNormal">※2</span></span></p>
				<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="btnT02 btnSMiddle glyphArrowTriangleRight w440 mb00 karte-hoverBtn03"><span>団信革命を詳しくみる</span></a>
				<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="btnT02 btnSMiddle glyphArrowTriangleRight w440 mb00"><span>団信革命を詳しくみる</span></a>
			</div>
			<div class="blockInnerB01 newAdjustment">
				<ul class="listMemo02 fsSmall tLeft">
					<li><span class="listNum">※1</span>2019年4月～10月の期間において、りそな銀行の「団信革命」ご利用者の業種で最も多いのが医療従事者です。（りそな銀行業種別データによる）</li>
					<li><span class="listNum">※2</span>2019年4月～10月の期間において、りそな銀行で「団信革命」をご利用可能な方のうち実際にお申込みいただいた割合</li>
				</ul>
				<p class="fsSmall">（注）詳細は<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite#anchor01" data-select-bank="りそな銀行" aria-hidden="false">商品概要</a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite#anchor01" data-select-bank="埼玉りそな銀行" aria-hidden="true">商品概要</a>をご確認ください。</p>
			</div>
		</div>
	</div>
	<p class="tRight">（登）C19E6196(2019.12.26)</p>
</div>

						</div>
					</div>
				</section>
			</main>
		</div><!-- //#mainArea -->
	</div><!-- //#wrap -->
	<footer role="contentinfo">
				<a href="#top" id="pageTop"><img src="/common/images/go_top.svg" alt="ページトップへ戻る" class="svgTop"></a>
		<div class="snsArea">
			<!-- PC ONLY CONTENTS START-->
			<dl>
				<dt><a href="https://www.resona-gr.co.jp/holdings/other/sns/" target="_blank">りそなソーシャルメディアネットワーク</a></dt>
				<dd>
					<ul class="footerSnsList">
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><figure><img src="/common/images/footer_sns_tw02.gif" alt="X"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><figure><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><figure><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><figure><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><figure><img src="/common/images/footer_sns_line02.gif" alt="Line"></figure></a></li>
					</ul>
				</dd>
			</dl>
			<!-- PC ONLY CONTENTS END-->
			<!-- SP ONLY CONTENTS START-->
			<ul class="spFooterSnsList">
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><img src="/common/images/footer_sns_tw02.gif" alt="X"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><img src="/common/images/footer_sns_line02.gif" alt="Line"></a></li>
			</ul>
			<!-- SP ONLY CONTENTS END-->
		</div><!-- //.snsArea -->

		<div data-select-bank="りそな銀行" aria-hidden="false">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">
			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=rb_unite&loc=7">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=rb_unite&loc=7"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=rb_unite&loc=7">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/?loc=7" target="_blank">つみたてボックス</a></li>
						<li><a href="/kojin/toshin/?loc=7">投資信託</a></li>
						<li><a href="/kojin/fundwrap/?loc=7" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=rb_unite&loc=7">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=rb_unite&loc=7">定期預金</a></li>
						<li><a href="/kojin/kinsen/?loc=7">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=rb_unite&loc=7">外貨預金</a></li>
						<li><a href="/kojin/shikin_unyo/?loc=7">資金運用プラン</a></li>
						<li><a href="/kojin/taishoku_unyo/?loc=7">退職金コース</a></li>
						<li><a href="/kojin/kokusai/?loc=7">個人向け国債・国債</a></li>
						<li><a href="/kojin/gaisai/?loc=7">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=rb_unite&loc=7">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=rb_unite&loc=7">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=7">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=7">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=rb_unite&loc=7">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=rb_unite&loc=7">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=7">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=rb_unite&loc=7">マイカーローン</a></li>
						<li><a href="/kojin/edu/?loc=7">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=rb_unite&loc=7">リフォームローン</a></li>
						<li><a href="/kojin/apaman/?loc=7">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=7">リバースモーゲージローン</a></li>
						<li><a href="/kojin/akiya/?loc=7">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/sonaeru.html?loc=7">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=rb_unite&loc=7">保険</a></li>
						<li><a href="/kojin/souzokusyoukei.html?loc=7">相続・承継</a></li>
						<li><a href="/kojin/zaisankanri.html?loc=7">財産管理業務</a></li>
						<li><a href="/kojin/private/?loc=7">プライベートバンキング業務</a></li>
						<li><a href="/nenkin/ideco/?loc=7">iDeCo(個人型確定拠出年金)</a></li>
						<li><a href="/nenkin/401k/?loc=7">企業型確定拠出年金</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/tsukau.html?loc=7">便利にお得に使いたい</a></h2>
					<ul>
						<li><a href="/direct/?loc=7">マイゲート</a></li>
						<li><a href="/kojin/app/?bank=rb_unite&loc=7"><span>スマートフォンアプリ</span></a></li>
						<li><a href="/club/?bank=rb_unite&loc=7">りそなクラブ</a></li>
						<li><a href="/kojin/exchange/gaika/?loc=7">外貨両替</a></li>
						<li><a href="/kojin/kaigai/soukin/?loc=7">海外送金</a></li>
						<li><a href="/kojin/visa_debit/?bank=rb_unite&loc=7">Visaデビットカード</a></li>
						<li><a href="/kojin/credit/?bank=rb_unite&loc=7">クレジットカード</a></li>
						<li><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=7">ペイジー料金払込サービス</a></li>
						<li><a href="/kojin/koueikyougi/?loc=7">即時口座決済</a></li>
						<li><a href="/kojin/yoyaku/?loc=7">来店予約サービス</a></li>
						<li><a href="/kojin/column/index2.html?loc=7">お金に関わるりそなのコラム</a></li>
						<li><a href="https://community.resonabank.co.jp/?loc=7" target="_blank">お金のあれこれ</a></li>
					</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="/"><span>個人のお客さま</span></a></li>
				<li><a href="/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="/nenkin/"><span>企業年金のお客さま</span></a></li>
				<li><a href="/about/"><span>りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="1">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com" target="_blank">採用情報</a></li>
					<li><a href="/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="/kojin/contact/#10">指定紛争解決機関</a></li>
					<li><a href="/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0010</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>近畿財務局長(登金)第3号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会 一般社団法人金融先物取引業協会 一般社団法人日本クレジット協会 一般社団法人日本投資顧問業協会</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">

			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=sr_unite">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=sr_unite"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/" target="_blank">つみたてボックス</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
						<li><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=sr_unite">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/">退職金コース</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/gaisai/">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=sr_unite">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=sr_unite">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
						<li><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=sr_unite">リフォームローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=sr_unite">リバースモーゲージローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/akiya/">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html">便利にお得に使いたい</a></h2>
						<ul>
							<li><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
							<li><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
							<li><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/">海外送金</a></li>
							<li><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
							<li><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/">即時口座決済</a></li>
						</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav saitamaFooterLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="https://www.saitamaresona.co.jp/"><span>個人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/about/"><span>埼玉りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="5">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="https://www.saitamaresona.co.jp/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com/" target="_blank">採用情報</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="https://www.saitamaresona.co.jp/kojin/contact/#13">指定紛争解決機関</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社埼玉りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0017</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>関東財務局長(登金)第593号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会&nbsp;&nbsp;&nbsp;一般社団法人金融先物取引業協会&nbsp;&nbsp;&nbsp;一般社団法人日本クレジット協会</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>信託契約代理業&nbsp;:</dt>
							<dd>登録番号　関東財務局長（代信）第22号</dd>
						</dl>
						<dl>
							<dt>所属信託会社の商号&nbsp;:</dt>
							<dd>株式会社りそな銀行</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Saitama Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
	</footer>
	<!-- FOOTER FIXED CONTENT START-->
	<script type="text/javascript" charset="UTF-8" src="https://mobi.lineomni.jp/web/mobi-agent-client-frame-loader.min.js?domainId=resonagr"></script>
<script type="text/javascript" charset="UTF-8" src="/kojin/contact/chat/decsupport/js/ds-jutaku_two.js"></script>
	<!-- HUMANCHAT -->
<div>
	<div class="fixedBottom01 fixedBottomChat01 jsFixedBottom01 jsFixedBottomChat">
		<div class="fixedBottomInner">
			<div class="fixedBottomContent">
				<div class="fixedBottomNav">
					<ul class="fixedBottomList01">
						<li>
							<a data-select-bank="りそな銀行" aria-hidden="false" href="/kojin/jutaku/modal/modal_menu.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
							<a data-select-bank="埼玉りそな銀行" aria-hidden="true" href="/kojin/jutaku/modal/modal_menu_saitama.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
						</li>
						<li><a href="javascript:void(0)" class="fixedBottomIconFaq01 spTCenter jsChatMenuTrg"><span class="fsSmall spFs10">お困りの方</span></a></li>
					</ul>
					<div class="fixedBottomChatMenu01 jsFixedBottomChatMenu">
						<ul class="chatMenuBlock01 tCenter">
							<li class="chatAbled rb_unite" data-select-bank="りそな銀行" aria-hidden="false">
								<a href="/kojin/contact/chat/ds/jutaku.html" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_03.svg" alt="有人チャット"></div>
									<div class="tBold fs18_13 spFs10"><span>有人チャット</span></div>
								</a>
							</li>
							<li>
								<a href="https://ct.resona-chatbot.jp/?site_id=7" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_02.svg" alt="AIチャット"></div>
									<div class="tBold fs18_13 spFs10"><span>AIチャット</span></div>
								</a>
							</li>
							<li class="chatFaqLink">
								<a href="/kojin/jutaku/faq/" data-select-bank="りそな銀行" aria-hidden="false">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
								<a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" data-select-bank="埼玉りそな銀行" aria-hidden="true">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
							</li>
						</ul>
					</div>
				</div>
				<div class="fixedBottomBtn">
					<ul class="fixedBottomList01">
						<li>
							<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">来店予約</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">来店予約</span></a>
						</li>
						<li>
							<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
						</li>
					</ul>
				</div>
			</div>
		</div>
	</div><!-- //.fixedBottom01 .jsFixedBottom01 -->
</div>

	<!-- FOOTER FIXED CONTENT END-->
</div>
<!-- 計測タグここから -->
<script>
(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','//www.google-analytics.com/analytics.js','ga');

<!--RB用 -->
ga('create', 'UA-45901352-1', 'auto');
ga('send', 'pageview');

<!--SR用 -->
ga('create', 'UA-45926600-1', 'auto', {'name':'sr'});
ga('sr.send', 'pageview');

</script>

<!--WebAntenna/RB -->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WATVaj-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WATVaj-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/RB -->

<!-- WebAntenna/SR-->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WAbSBh-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WAbSBh-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/SR-->
<!-- 計測タグここまで -->
<!-- WebAntenna -->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
function wa_rb_jutaku_shinsa_1() {
 if ( typeof( webantenna ) == 'function' ) {
  _wa = new WA_Processor();
  _wa.account = 'WATVaj-1';
  _wa.cv = 'rb_jutaku_shinsa_1';
  webantenna( true );
  var wait = function( msec ) {
   var s = ( new Date() ).getTime();
   while ( ( new Date ).getTime() - s < msec );
  }
  wait( 100 );
 }
}
</script>
<!-- end WebAntenna -->
<script type="text/javascript" src="/kojin/js/le-jutakuloan.js"></script>
<script src="/kojin/jutaku/js/jutaku_webantenna.js"></script>
</body>
</html>
"""

In [14]:
html_example = """

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="UTF-8">
<title>団体信用生命保険（団信）｜りそな銀行・埼玉りそな銀行</title>
<meta name="description" content="りそなの団体信用生命保険（団信）をご紹介いたします。りそなは所定のがんのみをカバーするがん保障特約から3大疾病（所定のがん、急性心筋梗塞・脳卒中による所定の状態）はもちろん、病気やケガによる所定の状態・所定の要介護状態など、様々なリスクをカバーする特約（団信革命）まであなたにあった保障タイプが選べます。">
<meta name="keywords" content="団体信用生命保険,団信,特約,住宅ローン,りそな">
<meta name="viewport" content="width=device-width,minimum-scale=1.0,maximum-scale=1.0,user-scalable=0">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="format-detection" content="telephone=no">
<link rel="canonical" href="https://www.resonabank.co.jp/kojin/jutaku/danshin/">
<link href="/apple-touch-icon.png" rel="apple-touch-icon-precomposed" type="image/png">
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon">
<link href="/favicon.ico" rel="icon" type="image/x-icon">
<link rel="stylesheet" href="/common/type2018/css/reset.css">
<link rel="stylesheet" href="/common/type2018/css/base.css">
<link rel="stylesheet" href="/common/type2018/css/base_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/parts.css">
<link rel="stylesheet" href="/common/type2018/css/parts_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/font.css">
<link rel="stylesheet" href="/common/type2018/css/yougo_load.css">
<link rel="stylesheet" href="/kojin/jutaku/css/parts_ver4.css">
<link rel="stylesheet" href="/kojin/jutaku/css/parts_ver4_rwd.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_renewal2023.css">
<link rel="stylesheet" href="/kojin/jutaku/danshin/css/danshin_renewal2023.css">
<script src="/common/type2018/js/jquery-3.5.1.min.js"></script>
<script src="/common/type2018/js/modernizr.custom.39326.js"></script>
<script src="/common/type2018/js/jquery.plugin_jq3.js"></script>
<script src="/common/type2018/js/common_jq3.js"></script>
<script src="/common/type2018/js/drawer.rb.js"></script>
<script src="/common/type2018/js/component_jq3.js"></script>
<script src="/common/type2018/js/common_page_united.js"></script>
<script src="/common/type2018/js/yougo_load_jq3.js"></script>
<script src="/kojin/jutaku/js/component_ver4_jq3.js"></script>
<script src="/kojin/jutaku/danshin/js/danshin_renewal2023.js"></script>
<!-- 合言葉 -->
<link rel="stylesheet" href="/kojin/jutaku/wallpaper/css/wallpaper_word.css">
<script src="/kojin/jutaku/wallpaper/js/wallpaper_word.js"></script>
<!-- // 合言葉 -->
<script src="https://cdn-blocks.karte.io/4e588d08afda9814698a46572afbf27c/builder.js"></script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-5FQFWZZ');</script>
<!-- End Google Tag Manager -->

<!-- print CSS-->
<link rel="stylesheet" href="/common/type2018/css/print.css" media="print">
<!-- FAQ LOADLIST -->
<link rel="stylesheet" href="/kojin/faq/css/faq_load.css">
<script src="/kojin/faq/js/faq_load_jq3.js"></script>
<!-- Google Tag Manager RB -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-MCPCNC');</script>
<!-- End Google Tag Manager RB -->

<!-- Google Tag Manager SR -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-N44PMJ');</script>
<!-- End Google Tag Manager SR -->
<script type="text/javascript">
	window.pageSettings = {};
	window.pageSettings.CATEGORY_NAV_CURRENT = '/kojin/kariru.html';
</script>
<script type="application/ld+json">
	{
		"@context": "https://schema.org",
		"@type": "BreadcrumbList",
		"itemListElement": [{
			"@type": "ListItem",
			"position": 1,
			"name": "銀行ホーム",
			"item": "https://www.resonabank.co.jp/"
		},{
			"@type": "ListItem",
			"position": 2,
			"name": "お金を借りたい",
			"item": "https://www.resonabank.co.jp/kojin/kariru.html"
		},{
			"@type": "ListItem",
			"position": 3,
			"name": "住宅ローン（新規）",
			"item": "https://www.resonabank.co.jp/kojin/jutaku/"
		},{
			"@type": "ListItem",
			"position": 4,
			"name": "団体信用生命保険（団信）",
			"item": "https://www.resonabank.co.jp/kojin/jutaku/danshin/"
		}]
	}
</script>
</head>
<body class="architectureType2018">
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-5FQFWZZ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->

<!-- Google Tag Manager (noscript) RB -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-MCPCNC"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) RB -->

<!-- Google Tag Manager (noscript) SR -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-N44PMJ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) SR -->
<div id="drawerMenu">
	<div data-select-bank="りそな銀行" aria-hidden="false"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=rb_unite&loc=1sp"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=rb_unite&loc=1sp">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="/kojin/tsumitatebox/?loc=1sp" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="/kojin/toshin/?loc=1sp">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="/kojin/fundwrap/?loc=1sp" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=rb_unite&loc=1sp">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=rb_unite&loc=1sp">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="/kojin/kinsen/?loc=1sp">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/gaika/?bank=rb_unite&loc=1sp">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="/kojin/shikin_unyo/?loc=1sp">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="/kojin/taishoku_unyo/?loc=1sp"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="/kojin/kokusai/?loc=1sp">個人向け国債・国債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=rb_unite&loc=1sp">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=rb_unite&loc=1sp">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=1sp">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=1sp">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=rb_unite&loc=1sp">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=rb_unite&loc=1sp">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=1sp">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?loc=1sp">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=rb_unite&loc=1sp">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=rb_unite&loc=1sp"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="/kojin/apaman/?loc=1sp"><span>アパート・マンションローン</span></a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=1sp"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=rb_unite&loc=1sp">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="/kojin/souzokusyoukei.html?loc=1sp">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="/kojin/zaisankanri.html?loc=1sp">財産管理業務</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="/kojin/private/?loc=1sp">プライベートバンキング業務</a></li>
											<li class="kojin04Menu07 menuIcon01" role="treeitem"><a href="/nenkin/ideco/?loc=1sp">iDeCo（個人型確定拠出年金）</a></li>
											<li class="kojin04Menu08 menuIcon01" role="treeitem"><a href="/nenkin/401k/?loc=1sp">企業型確定拠出年金</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="/direct/?loc=1sp">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=rb_unite&loc=1sp">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">コメント：電話マークアイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=rb_unite&loc=1sp">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="/kojin/exchange/gaika/?loc=1sp">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/kaigai/soukin/?loc=1sp"><span>海外送金</span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=rb_unite&loc=1sp">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=rb_unite&loc=1sp">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=1sp">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="/kojin/koueikyougi/?loc=1sp"><span>即時口座決済</span></a></li>
											<li class="kojin05Menu11 menuIcon01" role="treeitem"><a href="/kojin/yoyaku/?loc=1sp"><span>来店予約サービス</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="/hojin/?loc=1sp">法人のお客さま</a></li>
			<li><a href="/nenkin/?loc=1sp">企業年金・iDeCoのお客さま</a></li>
			<li><a href="/about/?loc=1sp">りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<div data-select-bank="埼玉りそな銀行" aria-hidden="true"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=sr_unite">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=sr_unite">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=sr_unite">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
											<li class="kojin04Menu04 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
											<li class="kojin04Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=sr_unite">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">電話アイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span>海外送金</span><span class="iconWrap"><span class="iconCCobaltGreen icoWAuto">NEW</span></span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
			<li><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<header role="banner">
		<div id="selectBank">
	<div id="selectBankInner">
		<dl>
			<dt class="selectBankTtl">ご利用中(予定)の銀行を選択してください</dt>
			<dd>
				<ul class="selectBankBtnList" role="tablist">
					<li class="select selectBankBtnWrap" data-select-bank="りそな銀行" role="tab" aria-selected="true" aria-label="りそな銀行">
						<a id="selectBankBtnResona" href="javascript:;" class="btnT01 selectBankBtn" tabindex="1"><img src="/common/images/logo.svg" id="svgSelectResona" alt="りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="false"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="false"><span>現在表示している銀行</span></span>
					</li>
					<li class="selectBankBtnWrap" data-select-bank="埼玉りそな銀行" role="tab" aria-selected="false" aria-label="埼玉りそな銀行">
						<a id="selectBankBtnSaitama" href="javascript:;" class="btnT01 selectBankBtn" tabindex="2"><img src="/common/images/logo_header_saitama_mark.svg" id="svgSelectSaitama" alt="埼玉りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="true"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="true"><span>現在表示している銀行</span></span>
					</li>
				</ul>
			</dd>
		</dl>
		<div class="closeWrap">
			<a href="javascript:;" class="btnT01 glyphClose" id="closeSelectBank" tabindex="3"><span>閉じる</span></a>
		</div>
	</div>
</div>

		<div data-select-bank="りそな銀行" aria-hidden="false">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
					<p class="spContent"><a href="/sp/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="1">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menusp.shtml" target="_blank" class="glyphLogin"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/" target="_blank" class="glyphGps"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="/">個人のお客さま</a></li>
					<li class="nav02"><a href="/hojin/">法人のお客さま</a></li>
					<li class="nav03"><a href="/nenkin/">企業年金・iDeCoのお客さま</a></li>
					<li class="nav04"><a href="/about/">りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="1">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索" id="qHead">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=rb_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=rb_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=rb_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/gaika/?bank=rb_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="/kojin/kokusai/"><span>個人向け国債・国債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=rb_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=rb_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=rb_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10" role="treeitem"><a href="/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=rb_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=rb_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=rb_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=rb_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=rb_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink navDropImg navDropImg01 clear">
									<dl>
										<dt><a href="/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=rb_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu05"><a href="/kojin/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu07"><a href="/nenkin/ideco/"><span>iDeCo（個人型確定<br>拠出年金）</span></a></li>
												<li class="kojin04Menu08"><a href="/nenkin/401k/"><span>企業型確定拠出年金</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=rb_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>コメント：電話マークアイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=rb_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/kaigai/soukin/"><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=rb_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=rb_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
												<li class="kojin05Menu11"><a href="/kojin/yoyaku/"><span>来店予約サービス</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="https://www.saitamaresona.co.jp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
					<p class="spContent"><a href="https://www.saitamaresona.co.jp/sp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="https://www.saitamaresona.co.jp/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="5">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menusp.shtml" class="glyphLogin" target="_blank"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" class="glyphGps" target="_blank"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="https://www.saitamaresona.co.jp/">個人のお客さま</a></li>
					<li class="nav02"><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
					<li class="nav04"><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="5">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="https://www.saitamaresona.co.jp/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=sr_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=sr_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/"><span>個人向け国債・国債・地方債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=sr_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=sr_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10"><a href="https://www.saitamaresona.co.jp/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=sr_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=sr_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=sr_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=sr_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=sr_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=sr_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu04"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/"><span>きょういく信託</span></a></li>
												<li class="kojin04Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu06"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html"><span>iDeCo<br>(個人型確定拠出年金)</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="https://www.saitamaresona.co.jp/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>電話アイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=sr_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span class="dropLinkNewIcon01">New</span><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=sr_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=sr_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
	</header><!-- //#header -->
	<div id="topicPath">
		<div class="topicPathInner" data-select-bank="りそな銀行" aria-hidden="false">
			<span><a href="/"><span>銀行ホーム</span></a></span>
			<span><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></span>
			<span><a href="/kojin/jutaku/?bank=rb_unite"><span>住宅ローン（新規）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/?bank=rb_unite"><span>団体信用生命保険（団信）</span></a></span>
		</div>
		<div class="topicPathInner" data-select-bank="埼玉りそな銀行" aria-hidden="true">
			<span><a href="https://www.saitamaresona.co.jp/"><span>銀行ホーム</span></a></span>
			<span><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></span>
			<span><a href="/kojin/jutaku/?bank=sr_unite"><span>住宅ローン（新規）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/?bank=sr_unite"><span>団体信用生命保険（団信）</span></a></span>
		</div>
	</div><!-- #topicPath -->
	<div id="wrap">
		<div id="mainArea">
			<main>
				<section>
					<div class="coverTitle">
						<div class="coverTitleInner">
							<h1 class="h1Title01"><span class="titleInner">団体信用生命保険（団信）</span></h1>
							<p class="coverTitleLead">りそなの団体信用生命保険（団信）をご紹介いたします。りそなは所定のがんのみをカバーする特約（がん保障特約）から3大疾病（所定のがん、急性心筋梗塞・脳卒中による所定の状態）はもちろん、病気やケガによる所定の状態・所定の要介護状態など、様々なリスクをカバーする特約（団信革命）まであなたにあった保障タイプが選べます。</p>
						</div><!-- /.mainCoverInner -->
					</div>
					<div class="indexMvArea01">
						<div class="indexMvImg01">
							<span class="pcContent"><img src="/kojin/jutaku/danshin/images/index_mv_renewal01.png" alt="住宅ローンを借りた後のもしも・・・あなたはどこまで備えますか？ がん 脳梗塞 心筋梗塞 ケガによる障害 死亡や高度障害に備える住宅ローンに金利上乗せなし（0円）で加入できる一般団信"></span>
							<span class="spContent"><img src="/kojin/jutaku/danshin/images/index_mv_renewal01_sp.png" alt="住宅ローンを借りた後のもしも・・・あなたはどこまで備えますか？ がん 脳梗塞 心筋梗塞 ケガによる障害 死亡や高度障害に備える住宅ローンに金利上乗せなし（0円）で加入できる一般団信"></span>
						</div>
					</div>
					<div class="mainInner">
						<div class="mainContents">
							<div class="blockInnerB03 tCenter">
	<figure>
		<img src="/kojin/jutaku/images/bnr_danshincp.png" alt="冬のあったか金利優グーまつり　キャンペーン期間：2024年12月1日(日)～2025年3月31日(月)" class="pcContent">
		<img src="/kojin/jutaku/images/bnr_danshincp_sp.png" alt="冬のあったか金利優グーまつり　キャンペーン期間：2024年12月1日(日)～2025年3月31日(月)" class="spContent">
	</figure>
	<div>
		<a data-select-bank="りそな銀行" aria-hidden="false" href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 w300 spW100p mb00"><span>団信革命とは？</span></a>
		<a data-select-bank="埼玉りそな銀行" aria-hidden="true" href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 w300 spW100p mb00"><span>団信革命とは？</span></a>
	</div>
	<div class="tCenter">
		<a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false">
			<img src="/kojin/jutaku/images/common_bnr03_pc.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高がどちらも0円" class="pcContent">
			<img src="/kojin/jutaku/images/common_bnr03_sp.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高がどちらも0円" class="spContent">
		</a>
		<a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true">
			<img src="/kojin/jutaku/images/common_bnr03_pc.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高がどちらも0円" class="pcContent">
			<img src="/kojin/jutaku/images/common_bnr03_sp.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高がどちらも0円" class="spContent">
		</a>
	</div>
</div>

							<section class="normalBlockA01 jsAccTitle">
								<div role="tablist" class="blockInner">
									<h2 id="accSwitchA01" class="jsAccSwitch blockTitle indexGlyphBeginner" role="tab" aria-selected="false" aria-controls="accContentA01"><span>団体信用生命保険とは？</span></h2>
									<div id="accContentA01" class="blockDetail" role="tabpanel" aria-labelledby="accSwitchA01" aria-expanded="false" aria-hidden="true">
										<div class="blockInnerB05">
											<p>団体信用生命保険（団信）とは、万が一のことがあったときに、生命保険会社が住宅ローン残高に相当する保険金を銀行に支払い、債務の返済に充てる保険のことです。</p>
											<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal01.png" alt="死亡・高度障害 住宅ローン残高が0円に" class="w500 spW280"></div>
											<p>死亡や高度障害を保障する一般団信は金利上乗せなしで全員加入となっており、自分が備えたい範囲を広げる追加のオプションの特約をつけることができます。</p>
											<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal02.png" alt="死亡 高度障害状態＋がん・急性心筋梗塞・病気やケガによる所定の状態" class="w600 v4-spW300"></div>
										</div>
									</div>
								</div><!-- //.boxBlock01 -->
							</section>
							<dl class="blowoutBlockA01 marginLvH3">
								<dt class="title">動画で解説 りそなの保障特約付団信</dt>
								<dd class="detail">
									<div class="tCenter"><a href="https://api01-platform.stream.co.jp/apiservice/plt3/NzU3%23MzQ3Ng%3d%3d%23280%23168%230%2333E220F86400%23OzEwOzEwOzEw%23" target="_blank"><img src="/kojin/jutaku/danshin/images/common_img_movie01.png" alt="団信革命 りそなの「団信革命」は所定の状態に該当すれば全額保障！"></a></div>
								</dd>
							</dl>
							<section>
								<div class="blockInnerB01 tCenter">
									<div><img src="/kojin/jutaku/danshin/images/index_img_renewal03.png" alt="Step1" class="spW64"></div>
									<h2 class="h2Title01 noBorder"><span class="titleInner spFs20">まずは特約の種類と金額を知ろう</span></h2>
									<div class="blockInnerB05">
	<div id="danshinComparisonTableTab01" class="tabList jsTabAcc02">
		<ul role="tablist" class="tab2column clear">
			<li><div id="comparisonTableTabSwitch01" role="tab" aria-selected="true" aria-controls="comparisonTableTabContent01" data-tab-parent="danshinComparisonTableTab01"><a href="javascript:;" class="tCenter"><span>通常の団信</span></a></div></li>
			<li><div id="comparisonTableTabSwitch02" role="tab" aria-selected="false" aria-controls="comparisonTableTabContent02" data-tab-parent="danshinComparisonTableTab01"><a href="javascript:;" class="tCenter"><span>ペアローン限定の<br class="spContent">団信</span></a></div></li>
		</ul>
		<div id="comparisonTableTabContent01" class="blockInnerB05 mt80 spMt30 pl00 pr00 spPb00" role="tabpanel" aria-labelledby="comparisonTableTabSwitch01" aria-hidden="false">
			<table class="v4-tableLayoutA01 danshinComparisonTable01">
				<thead>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a><a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a></th>
						<th class="w30p v4-tableBg03"><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a></th>
						<th class="w30p v4-tableBg02 danshinBalloon01"><img src="/kojin/jutaku/danshin/images/danshin_balloon_01.png" alt="銀行員おすすめ" class="danshinBalloonImg01"><a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a></th>
					</tr>
				</thead>
				<tbody>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="3" class="spFsMiddle">
							<div>
								<div class="blockInnerB05">
									<p><span class="danshinCalcIcon01 spFs14"><span>月々いくら上乗せになるのか調べよう</span></span></p>
									<form action="#" class="danshinTableSimForm01 blockInnerB03 tCenter w53p spW100p mlrAuto spMt15">
										<dl class="danshinTableSimColumn01">
											<dt>年齢</dt>
											<dd class="layoutColumnA2_2">
												<div><a href="javascript:;" data-age="under40" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01 active">40歳未満</a></div>
												<div><a href="javascript:;" data-age="over40" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01">40歳以上</a></div>
											</dd>
										</dl>
										<dl class="danshinTableSimColumn01 spMt15">
											<dt>借りたい金額</dt>
											<dd class="danshinTableSimInputBlock01 mlrAuto">
												<input id="danshinSimInput01" class="danshinTableSimInput01" type="tel" name="danshinBorrowingAmount" value="" maxlength="5" autocomplete="off">
												<span class="unit">万円</span>
											</dd>
										</dl>
										<div class="newAdjustment spMt15">
											<ul class="listMemo01 tLeft danshinTableSimMemo01">
												<li><span class="listMark">※</span>変動金利<span id="floatingRate"></span>％、返済期間<span id="repaymentPeriod"></span>年で計算</li>
												<li><span class="listMark">※</span>特約付団信は50歳以上の方は申込みできません。</li>
											</ul>
										</div>
										<input type="text" name="typeHidden" disabled>
									</form>
								</div>
							</div>
						</th>
					</tr>
					<tr class="separatorLineNone danshinTableSimAdjust">
						<td class="v4-tableBg04"><span></span></td>
						<td class="v4-tableBg03"><span></span></td>
						<td class="v4-tableBg02"><span></span></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13">上乗せ金利</span></p></th>
					</tr>
					<tr class="cellNarrow separatorLineNone">
						<td class="v4-tableBg04 fsMaximum tBold">年<span id="interestRate_cancer"></span>％</td>
						<td class="v4-tableBg03 fsMaximum tBold">年<span id="interestRate_disease"></span>％</td>
						<td class="v4-tableBg02 fsMaximum tBold">年<span id="interestRate_danshin"></span>％</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13">月々の上乗せ金額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 fsMaximum tBold"><span id="addOn_cancer"></span>円</td>
						<td class="v4-tableBg03 fsMaximum tBold"><span id="addOn_disease"></span>円</td>
						<td class="v4-tableBg02 fsMaximum tBold"><span id="addOn_danshin"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="3"><p><span class="spFs13">毎月の返済額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 fsMaximum spFsLarge tBold"><span id="repaymentAmount_cancer"></span>円</td>
						<td class="v4-tableBg03 fsMaximum spFsLarge tBold"><span id="repaymentAmount_disease"></span>円</td>
						<td class="v4-tableBg02 fsMaximum spFsLarge tBold"><span id="repaymentAmount_danshin"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="3" class="spFsMiddle"><p><span class="danshinSecurityIcon01"><span>保障の範囲の違いを知ろう</span></span></p></th>
					</tr>
					<tr class="separatorLineNone">
						<td class="v4-tableBg04"><span></span></td>
						<td class="v4-tableBg03"><span></span></td>
						<td class="v4-tableBg02"><span></span></td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="3"><p><span class="spFs13">死亡・<a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" class="indexIcoModal01 v3-jsModal01"><span>高度障害状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor06" class="indexIcoModal01 danshin-jsModal01"><span>所定のがん</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor02" class="indexIcoModal01 danshin-jsModal01"><span>急性心筋梗塞・脳卒中</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor03" class="indexIcoModal01 danshin-jsModal01"><span>病気・ケガによる16の状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor04" class="indexIcoModal01 danshin-jsModal01"><span>所定の要介護状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
				</tbody>
				<tfoot>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a><a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a></th>
						<th class="w30p v4-tableBg03"><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a></th>
						<th class="w30p v4-tableBg02"><a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a></th>
					</tr>
				</tfoot>
			</table>
		</div>
		<div id="comparisonTableTabContent02" class="blockInnerB05 pl00 pr00 spP00" role="tabpanel" aria-labelledby="comparisonTableTabSwitch02" aria-hidden="true">
			<table class="v4-tableLayoutA01 danshinComparisonTable01 mt40 spMt20">
				<thead>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg01"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a></th>
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a></th>
					</tr>
				</thead>
				<tbody>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="2" class="spFsMiddle">
							<div>
								<div class="blockInnerB05">
									<p><span class="danshinCalcIcon01 spFs14"><span>月いくら追加になるのか調べよう</span></span></p>
									<div class="v4-decorationTextB02 fs18_20 spFs16 tBold spMt15"><span>お1人分ずつ確認できます</span></div>
									<form action="#" class="danshinTableSimForm01 blockInnerB03 tCenter w53p spW100p mlrAuto spMt15">
										<dl class="danshinTableSimColumn01">
											<dt>年齢</dt>
											<dd class="layoutColumnA2_2">
												<div><a href="javascript:;" data-age="under35" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01 active">35歳未満</a></div>
												<div><a href="javascript:;" data-age="over35" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01">35歳以上</a></div>
											</dd>
										</dl>
										<dl class="danshinTableSimColumn01 typePair spMt15">
											<dt>借りたい金額</dt>
											<dd class="danshinTableSimInputWrap01 mlrAuto">
												<div class="danshinTableSimInputBlock01">
													<input id="danshinSimInput02" class="danshinTableSimInput01" type="tel" name="danshinBorrowingAmount" value="" maxlength="5" autocomplete="off">
													<span class="unit">万円</span>
												</div>
												<p class="v3-itemBalloonA01 v3-balloonTopArrow danshinTableSimBalloon01 fsExtraS">お1人で借りる分の<br>金額を入力</p>
											</dd>
										</dl>
										<input type="text" name="typeHidden" disabled>
									</form>
									<div class="newAdjustment w66p spW100p mlrAuto spMt15">
										<ul class="listMemo01 tLeft danshinTableSimMemo01">
											<li><span class="listMark">※</span>変動金利<span id="floatingRate_pair"></span>％、返済期間<span id="repaymentPeriod_pair"></span>年で計算</li>
											<li><span class="listMark">※</span>りそなのペア団信は50歳以上の方は申込みできません</li>
											<li><span class="listMark">※</span>ペアがん団信の上乗せ金利は、2025年9月末までにお借入れされた方限定の特別な優遇金利です。</li>
											<li><span class="listMark">※</span>ペアのうち片方が全額繰上返済しても、ペア相手の上乗せ金利は変わりません。</li>
											<li><span class="listMark">※</span>ペア団信は、既加入金額も含めてご夫婦等合算で2億円以内です。</li>
										</ul>
									</div>
								</div>
							</div>
						</th>
					</tr>
					<tr class="separatorLineNone danshinTableSimAdjust02">
						<td class="v4-tableBg01"><span></span></td>
						<td class="v4-tableBg04"><span></span></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">上乗せ金利</span></p></th>
					</tr>
					<tr class="cellNarrow separatorLineNone">
						<td class="v4-tableBg01 fsMaximum tBold">年<span id="interestRate_pair"></span>％</td>
						<td class="v4-tableBg04 fsMaximum tBold">年<span id="interestRate_pairCancer"></span>％</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">月々の上乗せ金額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 fsMaximum tBold"><span id="addOn_pair"></span>円</td>
						<td class="v4-tableBg04 fsMaximum tBold"><span id="addOn_pairCancer"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="2"><p><span class="spFs13">毎月の返済額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 fsMaximum spFsLarge tBold"><span id="repaymentAmount_pair"></span>円</td>
						<td class="v4-tableBg04 fsMaximum spFsLarge tBold"><span id="repaymentAmount_pairCancer"></span>円</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">死亡・<a href="/kojin/jutaku/danshin/modal/detail_pair.html" data-modal="modalBody" class="indexIcoModal01 v3-jsModal01"><span>高度障害状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail_pair.html" data-modal="modalBody" data-modal-anchor02="modalPairAnchor02" class="indexIcoModal01 danshin-jsModal01"><span>所定のがん</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">リビング・ニーズ</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
				</tbody>
				<tfoot>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg01"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a></th>
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a></th>
					</tr>
				</tfoot>
			</table>
			<div class="v4-balloonCommentBox01 w85p spW100p mlrAuto">
				<div class="v4-balloonCommentBoxInner">
					<p class="tBold tCenter spTLeft">金利や返済期間など、より詳しいシミュレーションをご希望の方は住宅ローンシミュレーションをご活用ください！</p>
				</div>
			</div>
			<div class="layoutColumnA2_1 w84p spW100p mlrAuto">
				<div class="tCenter">
					<a href="/kojin/jutaku/simulation/shinki/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローンシミュレーション<br class="spContent">（新規）</span></a>
					<a href="/kojin/jutaku/simulation/shinki/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローンシミュレーション<br class="spContent">（新規）</span></a>
				</div>
				<div class="tCenter">
					<a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローンシミュレーション<br class="spContent">（お借換え）</span></a>
					<a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローンシミュレーション<br class="spContent">（お借換え）</span></a>
				</div>
			</div>
		</div>
	</div>
	<div class="tCenter"><a href="/kojin/jutaku/danshin/modal/check.html" data-modal="modalBody" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 overT01 v3-jsModal01 mb00"><span>団信選びセルフチェック</span></a></div>
</div>

								</div>
							</section>
							<section>
								<div class="blockInnerB01 tCenter">
									<div><img src="/kojin/jutaku/danshin/images/index_img_renewal04.png" alt="Step2" class="spW64"></div>
									<h2 class="h2Title01 noBorder"><span class="titleInner spFs20">団信を選ぶ時のポイントから<br class="spContent">りそなのスゴさを知る</span></h2>
								</div>
								<div class="danshinFailureStory">
									<div class="danshinFailureStoryImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal05.png" alt="悩む男性"></div>
									<div class="blockInnerB01 tCenter">
										<div class="danshinFailureStoryTag">団信の失敗話</div>
										<p class="spTLeft">〇大疾病といった保障の数が多い団信を選んだけど、<br class="pcContent">結局適用されなかったり保障額が少なかった…</p>
									</div>
								</div>
								<dl class="blowoutBlockA01">
									<dt class="title spFs16">保障の数や金利だけで判断するのではなく、<br class="spContent">条件やもらえる金額も重要です！</dt>
									<dd class="detail">
										<div class="v4-carousel01 v4-jsCarousel01 indexSecurityCarousel">
											<ul class="carouselList">
												<li class="v4-stepBlockA01">
													<div class="blockInnerB03">
														<div class="fsLarge tBold tCenter lh100">保障の<span class="fs20_40 spFs24 lh100">数</span></div>
														<figure><img src="/kojin/jutaku/danshin/images/index_img_renewal06.png" alt="りそなは保障の選択が多い" class="w420 spW100p"></figure>
														<p>まずは保障の数も大事なポイントです。3大疾病やケガによる障害など保障の選択が多いほど自分にあった団信を見つけることができます。</p>
														<div class="indexCommentBlock01">
															<div class="indexCommentImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal09.png" alt="りそなのここがスゴイ"></div>
															<p class="tBold tCenter">ケガによる所定の状態や介護状態までカバーできる保障<br class="pcContent">まであり選択肢が充実している</p>
														</div>
													</div>
												</li>
												<li class="v4-stepBlockA01">
													<div class="blockInnerB03">
														<div class="fsLarge tBold tCenter lh100">保障の<span class="fs20_40 spFs24 lh100">金額</span></div>
														<div class="fsSmall tBold tCenter">住宅ローン残高の保障は？</div>
														<figure><img src="/kojin/jutaku/danshin/images/index_img_renewal07.png" alt="りそなは全額（ローン残高0円）" class="w420 spW100p"></figure>
														<p>「入院月だけ」や「半額」といったように全額ではない場合もある為、選ぶ団信がどこまでの金額が保障されるかは確認しておく必要があります。</p>
														<div class="indexCommentBlock01">
															<div class="indexCommentImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal09.png" alt="りそなのここがスゴイ"></div>
															<p class="tBold tCenter">全ての団信で支払事由に該当すれば<br>住宅ローン残高0円（全額）</p>
														</div>
													</div>
												</li>
												<li class="v4-stepBlockA01">
													<div class="blockInnerB03">
														<div class="fsLarge tBold tCenter lh100">保障の<span class="fs20_40 spFs24 lh100">タイミング</span></div>
														<figure><img src="/kojin/jutaku/danshin/images/index_img_renewal08.png" alt="りそなは所定の状態で適用" class="w420 spW100p"></figure>
														<ul class="listMemo01 fsSmall">
															<li><span class="listMark">※</span>ただし、お借入日から90日以内に診断確定されたがんなど対象外となる場合があります。</li>
														</ul>
														<p>「がんと診断されたら」や「脳卒中は60日以上継続したら」など、商品によって支払事由は異なりますので確認が必要です。</p>
														<div class="indexCommentBlock01">
															<div class="indexCommentImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal09.png" alt="りそなのここがスゴイ"></div>
															<p class="tBold tCenter">所定の状態に該当すれば！<br>適用される保障条件となっている！</p>
														</div>
													</div>
												</li>
											</ul>
										</div>
									</dd>
								</dl>
							</section>
							<section>
								<div class="blockInnerB01 tCenter">
									<div><img src="/kojin/jutaku/danshin/images/index_img_renewal10.png" alt="Step3" class="spW64"></div>
									<h2 class="h2Title01 noBorder"><span class="titleInner spFs20">将来のリスクを想像して<br class="spContent">どこまで保障を備えればよいか考える</span></h2>
								</div>
								<div class="indexFutureLisk">
									<div class="tCenter indexFutureLiskText">
										<p>金利上乗せなしで<br>死亡保障が<br class="spContent">ついてくるから<br>特約はいいかな…</p>
									</div>
									<div class="indexFutureLiskImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal11.png" alt="なんて思っているあなた"></div>
								</div>
								<dl class="blowoutBlockA01">
									<dt class="title spFs16">住宅購入後の<span class="fs26_40 spFs26">約40%</span><span class="fs26_20 spFs15">※</span>の方は<br class="spContent">「団信の特約を付けておけばよかった」<br class="spContent">と後悔しています</dt>
									<dd class="detail blockInnerB05">
										<ul class="listMemo01 fsSmall tCenter spTLeft">
											<li><span class="listMark">※</span>カーディフ生命「第4回 生活価値観・住まいに関する意識調査」</li>
										</ul>
										<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal12.png" alt="健康診断の数値悪化…　家族が増えた…　周りの人ががんに…　あの時入っていれば…"></div>
										<div class="indexLiskCarouselWrap">
											<div class="indexLickComment"><img src="/kojin/jutaku/danshin/images/index_img_renewal13.png" alt="病気などのリスクは年々増加傾向に！"></div>
											<div class="v4-carousel01 v4-jsCarousel01 indexLiskCarousel">
												<ul class="carouselList">
													<li class="v4-stepBlockA01">
														<div class="blockInnerB03 newAdjustment">
															<div class="tBold tCenter">3大疾病リスク</div>
															<div class="fsSmall tBold tCenter">今後20年以内にがん※1・急性心筋梗塞・<br class="spContent">脳卒中※2になる確率</div>
															<div class="v4-detailBlockA01">
																<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal14.png" alt="年齢とともに高まる確率"></div>
															</div>
															<ul class="listMemo02 fsSmall">
																<li><span class="listNum">※1</span><span class="listInner">上皮内がんは含みません。</span></li>
																<li><span class="listNum">※2</span><span class="listInner">くも膜下出血、脳内出血、脳梗塞の合計です。<br>厚生労働省/「平成29年 患者調査」「平成29年 簡易生命表」をもとに算出</span></li>
															</ul>
														</div>
													</li>
													<li class="v4-stepBlockA01">
														<div class="blockInnerB03 newAdjustment">
															<div class="tBold tCenter">身体障害状態リスク（重い病気やケガ）</div>
															<div class="fsSmall tBold tCenter">身体障がい認定者数の推移（18歳以上）</div>
															<div class="v4-detailBlockA01">
																<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal15.png" alt="要介護リスク"></div>
															</div>
															<div>厚生労働省/「平成12年度社会福祉行政業務報告（厚生省報告例）」「令和3年度年度福祉行政報告例」</div>
														</div>
													</li>
													<li class="v4-stepBlockA01">
														<div class="blockInnerB03 newAdjustment">
															<div class="tBold tCenter">要介護リスク</div>
															<div class="fsSmall tBold tCenter">要介護（要支援）認定者数の推移</div>
															<div class="v4-detailBlockA01">
																<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal16.png" alt="身体障害状態リスク"></div>
															</div>
															<div>厚生労働省/「介護保険事業状況報告（暫定）」（2000年4月分、2020年3月分）</div>
														</div>
													</li>
												</ul>
											</div>
										</div>
									</dd>
								</dl>
								<div class="marginLvH3">
									<div class="indexColorBox01">
										<div class="blockInnerB05">
											<p class="tGreen01 tBold tCenter">既に保険に入っている人は<br class="spContent">団信に入ることで家計の支出が下がるかも？</p>
											<div class="tCenter"><img src="/kojin/jutaku/danshin/images/index_img_renewal17.png" alt="生命保険" class="w450 spW100p"></div>
											<p>住宅ローン契約後は団体信用生命保険への加入により、加入目的が重複している場合があります。重複した加入目的部分を見直すことで、保険料が下がるケースもあり、家計の支出が下がるかも？加入している生命保険の確認をしてみましょう。</p>
											<div class="indexCommentFlexColumn01 w850 spW100p mlrAuto">
												<div class="indexCommentText01 tBold"><span class="spFs15">ひとつ言えることは</span><br class="spContent"><span class="fsMaximum tGreen01">保険は見直せるけど<br class="spContent">団信は今だけです！</span><br><span class="spFs15">団信を軸として保険を<br class="spContent">見直すことをおすすめします。</span></div>
												<div class="indexCommentImg01"><img src="/kojin/jutaku/danshin/images/index_img_renewal18.png" alt=""></div>
											</div>
										</div>
									</div>
								</div>
							</section>
						</div>
					</div>
					<section id="ancApplication" class="v2-bgYellow01 v3_fullColumnA01 danshinApplicationBlock01">
	<div class="columnInner v2-cardBlockA03 blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb20"><span class="titleInner">団信特約の申込方法</span></h2>
		<section class="cardBlockA01 bgWhite spMt00">
			<div class="blockInnerB01 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_01.png" alt="Step0" class="danshinStepLabel01">（どの団信がいいか分からない方）</span><span class="title tBold">入りたい団信特約を決める</span>
				</h3>
				<div class="layoutColumnA2_2 spMt00">
					<p class="w80p spW79p">店舗に来店いただければ金融のプロの銀行員が一緒にあなたに最適な<br class="pcContent">団信選びをお手伝いします。</p>
					<figure class="w16p spW17p danshinStepImg01"><img src="/kojin/jutaku/danshin/images/danshin_img_04.png" alt="銀行員のイラスト画像"></figure>
				</div>
				<div class="blockInnerB01 tCenter">
					<div class="v4-decorationTextB02 fs18_20 spFs16 tBold">
						<span data-select-bank="りそな銀行" aria-hidden="false">土・日・祝日や17時以降も営業</span>
						<span data-select-bank="埼玉りそな銀行" aria-hidden="true">土・日・祝日も営業</span>
					</div>
					<div>
						<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで来店予約</span></a>
						<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで来店予約</span></a>
					</div>
				</div>
			</div>
		</section>
		<section class="cardBlockA01 bgWhite danshinBeforeChain01">
			<div class="blockInnerB01 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_02.png" alt="Step1" class="danshinStepLabel01"></span><span class="title tBold">店舗またはWEBから住宅ローンの事前審査</span>
				</h3>
				<div class="layoutColumnA2_2 tCenter spMt00">
					<div class="blockInnerB01">
						<div class="v4-decorationTextB02 fs18_20 spFs16 tBold">
							<span data-select-bank="りそな銀行" aria-hidden="false">土・日・祝日や<br class="spContent">17時以降も営業</span>
							<span data-select-bank="埼玉りそな銀行" aria-hidden="true">土・日・祝日も<br class="spContent">営業</span>
						</div>
						<div>
							<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">来店予約</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">来店予約</span></a>
						</div>
					</div>
					<div class="blockInnerB01">
						<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>審査結果も<br class="spContent">Webで確認！</span></div>
						<div>
							<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
						</div>
					</div>
				</div>
				<div class="newAdjustment">
					<ul class="listMemo01 fsSmall tCenter spTLeft">
						<li><span class="listMark">※</span>事前審査では団信のお申込みはございません。</li>
					</ul>
				</div>
			</div>
		</section>
		<section class="cardBlockA01 bgWhite danshinBeforeChain01">
			<div class="blockInnerB05 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_03.png" alt="Step2" class="danshinStepLabel01"></span><span class="title tBold">事前審査の結果が届いたら団信特約申込</span>
				</h3>
				<div id="danshinSelectTabAcc01" class="tabList jsTabAcc02">
					<ul role="tablist" class="tab2column clear tCenter">
						<li><div id="danshinSelectTabSwitch01" role="tab" aria-selected="true" aria-controls="danshinSelectTabCotent01" data-tab-parent="danshinSelectTabAcc01"><a href="javascript:;"><span>通常の団信</span></a></div></li>
						<li><div id="danshinSelectTabSwitch02" role="tab" aria-selected="false" aria-controls="danshinSelectTabCotent02" data-tab-parent="danshinSelectTabAcc01"><a href="javascript:;"><span>ペアローン限定の<br class="spContent">団信</span></a></div></li>
					</ul>
					<div id="danshinSelectTabCotent01" class="blockInnerB01 spPb00 spPl00 spPr00" role="tabpanel" aria-labelledby="danshinSelectTabSwitch01" aria-hidden="false">
						<div class="tCenter">
							<a href="/kojin/jutaku/danshin/login/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ネットDe団信ログイン</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/danshin/login/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ネットDe団信ログイン</span></a>
						</div>
						<div class="newAdjustment">
							<ul class="listMemo01 fsSmall">
								<li><span class="listMark">※</span>ネットDe団信は事前審査完了後に発行されるユーザーID、パスワードをお持ちの方のみご利用いただけます。</li>
							</ul>
						</div>
					</div>
					<div id="danshinSelectTabCotent02" class="blockInnerB01 spPb00 spPl00 spPr00" role="tabpanel" aria-labelledby="danshinSelectTabSwitch02" aria-hidden="true">
						<div class="tCenter">
							<a href="/kojin/jutaku/danshin/login_pair/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w480 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ニッセイ団信ダイレクトサービス<br class="spContent"><span class="ml05 spMl00">ログイン</span></span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/danshin/login_pair/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w480 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ニッセイ団信ダイレクトサービス<br class="spContent"><span class="ml05 spMl00">ログイン</span></span></a>
						</div>
						<div class="newAdjustment">
							<ul class="listMemo01 fsSmall">
								<li><span class="listMark">※</span>ニッセイ団信ダイレクトサービスは事前審査完了後に発行されるユーザーID、パスワードをお持ちの方のみご利用いただけます。</li>
							</ul>
						</div>
					</div>
				</div>
			</div>
		</section>
		<p class="danshinAttentionIcon01 tCenter"><span>団信は<span class="tBold">住宅ローン申込時にしか手続きできません！</span>入っておけばよかったと後悔する前にもう一度検討してみてください。</span></p>
	</div>
</section>

					<div class="mainInner">
						<div class="mainContents">
							<div class="v2-colorCardA08 marginLvH2">
	<div class="cardInner tCenter">
		<section id="ancAloneDanshin">
			<h2 class="h2Title01 noBorder tCenter spMb15"><span class="titleInner">単独で加入する団信</span></h2>
			<section class="blockInnerB05">
				<h3 class="danshinRecommendTitle01"><span><img src="/kojin/jutaku/danshin/images/danshin_balloon_02.png" alt="銀行員おすすめ" class="danshinRecommendBalloon01"><span class="danshinTextMarker01">りそなが自信をもって<br class="spContent">おすすめする！<br class="spContent">まさに革命的な団信</span></span></h3>
				<div><img src="/kojin/jutaku/danshin/images/danshin_logo_01.png" alt="団信革命" class="w25p spW67p"></div>
				<p>団信革命は病気やケガで所定の状態に該当すれば、<span class="v2-textMarker01 tBold">仕事に復帰しても<sup>※1</sup>住宅ローン残高が0円です。</span></p>
				<div class="spMt20">
					<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>団信革命について詳しくみる</span></a>
					<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>団信革命について詳しくみる</span></a>
				</div>
				<div class="newAdjustment spMt20">
					<ul class="listMemo02 fsSmall tLeft">
						<li><span class="listNum">※1</span>急性心筋梗塞の場合、初診日からその日を含めて60日以上労働の制限を必要とする状態が継続したと医師によって診断されたとき等、状態により支払要件は異なります。詳細は<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite#anchor01" data-select-bank="りそな銀行" aria-hidden="false">商品概要</a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite#anchor01" data-select-bank="埼玉りそな銀行" aria-hidden="true">商品概要</a>をご確認ください。</li>
					</ul>
				</div>
				<section class="blockInnerB05 marginLvH3">
					<h4 class="fsMaximum spFsMiddle tBold">もっと手ごろな団信にしたい方は</h4>
					<ul class="layoutColumnA2_2 w60p spW100p mlrAuto spMt10">
						<li>
							<a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="りそな銀行" aria-hidden="false">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">3大疾病保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_04.png" alt="３大疾病団信" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
							<a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">3大疾病保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_04.png" alt="３大疾病団信" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
						</li>
						<li>
							<a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="りそな銀行" aria-hidden="false">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">がん保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_05.png" alt="がん保障特約" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
							<a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">がん保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_05.png" alt="がん保障特約" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
						</li>
					</ul>
				</section>
				<section class="blockInnerB05 marginLvH3">
					<h4 class="fsMaximum spFsMiddle tBold">健康上の理由でご加入に不安がある方は</h4>
					<figure><img src="/kojin/jutaku/danshin/images/danshin_ico_06.png" alt="ワイド団信" class="w12p spW26p"></figure>
					<p>通常よりも加入条件が緩和されている団体信用生命保険。持病・病歴がある方でもご加入できる団信。</p>
					<div class="spMt20">
						<a href="/kojin/jutaku/danshin/wide_danshin/?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ワイド団信について詳しくみる</span></a>
						<a href="/kojin/jutaku/danshin/wide_danshin/?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ワイド団信について詳しくみる</span></a>
					</div>
					<div class="newAdjustment spMt20">
						<ul class="listMemo01 fsSmall spTLeft">
							<li><span class="listMark">※</span>保険会社による所定の審査が必要になります。必ずしもご加入を保証するものではありません。</li>
						</ul>
					</div>
					<div class="spMt20">
						<div class="blockInnerB01">
							<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>ご検討中の方はまずは<br class="spContent">お気軽にご相談ください </span></div>
							<div>
								<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle glyphArrowTriangleRight iconNewWindow w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで来店予約</span></a>
								<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle glyphArrowTriangleRight iconNewWindow w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで来店予約</span></a>
							</div>
						</div>
					</div>
				</section>
			</section>
		</section>
		<section id="ancPairDanshin" class="marginLvH2">
			<h2 class="h2Title01 noBorder tCenter spMb15"><span class="titleInner">ペアで入る団信</span></h2>
			<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>2人とも安心 </span></div>
			<div class="blockInnerB03">
				<p><img src="/kojin/jutaku/danshin/images/pair_img_11.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高どちらも0円" class="w600"></p>
				<div>
					<a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ペア団信について詳しくみる</span></a>
					<a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ペア団信について詳しくみる</span></a>
				</div>
			</div>
		</section>
	</div>
</div>

							<div class="blockInnerB03">
								<p class="tCenter spTLeft marginLvH2">保険金の支払いには条件があります。生命保険のご加入にあたっては、<br class="pcContent">「重要事項に関するご説明（契約概要・注意喚起情報）」を必ずお読みいただき、詳細をご確認ください。</p>
								<p class="tRight">（登）C23E6255（2023.11.22）</p>
							</div>
							<section>
								<h2 class="h2Title01 tCenter spTLeft"><span class="titleInner">よくあるご質問</span></h2>
								<div class="blockInnerB04" data-select-bank="りそな銀行" aria-hidden="false">
									<div id="faqLoadList1">
										<script>
											$(function() {
												LoadFaq.init({
													'id'      : 'faqLoadList1',
													'question': ['faq_jutaku_0014','faq_jutaku_0015','faq_jutaku_0028','faq_jutaku_0029','faq_jutaku_0030','faq_jutaku_0031','faq_jutaku_0032','faq_jutaku_0033','faq_jutaku_0034','faq_jutaku_0035'],
													'answer'  : true,//true or false
													'design'  : '1',
													'domain' : 'rb'
												});
											});
										</script>
									</div>
									<div><a href="/kojin/jutaku/faq/" class="btnT01 btnSMiddle glyphArrowTriangleRight btnWFull w380 mlrAuto overT01"><span>住宅ローンのよくあるご質問一覧</span></a></div>
								</div>
								<div class="blockInnerB04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
									<div id="faqLoadList2">
										<script>
											$(function() {
												LoadFaq.init({
													'id'      : 'faqLoadList2',
													'question': ['faq_jutaku_0014','faq_jutaku_0015','faq_jutaku_0028','faq_jutaku_0029','faq_jutaku_0030','faq_jutaku_0031','faq_jutaku_0032','faq_jutaku_0033','faq_jutaku_0034','faq_jutaku_0035'],
													'answer'  : true,//true or false
													'design'  : '1',
													'domain' : 'sr',
													'_blank' : true
												});
											});
										</script>
									</div>
									<div><a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow btnWFull w410 mlrAuto overT01" target="_blank"><span>住宅ローンのよくあるご質問一覧</span></a></div>
								</div>
							</section>
							<div class="v2-cardBlockA03 v2-bgGreen01 blockInnerB05 marginLvH2 newAdjustment">
	<section class="blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb30"><span class="titleInner">関連情報</span></h2>
		<div class="v4-carousel01 carouselType03 v4-jsCarousel01" data-select-bank="りそな銀行" aria-hidden="false">
	<ul class="carouselList">
		<li><a href="/kojin/landing/sx/?bank=rb_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_12.png" alt="住宅ローンや資産形成を通じてSDGsに貢献しよう！"></a></li>
		<li><a href="/kojin/jutaku/landing/keiyakugo.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_11.png" alt="必見！銀行員が教える住宅ローン契約後にやるべきこと"></a></li>
		<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.gif" alt="スマホでカンタン新生活必要なものリスト！"></a></li>
		<li><a href="/kojin/column/jutaku/enquete_0001.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_06.png" alt="年収別でみる　みんなの住宅購入事情　結果詳細を見る"></a></li>
		<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_02.gif" alt="スマホでカンタンマイホームToDoリスト"></a></li>
		<li>
			<a href="/kojin/jutaku/zeh/?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_08.png" alt="環境等配慮型住宅にすると金利が下がるって知ってました？"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/sumikae/?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_13.gif" alt="住みかえ検討中の方！りそなの住みかえプランがおすすめ！"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/reason/resona_club.html?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_10.png" alt="新規住宅ローン利用者限定　諸条件を満たすと10,000ポイントもらえる！　詳しくはこちら"></a>
		</li>
	</ul>
</div>
<div class="v4-carousel01 carouselType03 v4-jsCarousel01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
	<ul class="carouselList">
		<li><a href="/kojin/mycar/?bank=sr_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.jpg" alt="りそなマイカーローン"></a></li>
		<li><a href="/kojin/landing/sx/?bank=sr_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_12.png" alt="住宅ローンや資産形成を通じてSDGsに貢献しよう！"></a></li>
		<li><a href="https://www.saitamaresona.co.jp/kojin/jutaku/landing/keiyakugo.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_11.png" alt="必見！銀行員が教える住宅ローン契約後にやるべきこと"></a></li>
		<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.gif" alt="スマホでカンタン新生活必要なものリスト！"></a></li>
		<li><a href="/kojin/column/jutaku/enquete_0001.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_06.png" alt="年収別でみる　みんなの住宅購入事情　結果詳細を見る"></a></li>
		<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_02.gif" alt="スマホでカンタンマイホームToDoリスト"></a></li>
		<li>
			<a href="/kojin/jutaku/zeh/?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_08.png" alt="環境等配慮型住宅にすると金利が下がるって知ってました？"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/sumikae/?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_13.gif" alt="住みかえ検討中の方！りそなの住みかえプランがおすすめ！"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/reason/resona_club.html?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_10.png" alt="新規住宅ローン利用者限定　諸条件を満たすと10,000ポイントもらえる！　詳しくはこちら"></a>
		</li>
	</ul>
</div>

	</section>
	<section class="blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb30"><span class="titleInner">住宅ローンメニュー</span></h2>
		<ul class="w700 spW100p mlrAuto">
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA01" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA01"><span class="titleInner fs18_24 spFs18">住宅ローンの商品</span></h3>
						<div id="jutakuRelationAccContentA01" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA01" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a>
									<a href="/kojin/jutaku/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/karikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a>
									<a href="/kojin/jutaku/karikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a>
									<a href="/kojin/jutaku.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA02" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA02"><span class="titleInner fs18_24 spFs18">特典・プラン</span></h3>
						<div id="jutakuRelationAccContentA02" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA02" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/lin/" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな女性向け住宅ローン『凛next』</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/lin/" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな女性向け住宅ローン『凛next』</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/zeh/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a>
									<a href="/kojin/jutaku/zeh/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/sumikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a>
									<a href="/kojin/jutaku/sumikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA03" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA03"><span class="titleInner fs18_24 spFs18">団信・火災</span></h3>
						<div id="jutakuRelationAccContentA03" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA03" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/danshin/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a>
									<a href="/kojin/jutaku/danshin/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a>
									<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a>
									<a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>がん保障特約</span></a>
									<a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>がん保障特約</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>ペア団信</span></a>
									<a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>ペア団信</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/danshin.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a>
									<a href="/kojin/jutaku/danshin/danshin.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a>
									<a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/shizen/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a>
									<a href="/kojin/jutaku/shizen/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA04" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA04"><span class="titleInner fs18_24 spFs18">金利・シミュレーション</span></h3>
						<div id="jutakuRelationAccContentA04" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA04" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/loan_viewer.html" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>金利一覧</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/loan_viewer.html" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>金利一覧</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/simulation/shinki/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a>
									<a href="/kojin/jutaku/simulation/shinki/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a>
									<a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA05" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA05"><span class="titleInner fs18_24 spFs18">お手続き・諸費用など</span></h3>
						<div id="jutakuRelationAccContentA05" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA05" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/procedure1.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a>
									<a href="/kojin/jutaku/procedure1.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/procedure2.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a>
									<a href="/kojin/jutaku/procedure2.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/fee/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a>
									<a href="/kojin/jutaku/fee/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/documents.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a>
									<a href="/kojin/jutaku/documents.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/pair_loan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>ペアローン</span></a>
									<a href="/kojin/jutaku/pair_loan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>ペアローン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA06" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA06"><span class="titleInner fs18_24 spFs18">お申込み・審査</span></h3>
						<div id="jutakuRelationAccContentA06" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA06" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/jizen_moushikomi/" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a>
								</li>
								<li>
									<a href="https://www.jl.resona-gr.co.jp/fmweb/login" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで正式審査申込（りそな住宅ローンマイページ）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/econtract.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a>
									<a href="/kojin/jutaku/econtract.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a>
								</li>
								<li>
									<a href="https://navi.resona-gr.co.jp/jutaku/login?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a>
									<a href="https://navi.resona-gr.co.jp/jutaku/login?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA07" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA07"><span class="titleInner fs18_24 spFs18">はじめての方</span></h3>
						<div id="jutakuRelationAccContentA07" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA07" aria-expanded="false" aria-hidden="true">
							<ul>
								<li><a href="/kojin/column/jutaku/myhome_first.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>はじめてのマイホーム&amp;住宅ローン</span></a></li>
								<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>マイホームToDoリスト</span></a></li>
								<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新生活必要なものリスト</span></a></li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA08" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA08"><span class="titleInner fs18_24 spFs18">用語集・Q&amp;A</span></h3>
						<div id="jutakuRelationAccContentA08" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA08" aria-expanded="false" aria-hidden="true">
							<ul>
								<li><a href="/kojin/jutaku/yougo/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>住宅ローンの用語集</span></a></li>
								<li>
									<a href="/kojin/jutaku/faq/" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
		</ul>
	</section>
	<section class="karte-jutakuRanking"></section>
	<div class="v4-detailBlockA01">
		<ul class="layoutColumnA3_1 v2-wide01 tCenter">
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>毎月の返済はいくら？</span></p>
				<div class="jutakuRelationBtn01">
					<a href="/kojin/jutaku/simulation/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>住宅ローンシミュレーション</span></a>
					<a href="/kojin/jutaku/simulation/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>住宅ローンシミュレーション</span></a>
				</div>
			</li>
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>土・日・祝日や17時以降も営業</span></p>
				<div>
					<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>Webで来店予約</span></a>
					<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>Webで来店予約</span></a>
				</div>
			</li>
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>審査結果もWebで確認！</span></p>
				<div>
					<a href="/kojin/jutaku/jizen_moushikomi/" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow btnWFull"><span>Webで事前審査</span></a>
					<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow btnWFull"><span>Webで事前審査</span></a>
				</div>
			</li>
		</ul>
	</div>
	<section class="marginLvH2">
		<div class="v3-balloonBlockB01">
			<h2 class="v3-title">その他ローン</h2>
			<div class="v3-detail blockInnerB03">
				<ul class="v3-layoutColumnA4_2 mycarFlexAuto">
					<li>
						<a href="/kojin/jutaku/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローン</span></a>
						<a href="/kojin/jutaku/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローン</span></a>
					</li>
					<li>
						<a href="/kojin/cardloan/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>カードローン</span></a>
						<a href="/kojin/cardloan/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>カードローン</span></a>
					</li>
					<li>
						<a href="/kojin/freeloan/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>フリーローン</span></a>
						<a href="/kojin/freeloan/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>フリーローン</span></a>
					</li>
					<li>
						<a href="/kojin/edu/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>教育ローン</span></a>
						<a href="/kojin/edu/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>教育ローン</span></a>
					</li>
					<li>
						<a href="/kojin/reform/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>リフォームローン</span></a>
						<a href="/kojin/reform/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>リフォームローン</span></a>
					</li>
					<li>
						<a href="/kojin/akiya/" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>空き家専用ローン</span></a>
						<a href="https://www.saitamaresona.co.jp/kojin/akiya/" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>空き家専用ローン</span></a>
					</li>
				</ul>
				<div class="blockInnerB01 tCenter w100p">
					<p class="v4-decorationTextB02 tBold tOrange01"><span>15秒で簡単に診断！</span></p>
					<div>
						<a href="/kojin/column/loan/loan_simulation/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow overT01 w400" data-select-bank="りそな銀行" aria-hidden="false"><span>ローン適正診断</span></a>
						<a href="/kojin/loan/column/loan_simulation/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow overT01 w400" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ローン適正診断</span></a>
					</div>
				</div>
			</div>
		</div>
	</section>
</div>

						</div>
					</div>
				</section>
			</main>
		</div><!-- //#mainArea -->
	</div><!-- //#wrap -->

	<div id="risonya_keyword" class="hide">
		<div class="keywordInner">
			<div class="balloon">
				<p>りそにゃの壁紙<br class="spContent">プレゼント！<br><a href="/kojin/jutaku/wallpaper/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>詳細はこちら</span></a><a href="/kojin/jutaku/wallpaper/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>詳細はこちら</span></a></p>
				<div class="close"></div>
			</div>
			<img src="/kojin/jutaku/wallpaper/images/word_44GP.png" alt="">
		</div>
	</div>

	<footer role="contentinfo">
				<a href="#top" id="pageTop"><img src="/common/images/go_top.svg" alt="ページトップへ戻る" class="svgTop"></a>
		<div class="snsArea">
			<!-- PC ONLY CONTENTS START-->
			<dl>
				<dt><a href="https://www.resona-gr.co.jp/holdings/other/sns/" target="_blank">りそなソーシャルメディアネットワーク</a></dt>
				<dd>
					<ul class="footerSnsList">
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><figure><img src="/common/images/footer_sns_tw02.gif" alt="X"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><figure><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><figure><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><figure><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><figure><img src="/common/images/footer_sns_line02.gif" alt="Line"></figure></a></li>
					</ul>
				</dd>
			</dl>
			<!-- PC ONLY CONTENTS END-->
			<!-- SP ONLY CONTENTS START-->
			<ul class="spFooterSnsList">
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><img src="/common/images/footer_sns_tw02.gif" alt="X"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><img src="/common/images/footer_sns_line02.gif" alt="Line"></a></li>
			</ul>
			<!-- SP ONLY CONTENTS END-->
		</div><!-- //.snsArea -->

		<div data-select-bank="りそな銀行" aria-hidden="false">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">
			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=rb_unite&loc=7">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=rb_unite&loc=7"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=rb_unite&loc=7">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/?loc=7" target="_blank">つみたてボックス</a></li>
						<li><a href="/kojin/toshin/?loc=7">投資信託</a></li>
						<li><a href="/kojin/fundwrap/?loc=7" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=rb_unite&loc=7">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=rb_unite&loc=7">定期預金</a></li>
						<li><a href="/kojin/kinsen/?loc=7">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=rb_unite&loc=7">外貨預金</a></li>
						<li><a href="/kojin/shikin_unyo/?loc=7">資金運用プラン</a></li>
						<li><a href="/kojin/taishoku_unyo/?loc=7">退職金コース</a></li>
						<li><a href="/kojin/kokusai/?loc=7">個人向け国債・国債</a></li>
						<li><a href="/kojin/gaisai/?loc=7">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=rb_unite&loc=7">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=rb_unite&loc=7">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=7">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=7">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=rb_unite&loc=7">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=rb_unite&loc=7">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=7">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=rb_unite&loc=7">マイカーローン</a></li>
						<li><a href="/kojin/edu/?loc=7">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=rb_unite&loc=7">リフォームローン</a></li>
						<li><a href="/kojin/apaman/?loc=7">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=7">リバースモーゲージローン</a></li>
						<li><a href="/kojin/akiya/?loc=7">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/sonaeru.html?loc=7">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=rb_unite&loc=7">保険</a></li>
						<li><a href="/kojin/souzokusyoukei.html?loc=7">相続・承継</a></li>
						<li><a href="/kojin/zaisankanri.html?loc=7">財産管理業務</a></li>
						<li><a href="/kojin/private/?loc=7">プライベートバンキング業務</a></li>
						<li><a href="/nenkin/ideco/?loc=7">iDeCo(個人型確定拠出年金)</a></li>
						<li><a href="/nenkin/401k/?loc=7">企業型確定拠出年金</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/tsukau.html?loc=7">便利にお得に使いたい</a></h2>
					<ul>
						<li><a href="/direct/?loc=7">マイゲート</a></li>
						<li><a href="/kojin/app/?bank=rb_unite&loc=7"><span>スマートフォンアプリ</span></a></li>
						<li><a href="/club/?bank=rb_unite&loc=7">りそなクラブ</a></li>
						<li><a href="/kojin/exchange/gaika/?loc=7">外貨両替</a></li>
						<li><a href="/kojin/kaigai/soukin/?loc=7">海外送金</a></li>
						<li><a href="/kojin/visa_debit/?bank=rb_unite&loc=7">Visaデビットカード</a></li>
						<li><a href="/kojin/credit/?bank=rb_unite&loc=7">クレジットカード</a></li>
						<li><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=7">ペイジー料金払込サービス</a></li>
						<li><a href="/kojin/koueikyougi/?loc=7">即時口座決済</a></li>
						<li><a href="/kojin/yoyaku/?loc=7">来店予約サービス</a></li>
						<li><a href="/kojin/column/index2.html?loc=7">お金に関わるりそなのコラム</a></li>
						<li><a href="https://community.resonabank.co.jp/?loc=7" target="_blank">お金のあれこれ</a></li>
					</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="/"><span>個人のお客さま</span></a></li>
				<li><a href="/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="/nenkin/"><span>企業年金のお客さま</span></a></li>
				<li><a href="/about/"><span>りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="1">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com" target="_blank">採用情報</a></li>
					<li><a href="/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="/kojin/contact/#10">指定紛争解決機関</a></li>
					<li><a href="/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0010</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>近畿財務局長(登金)第3号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会 一般社団法人金融先物取引業協会 一般社団法人日本クレジット協会 一般社団法人日本投資顧問業協会</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">

			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=sr_unite">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=sr_unite"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/" target="_blank">つみたてボックス</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
						<li><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=sr_unite">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/">退職金コース</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/gaisai/">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=sr_unite">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=sr_unite">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
						<li><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=sr_unite">リフォームローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=sr_unite">リバースモーゲージローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/akiya/">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html">便利にお得に使いたい</a></h2>
						<ul>
							<li><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
							<li><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
							<li><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/">海外送金</a></li>
							<li><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
							<li><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/">即時口座決済</a></li>
						</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav saitamaFooterLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="https://www.saitamaresona.co.jp/"><span>個人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/about/"><span>埼玉りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="5">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="https://www.saitamaresona.co.jp/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com/" target="_blank">採用情報</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="https://www.saitamaresona.co.jp/kojin/contact/#13">指定紛争解決機関</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社埼玉りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0017</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>関東財務局長(登金)第593号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会&nbsp;&nbsp;&nbsp;一般社団法人金融先物取引業協会&nbsp;&nbsp;&nbsp;一般社団法人日本クレジット協会</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>信託契約代理業&nbsp;:</dt>
							<dd>登録番号　関東財務局長（代信）第22号</dd>
						</dl>
						<dl>
							<dt>所属信託会社の商号&nbsp;:</dt>
							<dd>株式会社りそな銀行</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Saitama Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
	</footer>
	<!-- FOOTER FIXED CONTENT START-->
	<script type="text/javascript" charset="UTF-8" src="https://mobi.lineomni.jp/web/mobi-agent-client-frame-loader.min.js?domainId=resonagr"></script>
<script type="text/javascript" charset="UTF-8" src="/kojin/contact/chat/decsupport/js/ds-jutaku_two.js"></script>
	<!-- HUMANCHAT -->
<div>
	<div class="fixedBottom01 fixedBottomChat01 jsFixedBottom01 jsFixedBottomChat">
		<div class="fixedBottomInner">
			<div class="fixedBottomContent">
				<div class="fixedBottomNav">
					<ul class="fixedBottomList01">
						<li>
							<a data-select-bank="りそな銀行" aria-hidden="false" href="/kojin/jutaku/modal/modal_menu.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
							<a data-select-bank="埼玉りそな銀行" aria-hidden="true" href="/kojin/jutaku/modal/modal_menu_saitama.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
						</li>
						<li><a href="javascript:void(0)" class="fixedBottomIconFaq01 spTCenter jsChatMenuTrg"><span class="fsSmall spFs10">お困りの方</span></a></li>
					</ul>
					<div class="fixedBottomChatMenu01 jsFixedBottomChatMenu">
						<ul class="chatMenuBlock01 tCenter">
							<li class="chatAbled rb_unite" data-select-bank="りそな銀行" aria-hidden="false">
								<a href="/kojin/contact/chat/ds/jutaku.html" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_03.svg" alt="有人チャット"></div>
									<div class="tBold fs18_13 spFs10"><span>有人チャット</span></div>
								</a>
							</li>
							<li>
								<a href="https://ct.resona-chatbot.jp/?site_id=7" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_02.svg" alt="AIチャット"></div>
									<div class="tBold fs18_13 spFs10"><span>AIチャット</span></div>
								</a>
							</li>
							<li class="chatFaqLink">
								<a href="/kojin/jutaku/faq/" data-select-bank="りそな銀行" aria-hidden="false">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
								<a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" data-select-bank="埼玉りそな銀行" aria-hidden="true">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
							</li>
						</ul>
					</div>
				</div>
				<div class="fixedBottomBtn">
					<ul class="fixedBottomList01">
						<li>
							<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">来店予約</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">来店予約</span></a>
						</li>
						<li>
							<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
						</li>
					</ul>
				</div>
			</div>
		</div>
	</div><!-- //.fixedBottom01 .jsFixedBottom01 -->
</div>

	<!-- FOOTER FIXED CONTENT END-->
</div>
<!-- 計測タグここから -->
<script>
(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','//www.google-analytics.com/analytics.js','ga');

<!--RB用 -->
ga('create', 'UA-45901352-1', 'auto');
ga('send', 'pageview');

<!--SR用 -->
ga('create', 'UA-45926600-1', 'auto', {'name':'sr'});
ga('sr.send', 'pageview');

</script>

<!--WebAntenna/RB -->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WATVaj-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WATVaj-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/RB -->

<!-- WebAntenna/SR-->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WAbSBh-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WAbSBh-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/SR-->
<!-- 計測タグここまで -->
<!-- WebAntenna -->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
function wa_rb_jutaku_shinsa_1() {
 if ( typeof( webantenna ) == 'function' ) {
  _wa = new WA_Processor();
  _wa.account = 'WATVaj-1';
  _wa.cv = 'rb_jutaku_shinsa_1';
  webantenna( true );
  var wait = function( msec ) {
   var s = ( new Date() ).getTime();
   while ( ( new Date ).getTime() - s < msec );
  }
  wait( 100 );
 }
}
</script>
<!-- end WebAntenna -->
<script type="text/javascript" src="/kojin/js/le-jutakuloan.js"></script>
<!--// WA 店舗検索 RB -->
<script>
	function wa_rb_kouza_shop_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WATVaj-1';
			_wa.cv = 'rb_kouza_shop_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 店舗検索 RB -->
<!--// WA Webで相談予約 SR -->
<script>
	function wa_sr_sevendays_soudan() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WAbSBh-1';
			_wa.cv = 'sr_sevendays_soudan';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA Webで相談予約 SR -->
<!--// WA 住宅ローン RB -->
<script>
	function wa_rb_jutaku_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WATVaj-1';
			_wa.cv = 'rb_jutaku_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 住宅ローン RB -->
<!--// WA 住宅ローン SR -->
<script>
	function wa_sr_jutaku_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WAbSBh-1';
			_wa.cv = 'sr_jutaku_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 住宅ローン SR -->
</body>
</html>
"""

In [22]:
html_example = """
<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="UTF-8">
<title>3大疾病保障特約｜りそな銀行・埼玉りそな銀行</title>
<meta name="description" content="りそな住宅ローンの3大疾病保障特約付の団体信用生命保険（団信）は、3大疾病（所定のがん、急性心筋梗塞・脳卒中による所定の状態）に該当した場合、住宅ローン残高が0円になる特約付きの住宅ローンです。">
<meta name="keywords" content="3大疾病,特約,団信,住宅ローン,りそな">
<meta name="viewport" content="width=device-width,minimum-scale=1.0,maximum-scale=1.0,user-scalable=0">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="format-detection" content="telephone=no">
<link rel="canonical" href="https://www.resonabank.co.jp/kojin/jutaku/danshin/sandai.html">
<link href="/apple-touch-icon.png" rel="apple-touch-icon-precomposed" type="image/png">
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon">
<link href="/favicon.ico" rel="icon" type="image/x-icon">
<link rel="stylesheet" href="/common/type2018/css/reset.css">
<link rel="stylesheet" href="/common/type2018/css/base.css">
<link rel="stylesheet" href="/common/type2018/css/base_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/parts.css">
<link rel="stylesheet" href="/common/type2018/css/parts_rwd.css">
<link rel="stylesheet" href="/common/type2018/css/font.css">
<link rel="stylesheet" href="/common/type2018/css/yougo_load.css">
<link rel="stylesheet" href="/kojin/jutaku/css/parts_ver4.css">
<link rel="stylesheet" href="/kojin/jutaku/css/parts_ver4_rwd.css">
<link rel="stylesheet" href="/kojin/jutaku/css/jutaku_renewal2023.css">
<link rel="stylesheet" href="/kojin/jutaku/danshin/css/danshin_renewal2023.css">
<script src="/common/type2018/js/jquery-3.5.1.min.js"></script>
<script src="/common/type2018/js/modernizr.custom.39326.js"></script>
<script src="/common/type2018/js/jquery.plugin_jq3.js"></script>
<script src="/common/type2018/js/common_jq3.js"></script>
<script src="/common/type2018/js/drawer.rb.js"></script>
<script src="/common/type2018/js/component_jq3.js"></script>
<script src="/common/type2018/js/common_page_united.js"></script>
<script src="/common/type2018/js/yougo_load_jq3.js"></script>
<script src="/kojin/jutaku/js/component_ver4_jq3.js"></script>
<script src="/kojin/jutaku/danshin/js/danshin_renewal2023.js"></script>
<script src="https://cdn-blocks.karte.io/4e588d08afda9814698a46572afbf27c/builder.js"></script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-5FQFWZZ');</script>
<!-- End Google Tag Manager -->

<!-- print CSS-->
<link rel="stylesheet" href="/common/type2018/css/print.css" media="print">
<!-- FAQ LOADLIST -->
<link rel="stylesheet" href="/kojin/faq/css/faq_load.css">
<script src="/kojin/faq/js/faq_load_jq3.js"></script>
<!-- Google Tag Manager RB -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-MCPCNC');</script>
<!-- End Google Tag Manager RB -->

<!-- Google Tag Manager SR -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-N44PMJ');</script>
<!-- End Google Tag Manager SR -->
<script type="text/javascript">
	window.pageSettings = {};
	window.pageSettings.CATEGORY_NAV_CURRENT = '/kojin/kariru.html';
</script>
<script type="application/ld+json">
	{
		"@context": "https://schema.org",
		"@type": "BreadcrumbList",
		"itemListElement": [{
			"@type": "ListItem",
			"position": 1,
			"name": "銀行ホーム",
			"item": "https://www.resonabank.co.jp/"
		},{
			"@type": "ListItem",
			"position": 2,
			"name": "お金を借りたい",
			"item": "https://www.resonabank.co.jp/kojin/kariru.html"
		},{
			"@type": "ListItem",
			"position": 3,
			"name": "住宅ローン（新規）",
			"item": "https://www.resonabank.co.jp/kojin/jutaku/"
		},{
			"@type": "ListItem",
			"position": 4,
			"name": "団体信用生命保険（団信）",
			"item": "https://www.resonabank.co.jp/kojin/jutaku/danshin/"
		},{
			"@type": "ListItem",
			"position": 5,
			"name": "3大疾病保障特約",
			"item": "https://www.resonabank.co.jp/kojin/jutaku/danshin/sandai.html"
		}]
	}
</script>
</head>
<body class="architectureType2018">
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-5FQFWZZ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->

<!-- Google Tag Manager (noscript) RB -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-MCPCNC"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) RB -->

<!-- Google Tag Manager (noscript) SR -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-N44PMJ"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) SR -->
<div id="drawerMenu">
	<div data-select-bank="りそな銀行" aria-hidden="false"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=rb_unite&loc=1sp"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=rb_unite&loc=1sp">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="/kojin/tsumitatebox/?loc=1sp" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="/kojin/toshin/?loc=1sp">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="/kojin/fundwrap/?loc=1sp" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=rb_unite&loc=1sp">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=rb_unite&loc=1sp">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="/kojin/kinsen/?loc=1sp">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/gaika/?bank=rb_unite&loc=1sp">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="/kojin/shikin_unyo/?loc=1sp">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="/kojin/taishoku_unyo/?loc=1sp"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="/kojin/kokusai/?loc=1sp">個人向け国債・国債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=rb_unite&loc=1sp">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=rb_unite&loc=1sp">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=1sp">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=1sp">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=rb_unite&loc=1sp">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=rb_unite&loc=1sp">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=1sp">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?loc=1sp">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=rb_unite&loc=1sp">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=rb_unite&loc=1sp"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="/kojin/apaman/?loc=1sp"><span>アパート・マンションローン</span></a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=1sp"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=rb_unite&loc=1sp">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="/kojin/souzokusyoukei.html?loc=1sp">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="/kojin/zaisankanri.html?loc=1sp">財産管理業務</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="/kojin/private/?loc=1sp">プライベートバンキング業務</a></li>
											<li class="kojin04Menu07 menuIcon01" role="treeitem"><a href="/nenkin/ideco/?loc=1sp">iDeCo（個人型確定拠出年金）</a></li>
											<li class="kojin04Menu08 menuIcon01" role="treeitem"><a href="/nenkin/401k/?loc=1sp">企業型確定拠出年金</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="/direct/?loc=1sp">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=rb_unite&loc=1sp">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">コメント：電話マークアイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=rb_unite&loc=1sp">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="/kojin/exchange/gaika/?loc=1sp">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="/kojin/kaigai/soukin/?loc=1sp"><span>海外送金</span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=rb_unite&loc=1sp">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=rb_unite&loc=1sp">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=1sp">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="/kojin/koueikyougi/?loc=1sp"><span>即時口座決済</span></a></li>
											<li class="kojin05Menu11 menuIcon01" role="treeitem"><a href="/kojin/yoyaku/?loc=1sp"><span>来店予約サービス</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="/hojin/?loc=1sp">法人のお客さま</a></li>
			<li><a href="/nenkin/?loc=1sp">企業年金・iDeCoのお客さま</a></li>
			<li><a href="/about/?loc=1sp">りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<div data-select-bank="埼玉りそな銀行" aria-hidden="true"><div class="btnSpPage"><a href="javascript:;" class="btnT02"><span>スマートフォン用ページはこちら</span></a></div>
<div class="spDrawerMenu jsDrawerKojin" role="menubar">
	<div class="closeDrawer"><a href="javascript:;"><span>閉じる</span></a></div>
	<nav>
		<ul class="firstHierarchy" role="menu">
			<li>
				<dl class="globalBlock" role="tree">
					<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/sp/"><span>個人のお客さま</span></a></dt>
					<dd role="group" aria-expanded="true">
						<ul class="secondHierarchy">
							<li>
								<dl class="categoryBlock spDrawerDirectLink">
									<dt role="treeitem"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin02MenuTop menuIcon01" role="treeitem"><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたいトップ</a></li>
											<li class="kojin03Menu12 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
											<li class="kojin02Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
											<li class="kojin02Menu02 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
											<li class="kojin02Menu03 menuIcon01" role="treeitem"><a href="/kojin/nisa/?bank=sr_unite">NISA（少額投資非課税制度）</a></li>
											<li class="kojin02Menu04 menuIcon01" role="treeitem"><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
											<li class="kojin02Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
											<li class="kojin02Menu07 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
											<li class="kojin02Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
											<li class="kojin02Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin03MenuTop menuIcon01" role="treeitem"><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたいトップ</a></li>
											<li class="kojin03Menu02 menuIcon01" role="treeitem"><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン（新規）</a></li>
											<li class="kojin03Menu03 menuIcon01" role="treeitem"><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン（お借換え）</a></li>
											<li class="kojin03Menu04 menuIcon01" role="treeitem"><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
											<li class="kojin03Menu05 menuIcon01" role="treeitem"><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
											<li class="kojin03Menu06 menuIcon01" role="treeitem"><a href="/kojin/freeloan/?bank=sr_unite">フリーローン（無担保型）</a></li>
											<li class="kojin03Menu11 menuIcon01" role="treeitem"><a href="/kojin/freeloan/secured/?bank=sr_unite">フリーローン（有担保型）</a></li>
											<li class="kojin03Menu07 menuIcon01" role="treeitem"><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
											<li class="kojin03Menu08 menuIcon01" role="treeitem"><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
											<li class="kojin03Menu09 menuIcon01" role="treeitem"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
											<li class="kojin03Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
											<li class="kojin03Menu13 menuIcon01" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージローン</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin04Menu01 menuIcon01" role="treeitem"><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
											<li class="kojin04Menu02 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
											<li class="kojin04Menu03 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
											<li class="kojin04Menu04 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
											<li class="kojin04Menu05 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
											<li class="kojin04Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
										</ul>
									</dd>
								</dl>
							</li>
							<li>
								<dl class="categoryBlock">
									<dt role="treeitem" aria-selected="true"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
									<dd role="group" aria-expanded="true">
										<ul class="thirdHierarchy">
											<li class="kojin05Menu01 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
											<li class="kojin05Menu02 menuIcon01" role="treeitem"><a href="/kojin/app/?bank=sr_unite">スマートフォンアプリ</a></li>
											<!--<li class="kojin05Menu03 menuIcon01" role="treeitem"><a href="#">電話アイコン</a></li>-->
											<li class="kojin05Menu04 menuIcon01" role="treeitem"><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
											<li class="kojin05Menu09 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
											<li class="kojin02Menu06 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span>海外送金</span><span class="iconWrap"><span class="iconCCobaltGreen icoWAuto">NEW</span></span></a></li>
											<li class="kojin05Menu05 menuIcon01" role="treeitem"><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
											<li class="kojin05Menu07 menuIcon01" role="treeitem"><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
											<li class="kojin05Menu08 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
											<li class="kojin05Menu10 menuIcon01" role="treeitem"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
										</ul>
									</dd>
								</dl>
							</li>
						</ul>
					</dd>
				</dl>
			</li>
			<li><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
			<li><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
		</ul>
	</nav>
</div>
</div>
	<header role="banner">
		<div id="selectBank">
	<div id="selectBankInner">
		<dl>
			<dt class="selectBankTtl">ご利用中(予定)の銀行を選択してください</dt>
			<dd>
				<ul class="selectBankBtnList" role="tablist">
					<li class="select selectBankBtnWrap" data-select-bank="りそな銀行" role="tab" aria-selected="true" aria-label="りそな銀行">
						<a id="selectBankBtnResona" href="javascript:;" class="btnT01 selectBankBtn" tabindex="1"><img src="/common/images/logo.svg" id="svgSelectResona" alt="りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="false"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="false"><span>現在表示している銀行</span></span>
					</li>
					<li class="selectBankBtnWrap" data-select-bank="埼玉りそな銀行" role="tab" aria-selected="false" aria-label="埼玉りそな銀行">
						<a id="selectBankBtnSaitama" href="javascript:;" class="btnT01 selectBankBtn" tabindex="2"><img src="/common/images/logo_header_saitama_mark.svg" id="svgSelectSaitama" alt="埼玉りそな銀行"></a>
						<span class="selectBankBtnBalloon selectBankLabel" aria-hidden="true"><span>選択中です。</span></span>
						<span class="selectBankBtnTxtSelected selectBankLabel" aria-hidden="true"><span>現在表示している銀行</span></span>
					</li>
				</ul>
			</dd>
		</dl>
		<div class="closeWrap">
			<a href="javascript:;" class="btnT01 glyphClose" id="closeSelectBank" tabindex="3"><span>閉じる</span></a>
		</div>
	</div>
</div>

		<div data-select-bank="りそな銀行" aria-hidden="false">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
					<p class="spContent"><a href="/sp/"><img src="/common/images/logo.svg" class="svgTopLogo" alt="りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="1">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/resonabank/red/inetbank/ibank-menusp.shtml" target="_blank" class="glyphLogin"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/" target="_blank" class="glyphGps"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="/">個人のお客さま</a></li>
					<li class="nav02"><a href="/hojin/">法人のお客さま</a></li>
					<li class="nav03"><a href="/nenkin/">企業年金・iDeCoのお客さま</a></li>
					<li class="nav04"><a href="/about/">りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="1">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索" id="qHead">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=rb_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=rb_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=rb_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=rb_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/gaika/?bank=rb_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="/kojin/kokusai/"><span>個人向け国債・国債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=rb_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=rb_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=rb_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10" role="treeitem"><a href="/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13" role="treeitem"><a href="/kojin/reverse-mortgage/?bank=rb_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=rb_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=rb_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=rb_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=rb_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=rb_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink navDropImg navDropImg01 clear">
									<dl>
										<dt><a href="/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=rb_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu05"><a href="/kojin/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu07"><a href="/nenkin/ideco/"><span>iDeCo（個人型確定<br>拠出年金）</span></a></li>
												<li class="kojin04Menu08"><a href="/nenkin/401k/"><span>企業型確定拠出年金</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=rb_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>コメント：電話マークアイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=rb_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="/kojin/kaigai/soukin/"><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=rb_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=rb_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
												<li class="kojin05Menu11"><a href="/kojin/yoyaku/"><span>来店予約サービス</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">		<div class="headerInner headerKojin">
			<noscript>当ウェブサイトでは、JavaScriptを使用しているページがございます。お使いのブラウザでJavaScript機能を無効にされている場合、正しく機能しない、もしくは正しく表示されないことがあります。ご覧になる際にはブラウザ設定でJavaScriptを有効にしてください。</noscript>
			<div class="headerTop">
				<div class="logoWrap">
					<p class="pcContent"><a href="https://www.saitamaresona.co.jp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
					<p class="spContent"><a href="https://www.saitamaresona.co.jp/sp/"><img src="/common/images/logo_header_saitama_mark.svg" class="svgTopLogo" alt="埼玉りそな銀行"></a></p>
				</div>
				<div class="btnSelectBank" aria-hidden="true">
					<a href="#selectBank"><span>銀行を変更</span></a>
					<span class="notifySelectBank">銀行の切替えはこちら</span>
				</div>
				<div class="headerGuide">
					<nav class="infoNav">
						<ul>
							<li class="infoNav01"><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" target="_blank">店舗案内・ATM</a></li>
							<li class="infoNav02"><a href="https://www.saitamaresona.co.jp/kojin/faq/" target="_blank">困ったときは</a></li>
							<li class="infoNav03">
								<div class="headerSearchWrap">
									<form class="headerSearch" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
										<input type="hidden" name="origin" value="5">
										<input type="hidden" name="aj_charset" value="utf-8">
										<input type="search" name="ask" aria-label="サイト内検索" placeholder="キーワードで探す"><button type="submit">検&nbsp;索</button>
									</form>
								</div>
								<a href="javascript:;" class="headerSearchBtn">検索</a>
							</li>
							<li class="infoNav04">
								<figure><img src="/images/index_logo_mygate01.gif" alt="りそなグループmy gate webコミュニケーション"></figure>
								<a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menu.shtml" class="btnT03 glyphLogin" target="_blank"><span>ログイン</span></a>
							</li>
						</ul>
					</nav>
				</div>
				<nav class="spHeaderMenu">
					<ul>
						<li><a href="https://www2.resona-gr.co.jp/net/saitamaresona/sai/inetbank/ibank-menusp.shtml" class="glyphLogin" target="_blank"><span>ログイン</span></a></li>
						<li><a href="/kojin/tenpo_atm/?type=%E5%9F%BC%E7%8E%89%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C" class="glyphGps" target="_blank"><span>店舗ATM</span></a></li>
						<li><a href="javascript:;" class="jsSearchSwitch glyphSearch" role="button" aria-controls="spHeaderSearch"><span>検索</span></a></li>
						<li><a href="javascript:;" class="jsDrawerMenuSwitch glyphMenu01"><span>メニュー</span></a></li>
					</ul>
				</nav>
			</div>
			<!-- ヘッダー緊急メッセージボックスここから -->
			<div class="headerAlertArea">
				
<link rel="stylesheet" href="/common/css/kinkyu_msg.css">
<script src="/common/js/kinkyu_msg.js"></script>
<div id="kinkyuKeisai">
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20210508141840/R20210508141840.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】りそな銀行を騙った不審な電子メール・SMS（ショートメッセージサービス）にご注意ください</span></a></li>
</ul>
<ul class="nonScroll">
<li><a href="/kinkyukeisai/R20250107174755/R20250107174755.html" target="_blank"><span><img src="/common/images/ico_attention01.svg">【重要】ネットワーク不具合による各種サービスへの影響について</span></a></li>
</ul>
</div>

			</div>
			<!-- ヘッダー緊急メッセージボックスここまで -->
			<nav class="gnavArea" role="navigation" aria-label="サイト全体のナビゲーション">
				<ul>
					<li class="nav01 current"><a href="https://www.saitamaresona.co.jp/">個人のお客さま</a></li>
					<li class="nav02"><a href="https://www.saitamaresona.co.jp/hojin/">法人のお客さま</a></li>
					<li class="nav04"><a href="https://www.saitamaresona.co.jp/about/">埼玉りそな銀行について</a></li>
				</ul>
			</nav>
			<div class="spHeaderSearch">
				<div class="jsSearchAcc">
					<div class="spHeaderSearchInner">
						<form role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
							<div class="input">
								<input type="hidden" name="origin" value="5">
								<input type="hidden" name="aj_charset" value="utf-8">
								<input type="search" name="ask" placeholder="サイト内検索">
							</div>
							<div class="button"><button type="submit"><img src="/common/images/icon_search.svg" alt="検索する" class="svgSearch"></button></div>
						</form>
					</div>
				</div>
			</div>
		</div><!-- //.headerInner -->

		<div class="categoryNavNew">
			<nav role="navigation" aria-label="カテゴリのナビゲーション">
				<ul>
					<li class="navKojin01">
						<dl>
							<dt class="categoryTitle notDrop"><a href="/kojin/hiraku/?bank=sr_unite"><span>口座を開きたい</span></a></dt>
						</dl>
					</li>
					<li class="navKojin02">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を増やしたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/huyasu.html?bank=sr_unite"><span>お金を<br>増やしたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu12"><a href="https://www.resonabank.co.jp/kojin/tsumitatebox/" target="_blank"><span>つみたてボックス</span></a></li>
												<li class="kojin02Menu01"><a href="https://www.saitamaresona.co.jp/kojin/toshin/"><span>投資信託</span></a></li>
												<li class="kojin02Menu02"><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank"><span>ファンドラップ</span></a></li>
												<li class="kojin02Menu03"><a href="/kojin/nisa/?bank=sr_unite"><span>NISA<br>(少額投資非課税制度)</span></a></li>
												<li class="kojin02Menu04"><a href="/kojin/teiki/?bank=sr_unite"><span>定期預金</span></a></li>
												<li class="kojin02Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kinsen/"><span>金銭信託</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.resonabank.co.jp/kojin/gaika/?bank=sr_unite"><span>外貨預金</span></a></li>
												<li class="kojin02Menu07"><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/"><span>資金運用プラン</span></a></li>
												<li class="kojin02Menu08"><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/"><span>退職金コース</span></a></li>
												<li class="kojin02Menu09"><a href="https://www.saitamaresona.co.jp/kojin/kokusai/"><span>個人向け国債・国債・地方債</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin03">
						<dl>
							<dt class="categoryTitle"><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を<br>借りたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin03Menu02"><a href="/kojin/jutaku/?bank=sr_unite"><span>住宅ローン<br>(新規)</span></a></li>
												<li class="kojin03Menu03"><a href="/kojin/jutaku/karikae/?bank=sr_unite"><span>住宅ローン<br>(お借換え)</span></a></li>
												<li class="kojin03Menu09"><a href="/kojin/reform/?bank=sr_unite"><span>リフォームローン</span></a></li>
												<li class="kojin03Menu10"><a href="https://www.saitamaresona.co.jp/kojin/apaman/"><span>アパート・<br>マンションローン</span></a></li>
												<li class="kojin03Menu13"><a href="/kojin/reverse-mortgage/?bank=sr_unite"><span>リバースモーゲージ<br>ローン</span></a></li>
												<li class="kojin03Menu05"><a href="/kojin/cardloan/?bank=sr_unite"><span>カードローン</span></a></li>
												<li class="kojin03Menu06"><a href="/kojin/freeloan/?bank=sr_unite"><span>フリーローン<br>（無担保型）</span></a></li>
												<li class="kojin03Menu11"><a href="/kojin/freeloan/secured/?bank=sr_unite"><span>フリーローン<br>（有担保型）</span></a></li>
												<li class="kojin03Menu07"><a href="/kojin/mycar/?bank=sr_unite"><span>マイカーローン</span></a></li>
												<li class="kojin03Menu08"><a href="/kojin/edu/?bank=sr_unite"><span>教育ローン</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin04">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に備えたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html"><span>将来や万一に<br>備えたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin04Menu01"><a href="/kojin/hoken/?bank=sr_unite"><span>保険</span></a></li>
												<li class="kojin04Menu02"><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html"><span>相続・承継</span></a></li>
												<li class="kojin04Menu03"><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html"><span>財産管理業務</span></a></li>
												<li class="kojin04Menu04"><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/"><span>きょういく信託</span></a></li>
												<li class="kojin04Menu05"><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/"><span>プライベート<br>バンキング業務</span></a></li>
												<li class="kojin04Menu06"><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html"><span>iDeCo<br>(個人型確定拠出年金)</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
					<li class="navKojin05">
						<dl>
							<dt class="categoryTitle"><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に使いたい</span></a></dt>
							<dd class="dropBlock dropCatLine02">
								<div class="navDropLink clear">
									<dl>
										<dt><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html"><span>便利にお得に<br>使いたい<br>トップ</span></a></dt>
										<dd>
											<ul>
												<li class="kojin05Menu01"><a href="https://www.saitamaresona.co.jp/direct/"><span>マイゲート</span></a></li>
												<li class="kojin05Menu02"><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
												<!--<li class="kojin05Menu03"><a href="#"><span>電話アイコン</span></a></li>-->
												<li class="kojin05Menu04"><a href="/club/?bank=sr_unite"><span>りそなクラブ</span></a></li>
												<li class="kojin02Menu06"><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/"><span class="dropLinkNewIcon01">New</span><span>海外送金</span></a></li>
												<li class="kojin05Menu05"><a href="/kojin/visa_debit/?bank=sr_unite"><span>Visaデビットカード</span></a></li>
												<li class="kojin05Menu07"><a href="/kojin/credit/?bank=sr_unite"><span>クレジットカード</span></a></li>
												<li class="kojin05Menu08"><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/"><span>ペイジー料金<br>払込サービス</span></a></li>
												<li class="kojin05Menu10"><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/"><span>即時口座決済</span></a></li>
											</ul>
										</dd>
									</dl>
								</div>
								<div class="closeNav"><a href="javascript:;" class="btnT01 glyphArrowCircleTop"><span>閉じる</span></a></div>
							</dd>
						</dl>
					</li>
				</ul>
			</nav>
		</div><!-- //#categoryNav -->
</div>
	</header><!-- //#header -->
	<div id="topicPath">
		<div class="topicPathInner" data-select-bank="りそな銀行" aria-hidden="false">
			<span><a href="/"><span>銀行ホーム</span></a></span>
			<span><a href="/kojin/kariru.html?bank=rb_unite"><span>お金を借りたい</span></a></span>
			<span><a href="/kojin/jutaku/?bank=rb_unite"><span>住宅ローン（新規）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/?bank=rb_unite"><span>団体信用生命保険（団信）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite"><span>3大疾病保障特約</span></a></span>
		</div>
		<div class="topicPathInner" data-select-bank="埼玉りそな銀行" aria-hidden="true">
			<span><a href="https://www.saitamaresona.co.jp/"><span>銀行ホーム</span></a></span>
			<span><a href="/kojin/kariru.html?bank=sr_unite"><span>お金を借りたい</span></a></span>
			<span><a href="/kojin/jutaku/?bank=sr_unite"><span>住宅ローン（新規）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/?bank=sr_unite"><span>団体信用生命保険（団信）</span></a></span>
			<span><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite"><span>3大疾病保障特約</span></a></span>
		</div>
	</div><!-- #topicPath -->
	<div id="wrap">
		<div id="mainArea">
			<main>
				<section>
					<div class="coverTitle">
						<div class="coverTitleInner">
							<h1 class="h1Title01"><span class="titleInner">3大疾病保障特約</span></h1>
							<p class="coverTitleLead">りそな住宅ローンの3大疾病保障特約付の団体信用生命保険（団信）は、3大疾病（所定のがん、急性心筋梗塞・脳卒中による所定の状態）に該当した場合、住宅ローン残高が0円になる特約付きの住宅ローンです。</p>
						</div><!-- /.mainCoverInner -->
					</div>
					<div class="sandaiMvArea">
						<div class="sandaiMvAreaInner newAdjustment">
							<p class="tBold tCenter fsXXL">全死亡者の約半数<sup class="fs40_26 spFs14">※1</sup>である<br class="spContent">死因をカバーする</p>
							<div class="sandaiMvTextArea mt30 spMt15 mb15">
								<div class="blockInnerB01 tCenter">
									<p class="u-fsXXL tBold"><span class="v2-textMarker01">3大疾病保障特約</span></p>
									<div><img src="/kojin/jutaku/danshin/images/sandai_img_mv01.png" alt="死亡・高度障害 がん 脳卒中・急性心筋梗塞" class="w530 pcContent"><img src="/kojin/jutaku/danshin/images/sandai_img_mv01_sp.png" alt="死亡・高度障害 がん 脳卒中・急性心筋梗塞" class="spContent"></div>
									<p class="fsLarge tBold">所定の状態<sup class="spFs14">※2</sup>で住宅ローン残高<span class="tOrange01"><span class="fs20_36 spFs28">0</span>円</span></p>
									<div><a href="#ancApplication" class="btnT03 btnSMiddle w400 spW280"><span>住宅ローンのお申込みはこちら</span></a></div>
								</div>
							</div>
							<ul class="listMemo02 fsSmall">
								<li><span class="listMark">※1</span><span class="listInner">出典元は令和3年（2021）人口動態統計（確定数）になります。算出方法は死亡総数に占める悪性新生物（26.5％） 心疾患（14.9％）脳血管疾患（7.3％）の合計になります。</span></li>
								<li><span class="listMark">※2</span><span class="listInner">ただし、お借入日から90日以内に診断確定されたがんなど対象外となる場合があります。詳細は<a href="#anchor01" class="tUnderLine"><span>商品概要</span></a>をご確認ください。</span></li>
							</ul>
						</div>
					</div>
					<div class="danshinNavArea">
						<div class="danshinNavAreaInner">
							<div class="blockInnerB03">
								<p class="tBold tCenter">りそなはあなたにあった保障タイプが選べる</p>
								<ul class="layoutColumnA3_3 danshinColumnLayout01">
									<li>
										<a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="v3-cardblockA01 v3-noMark01">
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">がん保障<br class="spContent">特約</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav01.png" alt="がん保障特約"></figure></dd>
												</dl>
											</div>
										</a>
										<a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="v3-cardblockA01 v3-noMark01">
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">がん保障<br class="spContent">特約</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav01.png" alt="がん保障特約"></figure></dd>
												</dl>
											</div>
										</a>
									</li>
									<li>
										<a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="v3-cardblockA01 v3-noMark01 current">
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">3大疾病<br class="spContent">保障特約</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav02.png" alt="3大疾病保障特約"></figure></dd>
												</dl>
											</div>
										</a>
										<a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="v3-cardblockA01 v3-noMark01 current">
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">3大疾病<br class="spContent">保障特約</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav02.png" alt="3大疾病保障特約"></figure></dd>
												</dl>
											</div>
										</a>
									</li>
									<li>
										<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="v3-cardblockA01 v3-noMark01">
											<div class="danshinPosItem01"><img src="/kojin/jutaku/danshin/images/danshin_img_nav01.png" alt="銀行員おすすめ" class="w170 spW120"></div>
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">団信革命</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav03.png" alt="団信革命"></figure></dd>
												</dl>
											</div>
										</a>
										<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="v3-cardblockA01 v3-noMark01">
											<div class="danshinPosItem01"><img src="/kojin/jutaku/danshin/images/danshin_img_nav01.png" alt="銀行員おすすめ" class="w170 spW120"></div>
											<div class="v3-blockInner">
												<dl class="v3-detailListB01">
													<dt class="v3-title tile01 v3-dispFlex v3-alignCenter v3-justifyCenter spFs16">団信革命</dt>
													<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_nav03.png" alt="団信革命"></figure></dd>
												</dl>
											</div>
										</a>
									</li>
								</ul>
								<div class="tCenter"><a href="#anc01" class="glyphArrowTriangleBottom"><span>保障内容や金額を比較する</span></a></div>
							</div>
						</div>
					</div>
					<div class="mainInner">
						<div class="mainContents">
							<div class="blockInnerB03 tCenter">
	<div>
		<a data-select-bank="りそな銀行" aria-hidden="false" href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 w300 spW100p mb00"><span>団信革命とは？</span></a>
		<a data-select-bank="埼玉りそな銀行" aria-hidden="true" href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight overT01 w300 spW100p mb00"><span>団信革命とは？</span></a>
	</div>
</div>

							<section>
								<h2 class="h2Title01 noBorder tCenter"><span class="titleInner">3大疾病保障特約のメリット</span></h2>
								<div class="blockInnerB05">
									<div class="blockInnerB03 tCenter">
										<div class="v4-commentColumnTitle01">
											<div class="v4-glyphComment"><img src="/kojin/jutaku/danshin/images/sandai_ico01.png" alt="備える"></div>
											<div class="v4-commentColumnText">
												<p><span class="v4-tGreen01 v4-textMarker01 tBold">特にリスクが高い<br class="spContent">3大疾病に特化して備える</span></p>
											</div>
										</div>
										<div><img src="/kojin/jutaku/danshin/images/sandai_img01.png" alt="死亡・高度障害 3大疾病" class="w550"></div>
										<p class="tBold">3大疾病患者数<span class="fsExtraL">462.9万人</span></p>
										<div class="blockInnerB01 tLeft">
											<p>死亡や高度障害状態はもちろんのこと、全死亡者の半数であるがん、急性心筋梗塞、脳卒中の3大疾病をカバーしています。</p>
											<p>出典：厚生労働省「生活のしづらさなどに関する調査」（2016年）、厚生労働省「社会福祉施設等調査」（2018年）等より厚生労働省社会・援護局障害保健福祉部で作成</p>
										</div>
									</div>
									<div class="blockInnerB03 tCenter">
										<div class="v4-commentColumnTitle01">
											<div class="v4-glyphComment"><img src="/kojin/jutaku/danshin/images/sandai_ico02.png" alt="安心"></div>
											<div class="v4-commentColumnText">
												<p><span class="v4-tGreen01 v4-textMarker01 tBold">所定の状態に該当すれば！<br class="spContent">適用される保障条件</span></p>
											</div>
										</div>
										<div class="danshinFailureStory">
											<div class="danshinFailureStoryImg"><img src="/kojin/jutaku/danshin/images/index_img_renewal05.png" alt="困る男性"></div>
											<div class="blockInnerB01 tCenter">
												<div class="danshinFailureStoryTag">団信の失敗話</div>
												<p class="spTLeft">保障の数が多い団信があったから選んだけど「余命宣告」などハードルの高い条件があった…</p>
											</div>
										</div>
										<p class="tBold tCenter">りそなは適用されやすい保障条件が充実！</p>
										<div class="layoutColumnA2_2">
											<div class="colorCardA01">
												<div class="cardInner tile05 newAdjustment">
													<div class="blockInnerB03 tCenter">
														<div class="blockInnerB02">
															<div class="blockInnerB01">
																<div class="tile03"><img src="/kojin/jutaku/danshin/images/sandai_img02.png" alt="がん" class="spW48"></div>
																<p class="fsSmall tile04">がん</p>
															</div>
														</div>
														<div class="blockInnerB01">
															<p><span class="tBold tGreen01">所定のがん</span><sup>※</sup>と<br class="spContent">診断されたら</p>
															<ul class="listMemo01 tLeft fsSmall">
																<li><span class="listMark">※</span>ただし、お借入日から90日以内に診断確定されたがんなど対象外となる場合があります。詳細は<a href="#anchor01" class="tUnderLine"><span>商品概要</span></a>をご確認ください。</li>
															</ul>
														</div>
													</div>
												</div>
											</div><!-- //.colorCardA01 -->
											<div class="colorCardA01">
												<div class="cardInner tile05 newAdjustment">
													<div class="blockInnerB03 tCenter">
														<div class="blockInnerB05">
															<div class="layoutColumnA2_2 sandaiLayoutColumn">
																<div class="blockInnerB01 spMt00">
																	<div class="tile03"><img src="/kojin/jutaku/danshin/images/sandai_img03.png" alt="ケガ病気による障害" class="spW48"></div>
																	<p class="fsSmall tile04">脳卒中</p>
																</div>
																<div class="blockInnerB01 spMt00">
																	<div class="tile03"><img src="/kojin/jutaku/danshin/images/sandai_img04.png" alt="要介護2" class="spW48"></div>
																	<p class="fsSmall tile04">急性心筋<br class="spContent">梗塞</p>
																</div>
															</div>
														</div>
														<p>60日以上所定の<br class="spContent">状態が継続または<br class="spContent"><span class="tBold tGreen01">所定の手術をしたら</span></p>
													</div>
												</div>
											</div><!-- //.colorCardA01 -->
										</div>
										<div><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" class="btnT01 btnSMiddle glyphArrowTriangleRight v3-jsModal01 overT01"><span>日数以外にも保障される条件は様々！<br class="spContent">所定の状態を確認する</span></a></div>
										<div class="colorCardA01 marginLvH2">
											<div class="cardInner newAdjustment">
												<div class="kakumeiPosIcon01"><img src="/kojin/jutaku/danshin/images/kakumei_ico03.png" alt=""></div>
												<div class="blockInnerB03 tCenter">
													<p class="tBold">たとえ、<span class="tOrange01">仕事に復帰しても</span><span class="fsExtraS">※</span><br class="spContent">住宅ローン残高が<span class="tOrange01">0円</span></p>
													<div><img src="/kojin/jutaku/danshin/images/kakumei_img07.png" alt="がん 脳血管疾患 心疾患 退院後・・・残高0円 仕事復帰も" class="w450"></div>
													<p class="tLeft">りそなは所定の状態に該当すれば仕事復帰しても<span class="fsExtraS">※</span>住宅ローン残高0円になるため、安心して社会復帰ができます。</p>
													<ul class="listMemo01 fsSmall tLeft">
														<li><span class="listMark">※</span>急性心筋梗塞の場合、初診日からその日を含めて60日以上労働の制限を必要とする状態が継続したと医師によって診断されたとき等、状態により支払要件は異なります。詳細は<a href="#anchor01" class="tUnderLine"><span>商品概要</span></a>をご確認ください。</li>
													</ul>
												</div>
											</div>
										</div>
									</div>
									<div class="blockInnerB03 tCenter">
										<div class="v4-commentColumnTitle01">
											<div class="v4-glyphComment"><img src="/kojin/jutaku/danshin/images/kakumei_ico04.png" alt="全額" class="w450"></div>
											<div class="v4-commentColumnText">
												<p><span class="v4-tGreen01 v4-textMarker01 tBold">所定の状態に該当すれば！<br class="spContent">住宅ローン残高0円！</span></p>
											</div>
										</div>
										<div class="tCenter"><img src="/kojin/jutaku/danshin/images/kakumei_img08.png" alt="所定の状態になる りそななら残高全額0円" class="w450"></div>
										<p class="tLeft">「保障される額は半額」や「生活習慣病の場合は入院した月だけ」という場合もあるので必ず保障の範囲は確認しておくようにしましょう。<br>りそなは所定の状態に該当すれば、住宅ローン残高の全額が保障されます！</p>
									</div>
								</div>
							</section>
						</div>
					</div>
					<section id="ancApplication" class="v2-bgYellow01 v3_fullColumnA01 danshinApplicationBlock01">
	<div class="columnInner v2-cardBlockA03 blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb20"><span class="titleInner">団信特約の申込方法</span></h2>
		<section class="cardBlockA01 bgWhite spMt00">
			<div class="blockInnerB01 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_01.png" alt="Step0" class="danshinStepLabel01">（どの団信がいいか分からない方）</span><span class="title tBold">入りたい団信特約を決める</span>
				</h3>
				<div class="layoutColumnA2_2 spMt00">
					<p class="w80p spW79p">店舗に来店いただければ金融のプロの銀行員が一緒にあなたに最適な<br class="pcContent">団信選びをお手伝いします。</p>
					<figure class="w16p spW17p danshinStepImg01"><img src="/kojin/jutaku/danshin/images/danshin_img_04.png" alt="銀行員のイラスト画像"></figure>
				</div>
				<div class="blockInnerB01 tCenter">
					<div class="v4-decorationTextB02 fs18_20 spFs16 tBold">
						<span data-select-bank="りそな銀行" aria-hidden="false">土・日・祝日や17時以降も営業</span>
						<span data-select-bank="埼玉りそな銀行" aria-hidden="true">土・日・祝日も営業</span>
					</div>
					<div>
						<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで来店予約</span></a>
						<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで来店予約</span></a>
					</div>
				</div>
			</div>
		</section>
		<section class="cardBlockA01 bgWhite danshinBeforeChain01">
			<div class="blockInnerB01 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_02.png" alt="Step1" class="danshinStepLabel01"></span><span class="title tBold">店舗またはWEBから住宅ローンの事前審査</span>
				</h3>
				<div class="layoutColumnA2_2 tCenter spMt00">
					<div class="blockInnerB01">
						<div class="v4-decorationTextB02 fs18_20 spFs16 tBold">
							<span data-select-bank="りそな銀行" aria-hidden="false">土・日・祝日や<br class="spContent">17時以降も営業</span>
							<span data-select-bank="埼玉りそな銀行" aria-hidden="true">土・日・祝日も<br class="spContent">営業</span>
						</div>
						<div>
							<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">来店予約</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">来店予約</span></a>
						</div>
					</div>
					<div class="blockInnerB01">
						<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>審査結果も<br class="spContent">Webで確認！</span></div>
						<div>
							<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
						</div>
					</div>
				</div>
				<div class="newAdjustment">
					<ul class="listMemo01 fsSmall tCenter spTLeft">
						<li><span class="listMark">※</span>事前審査では団信のお申込みはございません。</li>
					</ul>
				</div>
			</div>
		</section>
		<section class="cardBlockA01 bgWhite danshinBeforeChain01">
			<div class="blockInnerB05 w64p spW100p mlrAuto">
				<h3 class="danshinStepTitle01 fsLarge spFsMiddle">
					<span class="tOrange01"><img src="/kojin/jutaku/danshin/images/danshin_label_03.png" alt="Step2" class="danshinStepLabel01"></span><span class="title tBold">事前審査の結果が届いたら団信特約申込</span>
				</h3>
				<div id="danshinSelectTabAcc01" class="tabList jsTabAcc02">
					<ul role="tablist" class="tab2column clear tCenter">
						<li><div id="danshinSelectTabSwitch01" role="tab" aria-selected="true" aria-controls="danshinSelectTabCotent01" data-tab-parent="danshinSelectTabAcc01"><a href="javascript:;"><span>通常の団信</span></a></div></li>
						<li><div id="danshinSelectTabSwitch02" role="tab" aria-selected="false" aria-controls="danshinSelectTabCotent02" data-tab-parent="danshinSelectTabAcc01"><a href="javascript:;"><span>ペアローン限定の<br class="spContent">団信</span></a></div></li>
					</ul>
					<div id="danshinSelectTabCotent01" class="blockInnerB01 spPb00 spPl00 spPr00" role="tabpanel" aria-labelledby="danshinSelectTabSwitch01" aria-hidden="false">
						<div class="tCenter">
							<a href="/kojin/jutaku/danshin/login/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ネットDe団信ログイン</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/danshin/login/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ネットDe団信ログイン</span></a>
						</div>
						<div class="newAdjustment">
							<ul class="listMemo01 fsSmall">
								<li><span class="listMark">※</span>ネットDe団信は事前審査完了後に発行されるユーザーID、パスワードをお持ちの方のみご利用いただけます。</li>
							</ul>
						</div>
					</div>
					<div id="danshinSelectTabCotent02" class="blockInnerB01 spPb00 spPl00 spPr00" role="tabpanel" aria-labelledby="danshinSelectTabSwitch02" aria-hidden="true">
						<div class="tCenter">
							<a href="/kojin/jutaku/danshin/login_pair/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w480 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ニッセイ団信ダイレクトサービス<br class="spContent"><span class="ml05 spMl00">ログイン</span></span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/danshin/login_pair/" target="_blank" class="btnT02 btnSMiddle iconNewWindow w480 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ニッセイ団信ダイレクトサービス<br class="spContent"><span class="ml05 spMl00">ログイン</span></span></a>
						</div>
						<div class="newAdjustment">
							<ul class="listMemo01 fsSmall">
								<li><span class="listMark">※</span>ニッセイ団信ダイレクトサービスは事前審査完了後に発行されるユーザーID、パスワードをお持ちの方のみご利用いただけます。</li>
							</ul>
						</div>
					</div>
				</div>
			</div>
		</section>
		<p class="danshinAttentionIcon01 tCenter"><span>団信は<span class="tBold">住宅ローン申込時にしか手続きできません！</span>入っておけばよかったと後悔する前にもう一度検討してみてください。</span></p>
	</div>
</section>

					<div class="mainInner">
						<div class="mainContents">
							<dl class="blowoutBlockA01">
								<dt class="title">解説動画でもっと知る<span class="pcContent"> </span><br class="spContent">りそなの保障特約付団信</dt>
								<dd class="detail">
									<div class="tCenter"><a href="https://api01-platform.stream.co.jp/apiservice/plt3/NzU3%23MzQ3Ng%3d%3d%23280%23168%230%2333E220F86400%23OzEwOzEwOzEw%23" target="_blank"><img src="/kojin/jutaku/danshin/images/common_img_movie01.png" alt="団信革命 りそなの「団信革命」は所定の状態に該当すれば全額保障！"></a></div>
								</dd>
							</dl>
							<section id="anc01">
								<h2 class="h2Title01 noBorder tCenter"><span class="titleInner">りそなの他特約との比較</span></h2>
								<div class="blockInnerB05">
	<div id="danshinComparisonTableTab01" class="tabList jsTabAcc02">
		<ul role="tablist" class="tab2column clear">
			<li><div id="comparisonTableTabSwitch01" role="tab" aria-selected="true" aria-controls="comparisonTableTabContent01" data-tab-parent="danshinComparisonTableTab01"><a href="javascript:;" class="tCenter"><span>通常の団信</span></a></div></li>
			<li><div id="comparisonTableTabSwitch02" role="tab" aria-selected="false" aria-controls="comparisonTableTabContent02" data-tab-parent="danshinComparisonTableTab01"><a href="javascript:;" class="tCenter"><span>ペアローン限定の<br class="spContent">団信</span></a></div></li>
		</ul>
		<div id="comparisonTableTabContent01" class="blockInnerB05 mt80 spMt30 pl00 pr00 spPb00" role="tabpanel" aria-labelledby="comparisonTableTabSwitch01" aria-hidden="false">
			<table class="v4-tableLayoutA01 danshinComparisonTable01">
				<thead>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a><a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a></th>
						<th class="w30p v4-tableBg03"><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a></th>
						<th class="w30p v4-tableBg02 danshinBalloon01"><img src="/kojin/jutaku/danshin/images/danshin_balloon_01.png" alt="銀行員おすすめ" class="danshinBalloonImg01"><a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a></th>
					</tr>
				</thead>
				<tbody>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="3" class="spFsMiddle">
							<div>
								<div class="blockInnerB05">
									<p><span class="danshinCalcIcon01 spFs14"><span>月々いくら上乗せになるのか調べよう</span></span></p>
									<form action="#" class="danshinTableSimForm01 blockInnerB03 tCenter w53p spW100p mlrAuto spMt15">
										<dl class="danshinTableSimColumn01">
											<dt>年齢</dt>
											<dd class="layoutColumnA2_2">
												<div><a href="javascript:;" data-age="under40" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01 active">40歳未満</a></div>
												<div><a href="javascript:;" data-age="over40" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01">40歳以上</a></div>
											</dd>
										</dl>
										<dl class="danshinTableSimColumn01 spMt15">
											<dt>借りたい金額</dt>
											<dd class="danshinTableSimInputBlock01 mlrAuto">
												<input id="danshinSimInput01" class="danshinTableSimInput01" type="tel" name="danshinBorrowingAmount" value="" maxlength="5" autocomplete="off">
												<span class="unit">万円</span>
											</dd>
										</dl>
										<div class="newAdjustment spMt15">
											<ul class="listMemo01 tLeft danshinTableSimMemo01">
												<li><span class="listMark">※</span>変動金利<span id="floatingRate"></span>％、返済期間<span id="repaymentPeriod"></span>年で計算</li>
												<li><span class="listMark">※</span>特約付団信は50歳以上の方は申込みできません。</li>
											</ul>
										</div>
										<input type="text" name="typeHidden" disabled>
									</form>
								</div>
							</div>
						</th>
					</tr>
					<tr class="separatorLineNone danshinTableSimAdjust">
						<td class="v4-tableBg04"><span></span></td>
						<td class="v4-tableBg03"><span></span></td>
						<td class="v4-tableBg02"><span></span></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13">上乗せ金利</span></p></th>
					</tr>
					<tr class="cellNarrow separatorLineNone">
						<td class="v4-tableBg04 fsMaximum tBold">年<span id="interestRate_cancer"></span>％</td>
						<td class="v4-tableBg03 fsMaximum tBold">年<span id="interestRate_disease"></span>％</td>
						<td class="v4-tableBg02 fsMaximum tBold">年<span id="interestRate_danshin"></span>％</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13">月々の上乗せ金額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 fsMaximum tBold"><span id="addOn_cancer"></span>円</td>
						<td class="v4-tableBg03 fsMaximum tBold"><span id="addOn_disease"></span>円</td>
						<td class="v4-tableBg02 fsMaximum tBold"><span id="addOn_danshin"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="3"><p><span class="spFs13">毎月の返済額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 fsMaximum spFsLarge tBold"><span id="repaymentAmount_cancer"></span>円</td>
						<td class="v4-tableBg03 fsMaximum spFsLarge tBold"><span id="repaymentAmount_disease"></span>円</td>
						<td class="v4-tableBg02 fsMaximum spFsLarge tBold"><span id="repaymentAmount_danshin"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="3" class="spFsMiddle"><p><span class="danshinSecurityIcon01"><span>保障の範囲の違いを知ろう</span></span></p></th>
					</tr>
					<tr class="separatorLineNone">
						<td class="v4-tableBg04"><span></span></td>
						<td class="v4-tableBg03"><span></span></td>
						<td class="v4-tableBg02"><span></span></td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="3"><p><span class="spFs13">死亡・<a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" class="indexIcoModal01 v3-jsModal01"><span>高度障害状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor06" class="indexIcoModal01 danshin-jsModal01"><span>所定のがん</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor02" class="indexIcoModal01 danshin-jsModal01"><span>急性心筋梗塞・脳卒中</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor03" class="indexIcoModal01 danshin-jsModal01"><span>病気・ケガによる16の状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="3"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail.html" data-modal="modalBody" data-modal-anchor02="modalAnchor04" class="indexIcoModal01 danshin-jsModal01"><span>所定の要介護状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg03 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW25p"></td>
						<td class="v4-tableBg02 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW25p"></td>
					</tr>
				</tbody>
				<tfoot>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a><a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>がん保障特約</span></a></th>
						<th class="w30p v4-tableBg03"><a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a><a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>3大疾病<br class="spContent">保障特約</span></a></th>
						<th class="w30p v4-tableBg02"><a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>団信革命</span></a></th>
					</tr>
				</tfoot>
			</table>
		</div>
		<div id="comparisonTableTabContent02" class="blockInnerB05 pl00 pr00 spP00" role="tabpanel" aria-labelledby="comparisonTableTabSwitch02" aria-hidden="true">
			<table class="v4-tableLayoutA01 danshinComparisonTable01 mt40 spMt20">
				<thead>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg01"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a></th>
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a></th>
					</tr>
				</thead>
				<tbody>
					<tr class="separatorLine separatorLineNone danshinTableColorBlock01">
						<th colspan="2" class="spFsMiddle">
							<div>
								<div class="blockInnerB05">
									<p><span class="danshinCalcIcon01 spFs14"><span>月いくら追加になるのか調べよう</span></span></p>
									<div class="v4-decorationTextB02 fs18_20 spFs16 tBold spMt15"><span>お1人分ずつ確認できます</span></div>
									<form action="#" class="danshinTableSimForm01 blockInnerB03 tCenter w53p spW100p mlrAuto spMt15">
										<dl class="danshinTableSimColumn01">
											<dt>年齢</dt>
											<dd class="layoutColumnA2_2">
												<div><a href="javascript:;" data-age="under35" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01 active">35歳未満</a></div>
												<div><a href="javascript:;" data-age="over35" class="btnT01 btnSMiddle w200 spW100p mb00 danshinTableSimBtn01">35歳以上</a></div>
											</dd>
										</dl>
										<dl class="danshinTableSimColumn01 typePair spMt15">
											<dt>借りたい金額</dt>
											<dd class="danshinTableSimInputWrap01 mlrAuto">
												<div class="danshinTableSimInputBlock01">
													<input id="danshinSimInput02" class="danshinTableSimInput01" type="tel" name="danshinBorrowingAmount" value="" maxlength="5" autocomplete="off">
													<span class="unit">万円</span>
												</div>
												<p class="v3-itemBalloonA01 v3-balloonTopArrow danshinTableSimBalloon01 fsExtraS">お1人で借りる分の<br>金額を入力</p>
											</dd>
										</dl>
										<input type="text" name="typeHidden" disabled>
									</form>
									<div class="newAdjustment w66p spW100p mlrAuto spMt15">
										<ul class="listMemo01 tLeft danshinTableSimMemo01">
											<li><span class="listMark">※</span>変動金利<span id="floatingRate_pair"></span>％、返済期間<span id="repaymentPeriod_pair"></span>年で計算</li>
											<li><span class="listMark">※</span>りそなのペア団信は50歳以上の方は申込みできません</li>
											<li><span class="listMark">※</span>ペアがん団信の上乗せ金利は、2025年9月末までにお借入れされた方限定の特別な優遇金利です。</li>
											<li><span class="listMark">※</span>ペアのうち片方が全額繰上返済しても、ペア相手の上乗せ金利は変わりません。</li>
											<li><span class="listMark">※</span>ペア団信は、既加入金額も含めてご夫婦等合算で2億円以内です。</li>
										</ul>
									</div>
								</div>
							</div>
						</th>
					</tr>
					<tr class="separatorLineNone danshinTableSimAdjust02">
						<td class="v4-tableBg01"><span></span></td>
						<td class="v4-tableBg04"><span></span></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">上乗せ金利</span></p></th>
					</tr>
					<tr class="cellNarrow separatorLineNone">
						<td class="v4-tableBg01 fsMaximum tBold">年<span id="interestRate_pair"></span>％</td>
						<td class="v4-tableBg04 fsMaximum tBold">年<span id="interestRate_pairCancer"></span>％</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">月々の上乗せ金額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 fsMaximum tBold"><span id="addOn_pair"></span>円</td>
						<td class="v4-tableBg04 fsMaximum tBold"><span id="addOn_pairCancer"></span>円</td>
					</tr>
					<tr class="separatorLine separatorLineNone">
						<th colspan="2"><p><span class="spFs13">毎月の返済額</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 fsMaximum spFsLarge tBold"><span id="repaymentAmount_pair"></span>円</td>
						<td class="v4-tableBg04 fsMaximum spFsLarge tBold"><span id="repaymentAmount_pairCancer"></span>円</td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">死亡・<a href="/kojin/jutaku/danshin/modal/detail_pair.html" data-modal="modalBody" class="indexIcoModal01 v3-jsModal01"><span>高度障害状態</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13"><a href="/kojin/jutaku/danshin/modal/detail_pair.html" data-modal="modalBody" data-modal-anchor02="modalPairAnchor02" class="indexIcoModal01 danshin-jsModal01"><span>所定のがん</span></a></span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
					<tr class="separatorLine">
						<th colspan="2"><p><span class="spFs13">リビング・ニーズ</span></p></th>
					</tr>
					<tr class="cellNarrow">
						<td class="v4-tableBg01 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_03.png" alt="×" class="spW16p"></td>
						<td class="v4-tableBg04 tBold"><img src="/kojin/jutaku/danshin/images/index_ico_02.png" alt="〇" class="spW16p"></td>
					</tr>
				</tbody>
				<tfoot>
					<tr class="vMiddle">
						<th class="w30p v4-tableBg01"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペア一般団信</span></a></th>
						<th class="w30p v4-tableBg04"><a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a><a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight danshinLinkItem"><span>ペアがん団信</span></a></th>
					</tr>
				</tfoot>
			</table>
			<div class="v4-balloonCommentBox01 w85p spW100p mlrAuto">
				<div class="v4-balloonCommentBoxInner">
					<p class="tBold tCenter spTLeft">金利や返済期間など、より詳しいシミュレーションをご希望の方は住宅ローンシミュレーションをご活用ください！</p>
				</div>
			</div>
			<div class="layoutColumnA2_1 w84p spW100p mlrAuto">
				<div class="tCenter">
					<a href="/kojin/jutaku/simulation/shinki/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローンシミュレーション<br class="spContent">（新規）</span></a>
					<a href="/kojin/jutaku/simulation/shinki/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローンシミュレーション<br class="spContent">（新規）</span></a>
				</div>
				<div class="tCenter">
					<a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローンシミュレーション<br class="spContent">（お借換え）</span></a>
					<a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow w480 overT01 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローンシミュレーション<br class="spContent">（お借換え）</span></a>
				</div>
			</div>
		</div>
	</div>
	<div class="tCenter"><a href="/kojin/jutaku/danshin/modal/check.html" data-modal="modalBody" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 overT01 v3-jsModal01 mb00"><span>団信選びセルフチェック</span></a></div>
</div>

							</section>
							<div class="colorCardA01 sandaiMargin">
								<div class="cardInner newAdjustment">
									<div class="blockInnerB03">
										<div class="tCenter">
											<div class="kakumeiPos01">
												<div class="kakumeiPosIco02"><img src="/kojin/jutaku/danshin/images/sandai_ico03.png" alt="ちょっと待った！"></div>
												<h2 class="fsExtraL spFs20 tCenter tBold">本当にその保障内容だけで大丈夫？</h2>
											</div>
										</div>
										<div class="layoutColumnA2_1 v2-wide01">
											<div class="v4-detailBlockA01 tCenter">
												<div class="blockInnerB05 tile06 spTileClear">
													<p class="tBold tGreen01">ケガや病気による<br class="spContent">要介護のリスクは年々高まっている…</p>
													<p>要介護（要支援）認定者数の推移</p>
													<div><img src="/kojin/jutaku/danshin/images/sandai_img05.png" alt="要介護認定者数の推移" class="w400 spW280"></div>
													<p class="tLeft">厚生労働省/「介護保険事業状況報告（暫定）」（2000年4月分、2020年3月分）</p>
												</div>
											</div>
											<div class="v4-detailBlockA01 tCenter">
												<div class="blockInnerB05 tile06 spTileClear">
													<p class="tBold tGreen01">公的な保険では要介護状態は<br class="spContent">保障されない場合も…</p>
													<div><img src="/kojin/jutaku/danshin/images/sandai_img06.png" alt="公的な建物とケガしている男性" class="w360 spW240"></div>
													<p class="tLeft">要介護状態になっても、39歳以下の方は公的介護保険の給付対象になりません。40～64歳の方は交通事故が原因のときには給付対象になりません。</p>
												</div>
											</div>
										</div>
									</div>
									<div class="sandaiPosItem01">
										<p class="sandaiPosItemText01">明日起きるかもしれないケガによる所定の状態までカバーしている充実保障の<span class="tBold">団信革命</span>も検討してみてください。</p>
										<div class="sandaiPosItemImg01"><img src="/kojin/jutaku/danshin/images/sandai_img07.png" alt="男性"></div>
									</div>
								</div>
							</div>
							<div class="v2-colorCardA08 marginLvH2">
	<div class="cardInner tCenter">
		<section id="ancAloneDanshin">
			<h2 class="h2Title01 noBorder tCenter spMb15"><span class="titleInner">単独で加入する団信</span></h2>
			<section class="blockInnerB05">
				<h3 class="danshinRecommendTitle01"><span><img src="/kojin/jutaku/danshin/images/danshin_balloon_02.png" alt="銀行員おすすめ" class="danshinRecommendBalloon01"><span class="danshinTextMarker01">りそなが自信をもって<br class="spContent">おすすめする！<br class="spContent">まさに革命的な団信</span></span></h3>
				<div><img src="/kojin/jutaku/danshin/images/danshin_logo_01.png" alt="団信革命" class="w25p spW67p"></div>
				<p>団信革命は病気やケガで所定の状態に該当すれば、<span class="v2-textMarker01 tBold">仕事に復帰しても<sup>※1</sup>住宅ローン残高が0円です。</span></p>
				<div class="spMt20">
					<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>団信革命について詳しくみる</span></a>
					<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w350 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>団信革命について詳しくみる</span></a>
				</div>
				<div class="newAdjustment spMt20">
					<ul class="listMemo02 fsSmall tLeft">
						<li><span class="listNum">※1</span>急性心筋梗塞の場合、初診日からその日を含めて60日以上労働の制限を必要とする状態が継続したと医師によって診断されたとき等、状態により支払要件は異なります。詳細は<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite#anchor01" data-select-bank="りそな銀行" aria-hidden="false">商品概要</a><a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite#anchor01" data-select-bank="埼玉りそな銀行" aria-hidden="true">商品概要</a>をご確認ください。</li>
					</ul>
				</div>
				<section class="blockInnerB05 marginLvH3">
					<h4 class="fsMaximum spFsMiddle tBold">もっと手ごろな団信にしたい方は</h4>
					<ul class="layoutColumnA2_2 w60p spW100p mlrAuto spMt10">
						<li>
							<a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="りそな銀行" aria-hidden="false">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">3大疾病保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_04.png" alt="３大疾病団信" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
							<a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">3大疾病保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_04.png" alt="３大疾病団信" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
						</li>
						<li>
							<a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="りそな銀行" aria-hidden="false">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">がん保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_05.png" alt="がん保障特約" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
							<a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" class="v3-cardblockA01 danshinCardBlock01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
								<div class="v3-blockInner">
									<dl class="v3-detailListB01">
										<dt class="v3-title spFsMiddle">がん保障特約</dt>
										<dd class="v3-detail"><figure><img src="/kojin/jutaku/danshin/images/danshin_ico_05.png" alt="がん保障特約" class="w54p spW58p"></figure></dd>
									</dl>
								</div>
							</a>
						</li>
					</ul>
				</section>
				<section class="blockInnerB05 marginLvH3">
					<h4 class="fsMaximum spFsMiddle tBold">健康上の理由でご加入に不安がある方は</h4>
					<figure><img src="/kojin/jutaku/danshin/images/danshin_ico_06.png" alt="ワイド団信" class="w12p spW26p"></figure>
					<p>通常よりも加入条件が緩和されている団体信用生命保険。持病・病歴がある方でもご加入できる団信。</p>
					<div class="spMt20">
						<a href="/kojin/jutaku/danshin/wide_danshin/?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ワイド団信について詳しくみる</span></a>
						<a href="/kojin/jutaku/danshin/wide_danshin/?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ワイド団信について詳しくみる</span></a>
					</div>
					<div class="newAdjustment spMt20">
						<ul class="listMemo01 fsSmall spTLeft">
							<li><span class="listMark">※</span>保険会社による所定の審査が必要になります。必ずしもご加入を保証するものではありません。</li>
						</ul>
					</div>
					<div class="spMt20">
						<div class="blockInnerB01">
							<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>ご検討中の方はまずは<br class="spContent">お気軽にご相談ください </span></div>
							<div>
								<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle glyphArrowTriangleRight iconNewWindow w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで来店予約</span></a>
								<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle glyphArrowTriangleRight iconNewWindow w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで来店予約</span></a>
							</div>
						</div>
					</div>
				</section>
			</section>
		</section>
		<section id="ancPairDanshin" class="marginLvH2">
			<h2 class="h2Title01 noBorder tCenter spMb15"><span class="titleInner">ペアで入る団信</span></h2>
			<div class="v4-decorationTextB02 fs18_20 spFs16 tBold"><span>2人とも安心 </span></div>
			<div class="blockInnerB03">
				<p><img src="/kojin/jutaku/danshin/images/pair_img_11.png" alt="ペアローンならペア団信！それぞれの住宅ローン残高どちらも0円" class="w600"></p>
				<div>
					<a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="りそな銀行" aria-hidden="false"><span>ペア団信について詳しくみる</span></a>
					<a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" class="btnT01 btnSMiddle glyphArrowTriangleRight w400 mb00" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ペア団信について詳しくみる</span></a>
				</div>
			</div>
		</section>
	</div>
</div>

							<section id="anchor01" class="jsAccTitle jsAccPcOpen">
								<div role="tablist">
									<h2 id="accSwitchB01" class="jsAccSwitch h2Title01 tCenter spTLeft" role="tab" aria-selected="true" aria-controls="accContentB01"><span class="titleInner">商品概要</span></h2>
									<div id="accContentB01" class="blockInnerB01" role="tabpanel" aria-labelledby="accSwitchB01" aria-expanded="true" aria-hidden="false">
										<table class="tableColumn mb00">
											<tbody>
												<tr>
													<th class="w25p">対象商品</th>
													<td class="w75p">
														<ul class="listCircle01 mb00">
															<li>りそな住宅ローン</li>
															<li>りそな借りかえローン</li>
															<li>りそな住みかえローン</li>
															<li>りそな建てかえローン</li>
															<li>りそな＜すまい・るパッケージ（フラット35）＞</li>
															<li data-select-bank="りそな銀行" aria-hidden="false">りそな女性向け住宅ローン「凛next」</li>
															<li data-select-bank="埼玉りそな銀行" aria-hidden="true">りそな女性向け住宅ローン「凛next」<br>（こちらの商品では、ご融資利率が他商品よりも年0.1％割引となります。）</li>
															<li data-select-bank="りそな銀行" aria-hidden="false">りそなフラットON</li>
														</ul>
														<p>商品毎にお申込みの条件が異なりますので、詳しい内容は各商品の<span data-select-bank="埼玉りそな銀行" aria-hidden="true">チラシ・</span>商品概要説明書をご覧ください。</p>
													</td>
												</tr>
												<tr>
													<th>ご利用いただける方</th>
													<td>
														<ul class="listCircle01 mb00">
															<li>上記商品を新規にお借入れになる方で、お借入時年齢満50歳未満の方</li>
															<li>悪性新生物（がん）に罹患したことのない方</li>
														</ul>
														<ul class="listMemo01 fsSmall mb00">
															<li><span class="listMark">※</span>告知内容によっては、保険会社がご加入をお断りすることがあります。</li>
														</ul>
													</td>
												</tr>
												<tr>
													<th>お借入金額</th>
													<td>
														<p>3億円以内</p>
														<ul class="listMemo01 fsSmall mb00">
															<li><span class="listMark">※</span>りそなリフォームローンは500万円以内となります。</li>
															<li><span class="listMark">※</span>りそな＜すまい・るパッケージ（フラット35）＞は800万円以内となります。</li>
															<li><span class="listMark">※</span>お借入金額が5,000万円超の場合は、所定の診断書をご提出いただきます。</li>
														</ul>
													</td>
												</tr>
												<tr>
													<th>お借入金利</th>
													<td>
														<dl class="v3-packingHorizonA01">
															<dt>上記対象商品の店頭金利</dt>
															<dd>
																<ul>
																	<li>＋年0.25％　（加入時年齢40歳以上）</li>
																	<li>＋年0.20％　（加入時年齢40歳未満）</li>
																</ul>
															</dd>
														</dl>
														<ul class="listMemo01 fsSmall mb00">
															<li><span class="listMark">※</span>「凛next」の場合は、店頭金利から一律＋年0.15％</li>
															<li><span class="listMark">※</span>各種金利プランもあわせてご利用いただけますが、その場合も同金利を上乗せさせていただきます。</li>
														</ul>
													</td>
												</tr>
												<tr>
													<th>中途加入・解約について</th>
													<td>既に当社住宅ローンをご利用の場合、ご返済の途中で団体信用生命保険（3大疾病保障特約付）に切り換えることはできません。また、団体信用生命保険（3大疾病保障特約付）をご利用されている場合に、ご返済の途中で3大疾病保障特約のみを解約することはできませんのでご注意ください。</td>
												</tr>
												<tr>
													<th>付帯される保険の概要</th>
													<td>
														<ul class="listCircle01 mb00">
															<li>銀行が保険契約者となり、住宅ローンをご利用になるお客さまを被保険者とする生命保険＊にご加入いただきます。</li>
															<li class="hasChild">
																<ul class="listMemo01 fsSmall pl00">
																	<li><span class="listMark">＊</span>団体信用生命保険（3大疾病保障特約付）</li>
																</ul>
															</li>
															<li>保険料は保険契約者である銀行が保険会社に払込みます。</li>
															<li>保険会社から支払われた保険金・特約保険金は銀行が受取り、住宅ローン残高の返済に充当します。</li>
														</ul>
														<ul class="listMemo01 fsSmall mb00">
															<li><span class="listMark">※</span>悪性新生物（がん）に罹患したことのある方は加入いただけません。</li>
														</ul>
														<h3 class="h3Title01"><span class="titleInner">＜死亡・高度障害の場合＞</span></h3>
														<ul class="listCircle01 mb00">
															<li>
																<p>お借入日以降に死亡された場合、またはお借入日以降の傷害または疾病が原因で所定の高度障害状態に該当された場合に保険金が支払われます。</p>
																<p class="tBold">【保険金が支払われない場合の例】</p>
																<ol class="listNum01 mb00">
																	<li><span class="listNum">1.</span>お借入日から1年以内の自殺</li>
																	<li data-select-bank="りそな銀行" aria-hidden="false"><span class="listNum">2.</span>お借入日より前の傷害または疾病が原因で高度障害状態に該当した場合</li>
																	<li data-select-bank="埼玉りそな銀行" aria-hidden="true"><span class="listNum">2.</span>お借入日以前の傷害または疾病が原因で高度障害状態に該当した場合</li>
																</ol>
															</li>
														</ul>
														<h3 class="h3Title01"><span class="titleInner">＜悪性新生物（がん）の場合＞</span></h3>
														<ul class="listCircle01 mb00">
															<li>
																<p>お借入日から起算して90日を経過した日の翌日以降に悪性新生物（がん）に罹患したと医師に診断確定された場合に特約保険金が支払われます。ただし、上皮内がん、および皮膚の悪性黒色腫以外の皮膚がんは除きます。</p>
																<p class="tBold">【特約保険金が支払われない場合の例】</p>
																<ol class="listNum01 mb00">
																	<li data-select-bank="りそな銀行" aria-hidden="false"><span class="listNum">1.</span>お借入日より前に悪性新生物（がん）に罹患したと診断確定されていた場合</li>
																	<li data-select-bank="埼玉りそな銀行" aria-hidden="true"><span class="listNum">1.</span>お借入日以前に悪性新生物（がん）に罹患したと診断確定されていた場合</li>
																	<li><span class="listNum">2.</span>お借入日から90日以内に悪性新生物（がん）に罹患したと診断確定された場合</li>
																	<li><span class="listNum">3.</span>お借入日から90日を経過した日の翌日以降に悪性新生物（がん）に罹患したと医師に診断確定されたが、その悪性新生物（がん）が、お借入日から90日以内に診断確定された悪性新生物（がん）の再発・転移等と認められた場合</li>
																</ol>
															</li>
														</ul>
														<h3 class="h3Title01"><span class="titleInner">＜急性心筋梗塞・脳卒中の場合＞</span></h3>
														<ul class="listCircle01 mb00">
															<li>
																<p>お借入日以降に急性心筋梗塞または脳卒中を発病し、その疾病により以下の【1】、【2】いずれかの状態に 該当された場合に特約保険金が支払われます。</p>
																<ol class="listNum02 mb00">
																	<li><span class="listNum">【1】.</span>医師の診断を受けた日から60日以上所定の状態が継続したと診断された場合</li>
																	<li><span class="listNum">【2】.</span>治療を直接の目的として所定の手術を受けたとき</li>
																</ol>
																<p class="tBold">【特約保険金が支払われない場合の例】</p>
																<ol class="listNum01 mb00">
																	<li><span class="listNum">1.</span>急性心筋梗塞で医師の診断を受けた日から59日以内に労働制限が必要なくなった場合</li>
																	<li><span class="listNum">2.</span>脳卒中で医師の診断を受けた日から59日以内に後遺症がなくなった場合</li>
																	<li><span class="listNum">3.</span>急性心筋梗塞または脳卒中で医師の診断を受けた日から所定の状態が継続していたが、60日以上継続する前にこの特約の被保険者でなくなってしまった場合</li>
																</ol>
															</li>
															<li>保障期間 ：住宅ローンのお借入期間</li>
															<li>幹事保険会社：第一生命保険株式会社</li>
														</ul>
														<table class="tableColumn">
															<tbody>
																<tr class="vMiddle">
																	<th class="w25p">ご留意事項</th>
																	<td class="w75p">保険金の支払いには条件があります。生命保険のご加入にあたっては、「重要事項に関するご説明（契約概要・注意喚起情報）」を必ずお読みいただき、詳細をご確認ください。</td>
																</tr>
															</tbody>
														</table>
														<div class="textBlock01">
															<p>■3大疾病保障特約付団体信用生命保険に関するお問合せ<br>　第一生命保険株式会社　団体保障事業部</p>
														</div>
														<ul class="listCircle01 mb00">
															<li>ご加入・告知に関するお問合せは　・・・0120-005-328</li>
															<li>お支払・お支払事由に関するお問合せは　・・・0120-709-471</li>
														</ul><!-- /.listCircle01 -->
														<ul class="listMemo01">
															<li><span class="listMark">※</span>受付専用フリーダイヤル<br>受付時間9：00 ～ 17：00（土・日・祝日・年末年始除く）</li>
														</ul>
														<p class="tRight">（登）C23E6139（2023.8.29）</p>
													</td>
												</tr>
											</tbody>
										</table>
									</div>
								</div>
							</section>
							<section>
								<h2 class="h2Title01 tCenter spTLeft"><span class="titleInner">よくあるご質問</span></h2>
								<div class="blockInnerB04" data-select-bank="りそな銀行" aria-hidden="false">
									<div id="faqLoadList1">
										<script>
											$(function() {
												LoadFaq.init({
													'id'      : 'faqLoadList1',
													'question': ['faq_jutaku_0014','faq_jutaku_0015','faq_jutaku_0028','faq_jutaku_0029','faq_jutaku_0030','faq_jutaku_0031','faq_jutaku_0032','faq_jutaku_0033','faq_jutaku_0034','faq_jutaku_0035'],
													'answer'  : true,//true or false
													'design'  : '1',
													'domain' : 'rb'
												});
											});
										</script>
									</div>
									<div><a href="/kojin/jutaku/faq/" class="btnT01 btnSMiddle glyphArrowTriangleRight btnWFull w380 mlrAuto overT01"><span>住宅ローンのよくあるご質問一覧</span></a></div>
								</div>
								<div class="blockInnerB04" data-select-bank="埼玉りそな銀行" aria-hidden="true">
									<div id="faqLoadList2">
										<script>
											$(function() {
												LoadFaq.init({
													'id'      : 'faqLoadList2',
													'question': ['faq_jutaku_0014','faq_jutaku_0015','faq_jutaku_0028','faq_jutaku_0029','faq_jutaku_0030','faq_jutaku_0031','faq_jutaku_0032','faq_jutaku_0033','faq_jutaku_0034','faq_jutaku_0035'],
													'answer'  : true,//true or false
													'design'  : '1',
													'domain' : 'sr',
													'_blank' : true
												});
											});
										</script>
									</div>
									<div><a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow btnWFull w410 mlrAuto overT01" target="_blank"><span>住宅ローンのよくあるご質問一覧</span></a></div>
								</div>
							</section>
							<div class="v2-cardBlockA03 v2-bgGreen01 blockInnerB05 marginLvH2 newAdjustment">
	<section class="blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb30"><span class="titleInner">関連情報</span></h2>
		<div class="v4-carousel01 carouselType03 v4-jsCarousel01" data-select-bank="りそな銀行" aria-hidden="false">
	<ul class="carouselList">
		<li><a href="/kojin/landing/sx/?bank=rb_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_12.png" alt="住宅ローンや資産形成を通じてSDGsに貢献しよう！"></a></li>
		<li><a href="/kojin/jutaku/landing/keiyakugo.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_11.png" alt="必見！銀行員が教える住宅ローン契約後にやるべきこと"></a></li>
		<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.gif" alt="スマホでカンタン新生活必要なものリスト！"></a></li>
		<li><a href="/kojin/column/jutaku/enquete_0001.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_06.png" alt="年収別でみる　みんなの住宅購入事情　結果詳細を見る"></a></li>
		<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_02.gif" alt="スマホでカンタンマイホームToDoリスト"></a></li>
		<li>
			<a href="/kojin/jutaku/zeh/?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_08.png" alt="環境等配慮型住宅にすると金利が下がるって知ってました？"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/sumikae/?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_13.gif" alt="住みかえ検討中の方！りそなの住みかえプランがおすすめ！"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/reason/resona_club.html?bank=rb_unite"><img src="/kojin/jutaku/images/jutaku_bnr_10.png" alt="新規住宅ローン利用者限定　諸条件を満たすと10,000ポイントもらえる！　詳しくはこちら"></a>
		</li>
	</ul>
</div>
<div class="v4-carousel01 carouselType03 v4-jsCarousel01" data-select-bank="埼玉りそな銀行" aria-hidden="true">
	<ul class="carouselList">
		<li><a href="/kojin/mycar/?bank=sr_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.jpg" alt="りそなマイカーローン"></a></li>
		<li><a href="/kojin/landing/sx/?bank=sr_unite" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_12.png" alt="住宅ローンや資産形成を通じてSDGsに貢献しよう！"></a></li>
		<li><a href="https://www.saitamaresona.co.jp/kojin/jutaku/landing/keiyakugo.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_11.png" alt="必見！銀行員が教える住宅ローン契約後にやるべきこと"></a></li>
		<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_01.gif" alt="スマホでカンタン新生活必要なものリスト！"></a></li>
		<li><a href="/kojin/column/jutaku/enquete_0001.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_06.png" alt="年収別でみる　みんなの住宅購入事情　結果詳細を見る"></a></li>
		<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank"><img src="/kojin/jutaku/images/jutaku_bnr_02.gif" alt="スマホでカンタンマイホームToDoリスト"></a></li>
		<li>
			<a href="/kojin/jutaku/zeh/?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_08.png" alt="環境等配慮型住宅にすると金利が下がるって知ってました？"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/sumikae/?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_13.gif" alt="住みかえ検討中の方！りそなの住みかえプランがおすすめ！"></a>
		</li>
		<li>
			<a href="/kojin/jutaku/reason/resona_club.html?bank=sr_unite"><img src="/kojin/jutaku/images/jutaku_bnr_10.png" alt="新規住宅ローン利用者限定　諸条件を満たすと10,000ポイントもらえる！　詳しくはこちら"></a>
		</li>
	</ul>
</div>

	</section>
	<section class="blockInnerB05">
		<h2 class="h2Title01 noBorder tCenter spMb30"><span class="titleInner">住宅ローンメニュー</span></h2>
		<ul class="w700 spW100p mlrAuto">
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA01" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA01"><span class="titleInner fs18_24 spFs18">住宅ローンの商品</span></h3>
						<div id="jutakuRelationAccContentA01" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA01" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a>
									<a href="/kojin/jutaku/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン（新規）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/karikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a>
									<a href="/kojin/jutaku/karikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン（お借換え）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a>
									<a href="/kojin/jutaku.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン商品の一覧</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA02" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA02"><span class="titleInner fs18_24 spFs18">特典・プラン</span></h3>
						<div id="jutakuRelationAccContentA02" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA02" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/lin/" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな女性向け住宅ローン『凛next』</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/lin/" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな女性向け住宅ローン『凛next』</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/zeh/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a>
									<a href="/kojin/jutaku/zeh/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>SX金利プラン</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/sumikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a>
									<a href="/kojin/jutaku/sumikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>住みかえプラン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA03" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA03"><span class="titleInner fs18_24 spFs18">団信・火災</span></h3>
						<div id="jutakuRelationAccContentA03" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA03" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/danshin/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a>
									<a href="/kojin/jutaku/danshin/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団体信用生命保険（団信）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/kakumei.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a>
									<a href="/kojin/jutaku/danshin/kakumei.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団信革命（特定状態保障特約付団信）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/sandai.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a>
									<a href="/kojin/jutaku/danshin/sandai.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>3大疾病保障特約</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/gan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>がん保障特約</span></a>
									<a href="/kojin/jutaku/danshin/gan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>がん保障特約</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/pair.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>ペア団信</span></a>
									<a href="/kojin/jutaku/danshin/pair.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>ペア団信</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/danshin/danshin.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a>
									<a href="/kojin/jutaku/danshin/danshin.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>団体信用生命保険の比較</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a>
									<a href="/kojin/jutaku/jutaku_loan/hoken.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン専用火災保険</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/shizen/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a>
									<a href="/kojin/jutaku/shizen/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>自然災害サポートオプション</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA04" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA04"><span class="titleInner fs18_24 spFs18">金利・シミュレーション</span></h3>
						<div id="jutakuRelationAccContentA04" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA04" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/loan_viewer.html" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>金利一覧</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/loan_viewer.html" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>金利一覧</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/simulation/shinki/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a>
									<a href="/kojin/jutaku/simulation/shinki/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新規お借入れシミュレーション</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/simulation/karikae/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a>
									<a href="/kojin/jutaku/simulation/karikae/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>お借換えシミュレーション</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA05" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA05"><span class="titleInner fs18_24 spFs18">お手続き・諸費用など</span></h3>
						<div id="jutakuRelationAccContentA05" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA05" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/procedure1.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a>
									<a href="/kojin/jutaku/procedure1.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お手続きの流れ（新規）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/procedure2.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a>
									<a href="/kojin/jutaku/procedure2.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お手続きの流れ（お借換え）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/fee/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a>
									<a href="/kojin/jutaku/fee/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>住宅ローンの手数料・諸費用</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/documents.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a>
									<a href="/kojin/jutaku/documents.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>お申込みに必要な書類</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/pair_loan.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>ペアローン</span></a>
									<a href="/kojin/jutaku/pair_loan.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>ペアローン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA06" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA06"><span class="titleInner fs18_24 spFs18">お申込み・審査</span></h3>
						<div id="jutakuRelationAccContentA06" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA06" aria-expanded="false" aria-hidden="true">
							<ul>
								<li>
									<a href="/kojin/jutaku/jizen_moushikomi/" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで事前審査申込</span></a>
								</li>
								<li>
									<a href="https://www.jl.resona-gr.co.jp/fmweb/login" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>Webで正式審査申込（りそな住宅ローンマイページ）</span></a>
								</li>
								<li>
									<a href="/kojin/jutaku/econtract.html?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a>
									<a href="/kojin/jutaku/econtract.html?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" class="glyphArrowTriangleRight"><span>りそな住宅ローン電子契約サービス</span></a>
								</li>
								<li>
									<a href="https://navi.resona-gr.co.jp/jutaku/login?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a>
									<a href="https://navi.resona-gr.co.jp/jutaku/login?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>りそな住宅ローンナビログイン</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA07" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA07"><span class="titleInner fs18_24 spFs18">はじめての方</span></h3>
						<div id="jutakuRelationAccContentA07" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA07" aria-expanded="false" aria-hidden="true">
							<ul>
								<li><a href="/kojin/column/jutaku/myhome_first.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>はじめてのマイホーム&amp;住宅ローン</span></a></li>
								<li><a href="/kojin/column/jutaku/todo_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>マイホームToDoリスト</span></a></li>
								<li><a href="/kojin/column/jutaku/newlife_list.html" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>新生活必要なものリスト</span></a></li>
							</ul>
						</div>
					</div>
				</section>
			</li>
			<li>
				<section class="jsAccTitle">
					<div role="tablist">
						<h3 id="jutakuRelationAccSwitchA08" class="jsAccSwitch h3Title01 mt15" role="tab" aria-selected="false" aria-controls="jutakuRelationAccContentA08"><span class="titleInner fs18_24 spFs18">用語集・Q&amp;A</span></h3>
						<div id="jutakuRelationAccContentA08" role="tabpanel" aria-labelledby="jutakuRelationAccSwitchA08" aria-expanded="false" aria-hidden="true">
							<ul>
								<li><a href="/kojin/jutaku/yougo/" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>住宅ローンの用語集</span></a></li>
								<li>
									<a href="/kojin/jutaku/faq/" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a>
									<a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="glyphArrowTriangleRight iconNewWindow"><span>よくあるご質問</span></a>
								</li>
							</ul>
						</div>
					</div>
				</section>
			</li>
		</ul>
	</section>
	<section class="karte-jutakuRanking"></section>
	<div class="v4-detailBlockA01">
		<ul class="layoutColumnA3_1 v2-wide01 tCenter">
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>毎月の返済はいくら？</span></p>
				<div class="jutakuRelationBtn01">
					<a href="/kojin/jutaku/simulation/?bank=rb_unite" data-select-bank="りそな銀行" aria-hidden="false" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>住宅ローンシミュレーション</span></a>
					<a href="/kojin/jutaku/simulation/?bank=sr_unite" data-select-bank="埼玉りそな銀行" aria-hidden="true" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>住宅ローンシミュレーション</span></a>
				</div>
			</li>
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>土・日・祝日や17時以降も営業</span></p>
				<div>
					<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>Webで来店予約</span></a>
					<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnSMiddle iconNewWindow btnWFull"><span>Webで来店予約</span></a>
				</div>
			</li>
			<li class="blockInnerB01">
				<p class="v4-decorationTextB02 tBold"><span>審査結果もWebで確認！</span></p>
				<div>
					<a href="/kojin/jutaku/jizen_moushikomi/" data-select-bank="りそな銀行" aria-hidden="false" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow btnWFull"><span>Webで事前審査</span></a>
					<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" data-select-bank="埼玉りそな銀行" aria-hidden="true" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnSMiddle iconNewWindow btnWFull"><span>Webで事前審査</span></a>
				</div>
			</li>
		</ul>
	</div>
	<section class="marginLvH2">
		<div class="v3-balloonBlockB01">
			<h2 class="v3-title">その他ローン</h2>
			<div class="v3-detail blockInnerB03">
				<ul class="v3-layoutColumnA4_2 mycarFlexAuto">
					<li>
						<a href="/kojin/jutaku/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>住宅ローン</span></a>
						<a href="/kojin/jutaku/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>住宅ローン</span></a>
					</li>
					<li>
						<a href="/kojin/cardloan/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>カードローン</span></a>
						<a href="/kojin/cardloan/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>カードローン</span></a>
					</li>
					<li>
						<a href="/kojin/freeloan/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>フリーローン</span></a>
						<a href="/kojin/freeloan/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>フリーローン</span></a>
					</li>
					<li>
						<a href="/kojin/edu/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>教育ローン</span></a>
						<a href="/kojin/edu/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>教育ローン</span></a>
					</li>
					<li>
						<a href="/kojin/reform/?bank=rb_unite" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>リフォームローン</span></a>
						<a href="/kojin/reform/?bank=sr_unite" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>リフォームローン</span></a>
					</li>
					<li>
						<a href="/kojin/akiya/" class="glyphArrowTriangleRight" data-select-bank="りそな銀行" aria-hidden="false"><span>空き家専用ローン</span></a>
						<a href="https://www.saitamaresona.co.jp/kojin/akiya/" class="glyphArrowTriangleRight" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>空き家専用ローン</span></a>
					</li>
				</ul>
				<div class="blockInnerB01 tCenter w100p">
					<p class="v4-decorationTextB02 tBold tOrange01"><span>15秒で簡単に診断！</span></p>
					<div>
						<a href="/kojin/column/loan/loan_simulation/?bank=rb_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow overT01 w400" data-select-bank="りそな銀行" aria-hidden="false"><span>ローン適正診断</span></a>
						<a href="/kojin/loan/column/loan_simulation/?bank=sr_unite" target="_blank" class="btnT01 btnSMiddle glyphArrowTriangleRight iconNewWindow overT01 w400" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>ローン適正診断</span></a>
					</div>
				</div>
			</div>
		</div>
	</section>
</div>

						</div>
					</div>
				</section>
			</main>
		</div><!-- //#mainArea -->
	</div><!-- //#wrap -->
	<footer role="contentinfo">
				<a href="#top" id="pageTop"><img src="/common/images/go_top.svg" alt="ページトップへ戻る" class="svgTop"></a>
		<div class="snsArea">
			<!-- PC ONLY CONTENTS START-->
			<dl>
				<dt><a href="https://www.resona-gr.co.jp/holdings/other/sns/" target="_blank">りそなソーシャルメディアネットワーク</a></dt>
				<dd>
					<ul class="footerSnsList">
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><figure><img src="/common/images/footer_sns_tw02.gif" alt="X"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><figure><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><figure><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><figure><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></figure></a></li>
						<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><figure><img src="/common/images/footer_sns_line02.gif" alt="Line"></figure></a></li>
					</ul>
				</dd>
			</dl>
			<!-- PC ONLY CONTENTS END-->
			<!-- SP ONLY CONTENTS START-->
			<ul class="spFooterSnsList">
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_069.html" target="_blank"><img src="/common/images/footer_sns_tw02.gif" alt="X"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_108.html" target="_blank"><img src="/common/images/footer_sns_rsn02.gif" alt="りそにゃ公式Twitter"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_131.html" target="_blank"><img src="/common/images/footer_sns_Instagram.svg" alt="Instagram" width="35"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/outlink/link_093.html" target="_blank"><img src="/common/images/footer_sns_yt02.gif" alt="YouTube"></a></li>
				<li><a href="https://www.resona-gr.co.jp/holdings/other/sns/index.html#line" target="_blank"><img src="/common/images/footer_sns_line02.gif" alt="Line"></a></li>
			</ul>
			<!-- SP ONLY CONTENTS END-->
		</div><!-- //.snsArea -->

		<div data-select-bank="りそな銀行" aria-hidden="false">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">
			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=rb_unite&loc=7">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=rb_unite&loc=7"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=rb_unite&loc=7">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/?loc=7" target="_blank">つみたてボックス</a></li>
						<li><a href="/kojin/toshin/?loc=7">投資信託</a></li>
						<li><a href="/kojin/fundwrap/?loc=7" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=rb_unite&loc=7">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=rb_unite&loc=7">定期預金</a></li>
						<li><a href="/kojin/kinsen/?loc=7">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=rb_unite&loc=7">外貨預金</a></li>
						<li><a href="/kojin/shikin_unyo/?loc=7">資金運用プラン</a></li>
						<li><a href="/kojin/taishoku_unyo/?loc=7">退職金コース</a></li>
						<li><a href="/kojin/kokusai/?loc=7">個人向け国債・国債</a></li>
						<li><a href="/kojin/gaisai/?loc=7">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=rb_unite&loc=7">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=rb_unite&loc=7">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=rb_unite&loc=7">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=rb_unite&loc=7">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=rb_unite&loc=7">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=rb_unite&loc=7">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=rb_unite&loc=7">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=rb_unite&loc=7">マイカーローン</a></li>
						<li><a href="/kojin/edu/?loc=7">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=rb_unite&loc=7">リフォームローン</a></li>
						<li><a href="/kojin/apaman/?loc=7">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=rb_unite&loc=7">リバースモーゲージローン</a></li>
						<li><a href="/kojin/akiya/?loc=7">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/sonaeru.html?loc=7">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=rb_unite&loc=7">保険</a></li>
						<li><a href="/kojin/souzokusyoukei.html?loc=7">相続・承継</a></li>
						<li><a href="/kojin/zaisankanri.html?loc=7">財産管理業務</a></li>
						<li><a href="/kojin/private/?loc=7">プライベートバンキング業務</a></li>
						<li><a href="/nenkin/ideco/?loc=7">iDeCo(個人型確定拠出年金)</a></li>
						<li><a href="/nenkin/401k/?loc=7">企業型確定拠出年金</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/tsukau.html?loc=7">便利にお得に使いたい</a></h2>
					<ul>
						<li><a href="/direct/?loc=7">マイゲート</a></li>
						<li><a href="/kojin/app/?bank=rb_unite&loc=7"><span>スマートフォンアプリ</span></a></li>
						<li><a href="/club/?bank=rb_unite&loc=7">りそなクラブ</a></li>
						<li><a href="/kojin/exchange/gaika/?loc=7">外貨両替</a></li>
						<li><a href="/kojin/kaigai/soukin/?loc=7">海外送金</a></li>
						<li><a href="/kojin/visa_debit/?bank=rb_unite&loc=7">Visaデビットカード</a></li>
						<li><a href="/kojin/credit/?bank=rb_unite&loc=7">クレジットカード</a></li>
						<li><a href="/kojin/kessai_nozei/payeasy_shunou/?bank=rb_unite&loc=7">ペイジー料金払込サービス</a></li>
						<li><a href="/kojin/koueikyougi/?loc=7">即時口座決済</a></li>
						<li><a href="/kojin/yoyaku/?loc=7">来店予約サービス</a></li>
						<li><a href="/kojin/column/index2.html?loc=7">お金に関わるりそなのコラム</a></li>
						<li><a href="https://community.resonabank.co.jp/?loc=7" target="_blank">お金のあれこれ</a></li>
					</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="/"><span>個人のお客さま</span></a></li>
				<li><a href="/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="/nenkin/"><span>企業年金のお客さま</span></a></li>
				<li><a href="/about/"><span>りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.resonabank.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="1">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com" target="_blank">採用情報</a></li>
					<li><a href="/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="/kojin/contact/#10">指定紛争解決機関</a></li>
					<li><a href="/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0010</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>近畿財務局長(登金)第3号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会 一般社団法人金融先物取引業協会 一般社団法人日本クレジット協会 一般社団法人日本投資顧問業協会</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
		<div data-select-bank="埼玉りそな銀行" aria-hidden="true">
					<!-- PC ONLY CONTENTS START-->
		<div class="footerNav">

			<nav>
				<div>
					<h2><a href="/kojin/hiraku/?bank=sr_unite">口座を開きたい</a></h2>
					<ul>
						<li><a href="/kojin/hiraku/?bank=sr_unite"><span>口座開設</span></a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/huyasu.html?bank=sr_unite">お金を増やしたい</a></h2>
					<ul>
						<li><a href="/kojin/tsumitatebox/" target="_blank">つみたてボックス</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/toshin/">投資信託</a></li>
						<li><a href="https://www.resonabank.co.jp/kojin/fundwrap/" target="_blank">ファンドラップ</a></li>
						<li><a href="/kojin/nisa/?bank=sr_unite">NISA(少額投資非課税制度)</a></li>
						<li><a href="/kojin/teiki/?bank=sr_unite">定期預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kinsen/">金銭信託</a></li>
						<li><a href="/kojin/gaika/?bank=sr_unite">外貨預金</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/shikin_unyo/">資金運用プラン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/taishoku_unyo/">退職金コース</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kokusai/">個人向け国債・国債・地方債</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/gaisai/">外債(金融商品仲介)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="/kojin/kariru.html?bank=sr_unite">お金を借りたい</a></h2>
					<ul>
						<li><a href="/kojin/jutaku/?bank=sr_unite">住宅ローン(新規)</a></li>
						<li><a href="/kojin/jutaku/karikae/?bank=sr_unite">住宅ローン(お借換え)</a></li>
						<li><a href="/kojin/jutaku/simulation/?bank=sr_unite">住宅ローンシミュレーション</a></li>
						<li><a href="/kojin/cardloan/?bank=sr_unite">カードローン</a></li>
						<li><a href="/kojin/freeloan/?bank=sr_unite">フリーローン<br>（無担保型）</a></li>
						<li><a href="/kojin/freeloan/secured/?bank=sr_unite">不動産担保ローン/フリーローン（有担保型）</a></li>
						<li><a href="/kojin/mycar/?bank=sr_unite">マイカーローン</a></li>
						<li><a href="/kojin/edu/?bank=sr_unite">教育ローン</a></li>
						<li><a href="/kojin/reform/?bank=sr_unite">リフォームローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/apaman/">アパート・マンションローン</a></li>
						<li><a href="/kojin/reverse-mortgage/?bank=sr_unite">リバースモーゲージローン</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/akiya/">空き家専用ローン</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/sonaeru.html">将来や万一に備えたい</a></h2>
					<ul>
						<li><a href="/kojin/hoken/?bank=sr_unite">保険</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/souzokusyoukei.html">相続・承継</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/zaisankanri.html">財産管理業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kyoiku_shintaku/">きょういく信託</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/kanri_unyo/private/">プライベートバンキング業務</a></li>
						<li><a href="https://www.saitamaresona.co.jp/kojin/ideco/detail.html">iDeCo(個人型確定拠出年金)</a></li>
					</ul>
				</div>
				<div>
					<h2><a href="https://www.saitamaresona.co.jp/kojin/tsukau.html">便利にお得に使いたい</a></h2>
						<ul>
							<li><a href="https://www.saitamaresona.co.jp/direct/">マイゲート</a></li>
							<li><a href="/kojin/app/?bank=sr_unite"><span>スマートフォンアプリ</span></a></li>
							<li><a href="/club/?bank=sr_unite">りそなクラブ</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/exchange/gaika/">外貨両替</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kaigai/soukin/">海外送金</a></li>
							<li><a href="/kojin/visa_debit/?bank=sr_unite">Visaデビットカード</a></li>
							<li><a href="/kojin/credit/?bank=sr_unite">クレジットカード</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/kessai_nozei/">ペイジー料金払込サービス</a></li>
							<li><a href="https://www.saitamaresona.co.jp/kojin/koueikyougi/">即時口座決済</a></li>
						</ul>
				</div>
			</nav>
		</div><!-- //.footerNav -->
		<!-- PC ONLY CONTENTS END-->
		<nav class="footerLargeCategoryNav saitamaFooterLargeCategoryNav" role="navigation">
			<ul>
				<li><a href="https://www.saitamaresona.co.jp/"><span>個人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/hojin/"><span>法人のお客さま</span></a></li>
				<li><a href="https://www.saitamaresona.co.jp/about/"><span>埼玉りそな銀行について</span></a></li>
			</ul>
		</nav><!-- //.footerLargeCategoryNav -->
		<!-- SP ONLY CONTENTS START-->
		<form class="spFooterSearch spSearchBlock01" role="search" action="https://search.saitamaresona.co.jp/askme.asp" method="get" target="_blank">
			<div>
				<div>
					<input type="hidden" name="origin" value="5">
					<input type="hidden" name="aj_charset" value="utf-8">
					<input type="search" name="ask" placeholder="サイト内検索">
				</div>
				<div><button type="submit">検&nbsp;索</button></div>
			</div>
		</form>
		<!-- SP ONLY CONTENTS END-->

					<div class="footerInfo footerRenewal2024Wrap">
			<nav class="infoNav">
				<ul>
					<li><a href="https://www.saitamaresona.co.jp/util/sitemap.html">サイトマップ</a></li>
					<li><a href="https://www.resona-saiyo.com/" target="_blank">採用情報</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html">本Webサイトのご利用にあたって</a></li>
					<li><a href="https://www.resona-gr.co.jp/holdings/other/accessibility/" target="_blank">Webアクセシビリティポリシー</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kanyu.html">勧誘方針・預金誤認防止について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kojin_jouhou.html">個人情報の取扱いについて</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/kameiten_jouhou.html">加盟店情報の共同利用について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hojin_jouhou.html">法人等のお客さまの情報について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/fiduciary.html">お客さま本位の業務運営</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/souhan.html">利益相反管理方針の概要</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/toshika_seido.html">特定投資家制度・期限日</a></li>
					<li><a href="https://www.resona-gr.co.jp/yokinhoken.html" target="_blank">預金保険制度</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/hoken_bosyu.html">保険募集指針</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dendai.html">電子決済等代行業者との連携について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/sitepolicy.html#02">リンクをご希望の方</a></li>
					<li><a href="https://www.saitamaresona.co.jp/kojin/contact/#13">指定紛争解決機関</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/loan_shisei.html">ローン取引に関する取組姿勢について</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/gaitame_houshin.html">外国為替取引に関する取組姿勢について</a></li>
					<li><a href="https://www.resona-gr.co.jp/resonagr/aml/transaction_confirmation.html" target="_blank">お取引時確認に関するお願い</a></li>
					<li><a href="https://www.saitamaresona.co.jp/util/dairigyou.html">銀行代理業者に関する事項</a></li>
				</ul>
			</nav>
			<div class="resonaInfoWrap">
				<dl class="resonaInfo">
					<dt>株式会社埼玉りそな銀行</dt>
					<dd>
						<dl>
							<dt>金融機関コード&nbsp;:</dt>
							<dd>0017</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>登録金融機関&nbsp;:</dt>
							<dd>関東財務局長(登金)第593号</dd>
						</dl>
						<dl>
							<dt>加入協会&nbsp;:</dt>
							<dd>日本証券業協会&nbsp;&nbsp;&nbsp;一般社団法人金融先物取引業協会&nbsp;&nbsp;&nbsp;一般社団法人日本クレジット協会</dd>
						</dl>
					</dd>
					<dd>
						<dl>
							<dt>信託契約代理業&nbsp;:</dt>
							<dd>登録番号　関東財務局長（代信）第22号</dd>
						</dl>
						<dl>
							<dt>所属信託会社の商号&nbsp;:</dt>
							<dd>株式会社りそな銀行</dd>
						</dl>
					</dd>
				</dl>
				<div class="pcSite"><a href="javascript:;" class="btnT01 btnWFull btnSMiddle glyphArrowLineRight"><span>PCサイトへ</span></a></div>
				<div class="resonaGroupInfo clear footerRenewal2024">
					<div class="footerRenewal2024Inner">
						<dl class="resonaGroup">
							<dt><a href="https://www.resona-gr.co.jp/" target="_blank"><img src="/common/images/logo_hd.svg" alt="りそなホールディングス"></a></dt>
							<dd>
								<ul>
									<li class="footerLogoResona"><a href="/"><img src="/common/images/logo_resona.svg" alt="りそな銀行"></a></li>
									<li class="footerLogoSaitama"><a href="https://www.saitamaresona.co.jp/" target="_blank"><img src="/common/images/logo_saitama.svg" alt="埼玉りそな銀行"></a></li>
									<li class="footerLogoKansaimirai"><a href="https://www.kansaimiraibank.co.jp/" target="_blank"><img src="/common/images/logo_kansaimirai2.svg" alt="関西みらい銀行"></a></li>
									<li class="footerLogoMinato"><a href="https://www.minatobk.co.jp/" target="_blank"><img src="/common/images/logo_minato.svg" alt="みなと銀行"></a></li>
								</ul>
							</dd>
						</dl>
						<figure><img src="/common/images/bg_purpose.svg" width="291" alt="金融＋で、未来をプラスに。RESONA GROUP"></figure>
					</div>
				</div>
			</div>
			<div class="copyRight"><small>Copyright (c) Saitama Resona Bank, Limited All Rights Reserved.</small></div>
		</div><!-- //.footerInfo -->

		</div>
	</footer>
	<!-- FOOTER FIXED CONTENT START-->
	<script type="text/javascript" charset="UTF-8" src="https://mobi.lineomni.jp/web/mobi-agent-client-frame-loader.min.js?domainId=resonagr"></script>
<script type="text/javascript" charset="UTF-8" src="/kojin/contact/chat/decsupport/js/ds-jutaku_two.js"></script>
	<!-- HUMANCHAT -->
<div>
	<div class="fixedBottom01 fixedBottomChat01 jsFixedBottom01 jsFixedBottomChat">
		<div class="fixedBottomInner">
			<div class="fixedBottomContent">
				<div class="fixedBottomNav">
					<ul class="fixedBottomList01">
						<li>
							<a data-select-bank="りそな銀行" aria-hidden="false" href="/kojin/jutaku/modal/modal_menu.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
							<a data-select-bank="埼玉りそな銀行" aria-hidden="true" href="/kojin/jutaku/modal/modal_menu_saitama.html" data-modal="modalBody" class="glyphMenu02 v3-jsModal01 spTCenter"><span class="fsSmall spFs10">住宅ローン<br class="spContent">メニュー</span></a>
						</li>
						<li><a href="javascript:void(0)" class="fixedBottomIconFaq01 spTCenter jsChatMenuTrg"><span class="fsSmall spFs10">お困りの方</span></a></li>
					</ul>
					<div class="fixedBottomChatMenu01 jsFixedBottomChatMenu">
						<ul class="chatMenuBlock01 tCenter">
							<li class="chatAbled rb_unite" data-select-bank="りそな銀行" aria-hidden="false">
								<a href="/kojin/contact/chat/ds/jutaku.html" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_03.svg" alt="有人チャット"></div>
									<div class="tBold fs18_13 spFs10"><span>有人チャット</span></div>
								</a>
							</li>
							<li>
								<a href="https://ct.resona-chatbot.jp/?site_id=7" target="_blank">
									<div class="mlrAuto chatMenuIcon01"><img src="/common/type2018/images/icon_chat_02.svg" alt="AIチャット"></div>
									<div class="tBold fs18_13 spFs10"><span>AIチャット</span></div>
								</a>
							</li>
							<li class="chatFaqLink">
								<a href="/kojin/jutaku/faq/" data-select-bank="りそな銀行" aria-hidden="false">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
								<a href="https://www.saitamaresona.co.jp/kojin/jutaku/faq/" data-select-bank="埼玉りそな銀行" aria-hidden="true">
									<div class="iconFaq tBold fs18_13 spFs10"><span class="tLeft">よくあるご質問を見る</span></div>
								</a>
							</li>
						</ul>
					</div>
				</div>
				<div class="fixedBottomBtn">
					<ul class="fixedBottomList01">
						<li>
							<a href="/kojin/tenpo_atm/?type=%E3%82%8A%E3%81%9D%E3%81%AA%E9%8A%80%E8%A1%8C&loanNew=1&loanRefinancing=1#A03" onclick="wa_rb_kouza_shop_button();" oncontextmenu="wa_rb_kouza_shop_button();" target="_blank" class="btnT02 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">来店予約</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/karikaekinri/plaza/plaza.html" onclick="this.href=_wa.link(this.href);wa_sr_sevendays_soudan();" oncontextmenu="wa_sr_sevendays_soudan();" target="_blank" class="btnT02 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">来店予約</span></a>
						</li>
						<li>
							<a href="/kojin/jutaku/jizen_moushikomi/" onclick="wa_rb_jutaku_button();" oncontextmenu="wa_rb_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="りそな銀行" aria-hidden="false"><span>Webで<br class="spContent">事前審査</span></a>
							<a href="https://www.saitamaresona.co.jp/kojin/jutaku/jizen_moushikomi/" onclick="this.href=_wa.link(this.href);wa_sr_jutaku_button();" oncontextmenu="wa_sr_jutaku_button();" target="_blank" class="btnT03 btnWFull" data-select-bank="埼玉りそな銀行" aria-hidden="true"><span>Webで<br class="spContent">事前審査</span></a>
						</li>
					</ul>
				</div>
			</div>
		</div>
	</div><!-- //.fixedBottom01 .jsFixedBottom01 -->
</div>

	<!-- FOOTER FIXED CONTENT END-->
</div>
<!-- 計測タグここから -->
<script>
(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','//www.google-analytics.com/analytics.js','ga');

<!--RB用 -->
ga('create', 'UA-45901352-1', 'auto');
ga('send', 'pageview');

<!--SR用 -->
ga('create', 'UA-45926600-1', 'auto', {'name':'sr'});
ga('sr.send', 'pageview');

</script>

<!--WebAntenna/RB -->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WATVaj-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WATVaj-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/RB -->

<!-- WebAntenna/SR-->
<script type="text/javascript" src="https://tr.webantenna.info/js/webantenna.js"></script>
<script type="text/javascript">
if ( typeof( webantenna ) == 'function' ) {
 _wa.account = 'WAbSBh-1';
 webantenna();
}
</script>
<noscript>
<img width="1" height="1" alt="" src="https://tr.webantenna.info/_webantenna.png?ga=WAbSBh-1&amp;jse=0">
</noscript>
<!-- end WebAntenna/SR-->
<!-- 計測タグここまで -->
<!--// WA 店舗検索 RB -->
<script>
	function wa_rb_kouza_shop_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WATVaj-1';
			_wa.cv = 'rb_kouza_shop_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 店舗検索 RB -->
<!--// WA Webで相談予約 SR -->
<script>
	function wa_sr_sevendays_soudan() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WAbSBh-1';
			_wa.cv = 'sr_sevendays_soudan';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA Webで相談予約 SR -->
<!--// WA 住宅ローン RB -->
<script>
	function wa_rb_jutaku_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WATVaj-1';
			_wa.cv = 'rb_jutaku_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 住宅ローン RB -->
<!--// WA 住宅ローン SR -->
<script>
	function wa_sr_jutaku_button() {
		if ( typeof( webantenna ) == 'function' ) {
			_wa = new WA_Processor();
			_wa.account = 'WAbSBh-1';
			_wa.cv = 'sr_jutaku_button';
			webantenna( true );
			var wait = function( msec ) {
				var s = ( new Date() ).getTime();
				while ( ( new Date ).getTime() - s < msec );
			}
			wait( 100 );
		}
	}
</script>
<!--// end WA 住宅ローン SR -->
</body>
</html>
"""

In [25]:
import re
from html.parser import HTMLParser
from html import unescape

class HTMLToMarkdownConverter(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.markdown = []
        self.list_stack = []
        self.list_item_count = 0
        self.code_block = False
        self.emphasis = False
        self.strong = False
        self.in_function = False    # 関数内かどうかを追跡
        self.in_script = False      # scriptタグ内かどうかを追跡
        self.in_excluded_tag = False # data-select-bank属性を持つタグ内かどうかを追跡
        
    def handle_starttag(self, tag, attrs):
        # 属性をdict形式に変換
        attrs = dict(attrs)
        
        # data-select-bank="埼玉りそな銀行"の属性を持つタグをチェック
        if 'data-select-bank' in attrs and attrs['data-select-bank'] == "埼玉りそな銀行":
            self.in_excluded_tag = True
            return

        # 除外されたタグ、関数内、scriptタグ内の場合は処理をスキップ
        if self.in_excluded_tag or self.in_function or self.in_script:
            return
            
        # scriptタグの開始
        if tag == 'script':
            self.in_script = True
            return
            
        if tag == 'h1':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('# ')
        elif tag == 'h2':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('## ')
        elif tag == 'h3':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('### ')
        elif tag == 'h4':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('#### ')
        elif tag == 'h5':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('##### ')
        elif tag == 'h6':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('###### ')
        elif tag == 'p':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
        elif tag == 'br':
            self.markdown.append('\n')
        elif tag == 'strong' or tag == 'b':
            self.strong = True
            self.markdown.append('**')
        elif tag == 'em' or tag == 'i':
            self.emphasis = True
            self.markdown.append('*')
        elif tag == 'code':
            content = ''.join(attrs.get('class', '')).lower()
            if 'function' in content or 'def' in content:
                self.in_function = True
            else:
                if 'class' in attrs:
                    self.code_block = True
                    self.markdown.append(f"\n```{attrs['class']}\n")
                else:
                    self.markdown.append('`')
        elif tag == 'pre':
            if not self.code_block and not self.in_function:
                self.code_block = True
                self.markdown.append('\n```\n')
        elif tag == 'ul':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.list_stack.append('*')
        elif tag == 'ol':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.list_stack.append('1')
            self.list_item_count = 0
        elif tag == 'li':
            # リストスタックが空の場合は処理をスキップ
            if not self.list_stack:
                return
            indent = '  ' * (len(self.list_stack) - 1)
            if self.list_stack[-1] == '*':
                self.markdown.append(f'\n{indent}* ')
            else:
                self.list_item_count += 1
                self.markdown.append(f'\n{indent}{self.list_item_count}. ')
        elif tag == 'blockquote':
            if self.markdown and self.markdown[-1] != '\n\n':
                self.markdown.append('\n\n')
            self.markdown.append('> ')
            
    def handle_endtag(self, tag):
        # 除外されたタグの終了をチェック
        if self.in_excluded_tag:
            self.in_excluded_tag = False
            return

        # scriptタグの終了
        if tag == 'script':
            self.in_script = False
            return
            
        if tag == 'code' and self.in_function:
            self.in_function = False
            return
            
        if self.in_function or self.in_script:
            return
            
        if tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            self.markdown.append('\n\n')
        elif tag == 'p':
            self.markdown.append('\n\n')
        elif tag == 'strong' or tag == 'b':
            self.strong = False
            self.markdown.append('**')
        elif tag == 'em' or tag == 'i':
            self.emphasis = False
            self.markdown.append('*')
        elif tag == 'code':
            if self.code_block:
                self.code_block = False
                self.markdown.append('\n```\n\n')
            else:
                self.markdown.append('`')
        elif tag == 'pre':
            if self.code_block:
                self.code_block = False
                self.markdown.append('\n```\n\n')
        elif tag in ['ul', 'ol']:
            # リストスタックが空でない場合のみpop処理を行う
            if self.list_stack:
                self.list_stack.pop()
                if not self.list_stack:
                    self.markdown.append('\n\n')
        elif tag == 'blockquote':
            self.markdown.append('\n\n')
            
    def handle_data(self, data):
        # 除外されたタグ、関数内、scriptタグ内の場合は処理をスキップ
        if self.in_excluded_tag or self.in_function or self.in_script:
            return
            
        if data.strip():
            # 'def'で始まる行か関数定義っぽい行をスキップ
            if not (data.strip().startswith('def ') or 
                   re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*\s*\([^)]*\)\s*:', data.strip())):
                self.markdown.append(data.strip())
            
    def convert(self, html):
        # 状態をリセット
        self.markdown = []
        self.list_stack = []
        self.list_item_count = 0
        self.code_block = False
        self.emphasis = False
        self.strong = False
        self.in_function = False
        self.in_script = False
        self.in_excluded_tag = False
        
        self.feed(html)
        markdown = ''.join(self.markdown)
        # 複数の空行を1つにまとめる
        markdown = re.sub(r'\n\s*\n', '\n\n', markdown)
        # 前後の余分な改行を削除
        markdown = markdown.strip()
        return markdown

def convert_html_to_markdown(html):
    """
    HTMLをMarkdownに変換する関数
    
    Parameters:
        html (str): 変換するHTML文字列
    
    Returns:
        str: 変換されたMarkdown文字列
    """
    converter = HTMLToMarkdownConverter()
    return converter.convert(html)

In [26]:
markdown = convert_html_to_markdown(html_example)
with open("test5.md", "w", encoding="utf-8") as f:
    f.write(markdown)

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LAParams

# LAParamsでテキストの読み取り方向を指定
laparams = LAParams(detect_vertical=True)  # 縦書きテキストの検出を有効化

# ページごとにテキストを抽出し、座標情報を使って適切にソート
def extract_sorted_text(pdf_path):
    text_blocks = []
    
    # ページ単位で処理
    for page_layout in extract_pages(pdf_path, laparams=laparams):
        page_texts = []
        
        # テキスト要素を抽出
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                # 座標情報とテキストを保存
                x0, y0, x1, y1 = element.bbox
                page_texts.append({
                    'text': element.get_text().strip(),
                    'y': -y0,  # Y座標を反転（上から下への順序にする）
                    'x': x0
                })
        
        # Y座標で先にソート（ページの上から下）
        # 同じY座標の場合はX座標でソート（左から右）
        page_texts.sort(key=lambda x: (x['y'], x['x']))
        
        # ソートされたテキストを追加
        text_blocks.extend([block['text'] for block in page_texts])
    
    return '\n'.join(text_blocks)

# 使用例
pdf_path = 'example.pdf'
sorted_text = extract_sorted_text(pdf_path)
print(sorted_text)